Our goal is to test whether heat waves and air pollution affect the number of hikers in the nature reserves and parks in Israel

In [173]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import calendar

In [174]:
Sites_df = pd.read_csv('https://raw.githubusercontent.com/EliranElizarov/Final_Project_Python/main/Sites1.csv')
VisitorsInSites_df = pd.read_csv('https://raw.githubusercontent.com/EliranElizarov/Final_Project_Python/main/VisitorsInSites1.csv')
Temp_df = pd.read_csv('https://raw.githubusercontent.com/EliranElizarov/Final_Project_Python/main/Temp1.csv')
Days_df = pd.read_csv('https://raw.githubusercontent.com/EliranElizarov/Final_Project_Python/main/Days.csv')
BeerSheva_Air_Quality_df = pd.read_csv('https://raw.githubusercontent.com/EliranElizarov/Final_Project_Python/main/BeerSheva11.csv')
Dalia_Air_Quality_df = pd.read_csv('https://raw.githubusercontent.com/EliranElizarov/Final_Project_Python/main/Dalia11.csv')
Haifa_Air_Quality_df = pd.read_csv('https://raw.githubusercontent.com/EliranElizarov/Final_Project_Python/main/Haifa11.csv')
Jerusalem_Air_Quality_df = pd.read_csv('https://raw.githubusercontent.com/EliranElizarov/Final_Project_Python/main/Jerusalem11.csv')
TelAviv_Air_Quality_df = pd.read_csv('https://raw.githubusercontent.com/EliranElizarov/Final_Project_Python/main/TelAviv11.csv')
UpperGalilee_Air_Quality_df = pd.read_csv('https://raw.githubusercontent.com/EliranElizarov/Final_Project_Python/main/UpperGalilee111.csv')


### **Data preparation**

In [175]:
Days_df.isnull().sum()

Date                       0
Is_Weekend                 0
Is_Jewish_Holiday        981
Is_Muslem_Holiday       1059
Is_Christian_Holiday    1074
Shabbaton               1093
dtype: int64

In [176]:
Days_cp = Days_df.copy()
Days_cp['Is_Jewish_Holiday'].fillna(value = 0,inplace = True)
Days_cp['Is_Muslem_Holiday'].fillna(value = 0,inplace = True)
Days_cp['Is_Christian_Holiday'].fillna(value = 0,inplace = True)
Days_cp['Shabbaton'].fillna(value = 0,inplace = True)
#Days_cp

In [177]:
merge1_df = pd.merge(left=VisitorsInSites_df, right=Days_cp, how='left')
#merge1_df.head(5)

In [178]:
merge2_df = pd.merge(left=merge1_df, right=Sites_df, how='left')
#merge2_df.head(5)

In [179]:
israeli_df = merge2_df[merge2_df['Visitors_type']=='Israeli']
#israeli_df.head(5)

In [180]:
israeli_df.drop('Visitors_type', axis=1, inplace=True)
israeli_df.rename({'Quantity': 'Israeli_Visitors'}, axis=1, inplace=True)
#israeli_df.head(5)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [181]:
tourist_df = merge2_df[merge2_df['Visitors_type']=='Tourist']
#tourist_df.head(5)

In [182]:
tourist_df.drop('Visitors_type', axis=1, inplace=True)
tourist_df.rename({'Quantity': 'Tourist_Visitors'}, axis=1, inplace=True)
#tourist_df.head(5)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [183]:
merge3_df = pd.merge(left=israeli_df, right=tourist_df,left_on=['Date', 'Site_Code'], right_on= ['Date', 'Site_Code'], how='left')
#merge3_df.head(5)

In [184]:
merge3_df.drop(['Is_Weekend_y','Is_Jewish_Holiday_y','Is_Muslem_Holiday_y','Is_Christian_Holiday_y','Shabbaton_y','Site_Name_y', 'Location_y', 'Temp_location_code_y', 'Is_water_area_y', 'Air_quality_location_y', 'Air_quality_Code_y', 'Is_National_Park_y', 'Is_Reserve_y', 'Is_Jewish_Site_y', 'Is_Christian_Site_y', 'Camping_y', 'Duration_Of_Visit_y'], axis=1, inplace=True)
#merge3_df.head(5)

In [185]:
merge3_df.rename({'Is_Weekend_x': 'Is_Weekend','Is_Jewish_Holiday_x':'Is_Jewish_Holiday','Is_Muslem_Holiday_x':'Is_Muslem_Holiday','Is_Christian_Holiday_x':'Is_Christian_Holiday','Shabbaton_x':'Shabbaton','Site_Name_x':'Site_Name','Location_x':'Location','Temp_location_code_x':'Temp_location_code','Is_water_area_x':'Is_water_area','Air_quality_location_x':'Air_quality_location','Air_quality_Code_x':'Air_quality_Code','Is_National_Park_x':'Is_National_Park','Is_Reserve_x':'Is_Reserve','Is_Jewish_Site_x':'Is_Jewish_Site','Is_Christian_Site_x':'Is_Christian_Site','Camping_x':'Camping','Duration_Of_Visit_x':'Duration_Of_Visit'}, axis=1, inplace=True)
#merge3_df.head(5)

In [186]:
merge4_df = pd.merge(left=merge3_df, right=Temp_df,left_on=['Date', 'Temp_location_code'], right_on= ['Date', 'Temp_location_code'], how='left')
#merge4_df.head(5)

In [187]:
UpperGalilee10_df = merge4_df[merge4_df['Air_quality_Code']==10]
BeerSheva3_df = merge4_df[merge4_df['Air_quality_Code']==3]
Dalia4_df = merge4_df[merge4_df['Air_quality_Code']==4]
Haifa5_df = merge4_df[merge4_df['Air_quality_Code']==5]
Jerusalem6_df = merge4_df[merge4_df['Air_quality_Code']==6]
TelAviv9_df = merge4_df[merge4_df['Air_quality_Code']==9]

In [188]:
mergeUpperGalilee10_df = pd.merge(left=UpperGalilee10_df, right=UpperGalilee_Air_Quality_df,left_on=['Date', 'Air_quality_Code'], right_on= ['Date', 'Air_quality_Code'], how='left')
mergeBeerSheva3_df = pd.merge(left=BeerSheva3_df, right=BeerSheva_Air_Quality_df,left_on=['Date', 'Air_quality_Code'], right_on= ['Date', 'Air_quality_Code'], how='left')
mergeDalia4_df = pd.merge(left=Dalia4_df, right=Dalia_Air_Quality_df,left_on=['Date', 'Air_quality_Code'], right_on= ['Date', 'Air_quality_Code'], how='left')
mergeHaifa5_df = pd.merge(left=Haifa5_df, right=Haifa_Air_Quality_df,left_on=['Date', 'Air_quality_Code'], right_on= ['Date', 'Air_quality_Code'], how='left')
mergeJerusalem6_df = pd.merge(left=Jerusalem6_df, right=Jerusalem_Air_Quality_df,left_on=['Date', 'Air_quality_Code'], right_on= ['Date', 'Air_quality_Code'], how='left')
mergeTelAviv9_df = pd.merge(left=TelAviv9_df, right=TelAviv_Air_Quality_df,left_on=['Date', 'Air_quality_Code'], right_on= ['Date', 'Air_quality_Code'], how='left')


In [189]:
MergeNum1 = pd.merge(left=mergeUpperGalilee10_df, right=mergeBeerSheva3_df, how='outer')

In [190]:
MergeNum2 = pd.merge(left=MergeNum1, right=mergeDalia4_df, how='outer')

In [191]:
MergeNum3 = pd.merge(left=MergeNum2, right=mergeHaifa5_df, how='outer')

In [192]:
MergeNum4 = pd.merge(left=MergeNum3, right=mergeJerusalem6_df, how='outer')

In [193]:
MergeNum5 = pd.merge(left=MergeNum4, right=mergeTelAviv9_df, how='outer')

In [194]:
#MergeNum5.head(5)

In [195]:
FinalTable_df = MergeNum5[['Date','Site_Code','Site_Name','Location','Israeli_Visitors','Tourist_Visitors','Temp_location_code','Temp_C','Air_quality_Code','Air_quality_location','pm25','pm10','no2','so2','Is_Weekend','Is_Jewish_Holiday','Is_Muslem_Holiday','Is_Christian_Holiday','Shabbaton','Is_water_area','Is_National_Park','Is_Reserve','Is_Jewish_Site','Is_Christian_Site','Camping','Duration_Of_Visit']]
FinalTable_df.head(5)

,Date,Site_Code,Site_Name,Location,Israeli_Visitors,Tourist_Visitors,Temp_location_code,Temp_C,Air_quality_Code,Air_quality_location,...,Is_Muslem_Holiday,Is_Christian_Holiday,Shabbaton,Is_water_area,Is_National_Park,Is_Reserve,Is_Jewish_Site,Is_Christian_Site,Camping,Duration_Of_Visit
0,1/1/2017,13,Bar'am,North,21,NaN,8643,15.2,10,Upper Galilee,...,0.0,1.0,0.0,no,1,0,1,0,0,1
1,1/1/2017,15,Tel Hatzor,North,19,9.0,8643,15.2,10,Upper Galilee,...,0.0,1.0,0.0,no,1,0,1,0,0,1
2,1/1/2017,16,Yechi'am,North,84,3.0,190,15.7,10,Upper Galilee,...,0.0,1.0,0.0,no,1,0,1,0,0,1
3,1/1/2017,17,Mivtzar Nimrod,North,251,79.0,8263,15.3,10,Upper Galilee,...,0.0,1.0,0.0,no,1,0,0,1,0,1
4,1/1/2017,18,Banias,North,788,713.0,8263,15.3,10,Upper Galilee,...,0.0,1.0,0.0,no,0,1,0,1,0,1


In [196]:
FinalTable_df['Tourist_Visitors'].fillna(value = int(0),inplace = True)
FinalTable_df.Is_water_area.replace({'no':0, 'yes':1}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [197]:
FinalTable_df[['Tourist_Visitors','Is_Jewish_Holiday','Is_Muslem_Holiday','Is_Christian_Holiday','Shabbaton']] = FinalTable_df[['Tourist_Visitors','Is_Jewish_Holiday','Is_Muslem_Holiday','Is_Christian_Holiday','Shabbaton']].astype(int)


### **Statistics and graphs**

In [198]:
FinalTable_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66103 entries, 0 to 66102
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date                  66103 non-null  object
 1   Site_Code             66103 non-null  int64 
 2   Site_Name             66103 non-null  object
 3   Location              66103 non-null  object
 4   Israeli_Visitors      66103 non-null  int64 
 5   Tourist_Visitors      66103 non-null  int64 
 6   Temp_location_code    66103 non-null  int64 
 7   Temp_C                63616 non-null  object
 8   Air_quality_Code      66103 non-null  int64 
 9   Air_quality_location  66103 non-null  object
 10  pm25                  65078 non-null  object
 11  pm10                  65023 non-null  object
 12  no2                   65003 non-null  object
 13  so2                   65058 non-null  object
 14  Is_Weekend            66103 non-null  int64 
 15  Is_Jewish_Holiday     66103 non-null

In [199]:
FinalTable_df = FinalTable_df.replace('-', np.nan)

In [200]:
FinalTable_df.pm25.unique()

array(['42', '37', '38', '39', '54', '51', '77', '78', '62', '59', '61',
       '47', '57', '66', '73', '58', '33', '46', '55', '67', '44', '31',
       '35', '48', '56', '65', '103', '69', '79', '86', '50', '36', '32',
       '41', '70', '81', '129', '83', '52', '43', ' ', '133', '45', '40',
       '53', '63', '64', '94', '88', '68', '76', '60', '75', '85', '49',
       '30', '80', '72', '71', '74', '100', '34', '29', '111', '87', '90',
       '104', '98', '96', '28', '27', '84', '82', '12', '25', '97', nan,
       '102', '105', '91', '23', '134', '115', '95', '92', '179', '116',
       '114', '118', '141', '117', '108', '211', '283', '164', '170',
       '142', '169', '236', '89', '113', '112', '146', '99', '167', '24',
       '26', '22', '21', '16', '17', '20', '19', '13', '14', '18', '93',
       '139', '120', '101', '106', '9', '8', '5', '6', '7', '130', '155',
       '187', '127', '124', '135', '126', '110', '122', '119', '123',
       '109', '15', '131', '138', '125', '107', '12

In [201]:
FinalTable_df = FinalTable_df.replace(' ', np.nan)

In [202]:
FinalTable_df['Temp_C'] = FinalTable_df['Temp_C'].astype(float)
FinalTable_df['pm25'] = FinalTable_df['pm25'].astype(float)
FinalTable_df['pm10'] = FinalTable_df['pm10'].astype(float)
FinalTable_df['no2'] = FinalTable_df['no2'].astype(float)
FinalTable_df['so2'] = FinalTable_df['so2'].astype(float)

In [203]:
FinalTable_df.head()

,Date,Site_Code,Site_Name,Location,Israeli_Visitors,Tourist_Visitors,Temp_location_code,Temp_C,Air_quality_Code,Air_quality_location,...,Is_Muslem_Holiday,Is_Christian_Holiday,Shabbaton,Is_water_area,Is_National_Park,Is_Reserve,Is_Jewish_Site,Is_Christian_Site,Camping,Duration_Of_Visit
0,1/1/2017,13,Bar'am,North,21,0,8643,15.2,10,Upper Galilee,...,0,1,0,0,1,0,1,0,0,1
1,1/1/2017,15,Tel Hatzor,North,19,9,8643,15.2,10,Upper Galilee,...,0,1,0,0,1,0,1,0,0,1
2,1/1/2017,16,Yechi'am,North,84,3,190,15.7,10,Upper Galilee,...,0,1,0,0,1,0,1,0,0,1
3,1/1/2017,17,Mivtzar Nimrod,North,251,79,8263,15.3,10,Upper Galilee,...,0,1,0,0,1,0,0,1,0,1
4,1/1/2017,18,Banias,North,788,713,8263,15.3,10,Upper Galilee,...,0,1,0,0,0,1,0,1,0,1


In [204]:
missing = FinalTable_df.isnull()
missing.sum()

Date                        0
Site_Code                   0
Site_Name                   0
Location                    0
Israeli_Visitors            0
Tourist_Visitors            0
Temp_location_code          0
Temp_C                   3415
Air_quality_Code            0
Air_quality_location        0
pm25                     4386
pm10                    27405
no2                      2753
so2                     40142
Is_Weekend                  0
Is_Jewish_Holiday           0
Is_Muslem_Holiday           0
Is_Christian_Holiday        0
Shabbaton                   0
Is_water_area               0
Is_National_Park            0
Is_Reserve                  0
Is_Jewish_Site              0
Is_Christian_Site           0
Camping                     0
Duration_Of_Visit           0
dtype: int64

In [205]:
FinalTable_df.duplicated().sum()

0

In [206]:
FinalTable_df.dtypes

Date                     object
Site_Code                 int64
Site_Name                object
Location                 object
Israeli_Visitors          int64
Tourist_Visitors          int64
Temp_location_code        int64
Temp_C                  float64
Air_quality_Code          int64
Air_quality_location     object
pm25                    float64
pm10                    float64
no2                     float64
so2                     float64
Is_Weekend                int64
Is_Jewish_Holiday         int64
Is_Muslem_Holiday         int64
Is_Christian_Holiday      int64
Shabbaton                 int64
Is_water_area             int64
Is_National_Park          int64
Is_Reserve                int64
Is_Jewish_Site            int64
Is_Christian_Site         int64
Camping                   int64
Duration_Of_Visit         int64
dtype: object

In [207]:
corr_df = FinalTable_df.corr()
corr_df

,Site_Code,Israeli_Visitors,Tourist_Visitors,Temp_location_code,Temp_C,Air_quality_Code,pm25,pm10,no2,so2,...,Is_Muslem_Holiday,Is_Christian_Holiday,Shabbaton,Is_water_area,Is_National_Park,Is_Reserve,Is_Jewish_Site,Is_Christian_Site,Camping,Duration_Of_Visit
Site_Code,1.000000,0.018369,0.092374,0.234676,-0.062081,-0.672285,-0.012304,0.043027,0.155388,0.024780,...,-0.003407,0.003611,-0.000085,-0.094098,-0.014189,-0.197228,-0.019442,-0.142206,0.155093,0.197844
Israeli_Visitors,0.018369,1.000000,0.141178,-0.014050,0.026184,0.012532,0.023086,0.002445,0.038035,-0.019796,...,0.012540,0.021438,0.017009,0.080242,0.026678,0.007431,-0.009131,0.016409,0.056831,0.159633
Tourist_Visitors,0.092374,0.141178,1.000000,0.122462,0.008548,-0.139743,-0.041156,0.017623,-0.062523,0.017971,...,-0.011677,-0.008379,0.007643,-0.160092,0.121509,-0.059458,0.099566,0.257378,0.010112,0.059483
Temp_location_code,0.234676,-0.014050,0.122462,1.000000,0.153737,-0.005746,-0.083015,0.021015,-0.141664,0.020143,...,-0.002911,0.000100,-0.001158,-0.067987,-0.066968,-0.010501,0.065138,-0.098458,0.010931,-0.042343
Temp_C,-0.062081,0.026184,0.008548,0.153737,1.000000,0.023273,0.151978,-0.007236,-0.194294,-0.084611,...,0.132188,-0.138133,0.015656,0.069643,-0.028333,0.082777,0.012258,-0.010675,-0.014028,-0.013233
Air_quality_Code,-0.672285,0.012532,-0.139743,-0.005746,0.023273,1.000000,0.122077,-0.013195,0.140965,-0.138809,...,0.001519,-0.002695,-0.000118,0.190732,-0.024974,0.190131,0.016585,0.177659,-0.192980,-0.080607
pm25,-0.012304,0.023086,-0.041156,-0.083015,0.151978,0.122077,1.000000,0.304325,0.200389,-0.011132,...,0.012542,-0.029998,0.008156,0.033666,0.037715,-0.022082,-0.025042,0.053393,0.004462,0.070196
pm10,0.043027,0.002445,0.017623,0.021015,-0.007236,-0.013195,0.304325,1.000000,0.077304,0.098977,...,-0.034021,0.010432,-0.005148,0.002281,0.009733,-0.018224,-0.006341,0.008977,0.015530,0.017416
no2,0.155388,0.038035,-0.062523,-0.141664,-0.194294,0.140965,0.200389,0.077304,1.000000,0.153108,...,-0.024824,-0.002081,-0.002448,0.027666,0.054324,-0.075386,-0.027135,0.095139,-0.025141,0.131466
so2,0.024780,-0.019796,0.017971,0.020143,-0.084611,-0.138809,-0.011132,0.098977,0.153108,1.000000,...,-0.043686,-0.010485,0.010592,-0.059664,-0.063001,0.049949,0.045406,-0.097329,-0.014420,-0.050312


In [208]:
fig = go.Figure()
fig.add_traces(go.Heatmap(
    z=corr_df, 
    x= corr_df.columns,  
    y= corr_df.columns,
))
fig.update_layout({
    'title':"Dataframe Correlation Heatmap"
})

fig.show()

In [209]:
for i in range(len(corr_df.columns)-1):
  print(corr_df.columns[i], 'is correlated to: ')
  for j in range(len(corr_df.columns)-1):
    if abs(corr_df.iloc[i,j])>=0.85 and i is not j:
      print(corr_df.columns[j], end=', ')
  print('\n----------------------')

Site_Code is correlated to: 

----------------------
Israeli_Visitors is correlated to: 

----------------------
Tourist_Visitors is correlated to: 

----------------------
Temp_location_code is correlated to: 

----------------------
Temp_C is correlated to: 

----------------------
Air_quality_Code is correlated to: 

----------------------
pm25 is correlated to: 

----------------------
pm10 is correlated to: 

----------------------
no2 is correlated to: 

----------------------
so2 is correlated to: 

----------------------
Is_Weekend is correlated to: 

----------------------
Is_Jewish_Holiday is correlated to: 

----------------------
Is_Muslem_Holiday is correlated to: 

----------------------
Is_Christian_Holiday is correlated to: 

----------------------
Shabbaton is correlated to: 

----------------------
Is_water_area is correlated to: 

----------------------
Is_National_Park is correlated to: 

----------------------
Is_Reserve is correlated to: 

----------------------
I

As we can see, there is no correlated features

In [210]:
# כל השורות שיש בהן ערכים חסרים
FinalTable_df[missing.any(axis=1)]

,Date,Site_Code,Site_Name,Location,Israeli_Visitors,Tourist_Visitors,Temp_location_code,Temp_C,Air_quality_Code,Air_quality_location,...,Is_Muslem_Holiday,Is_Christian_Holiday,Shabbaton,Is_water_area,Is_National_Park,Is_Reserve,Is_Jewish_Site,Is_Christian_Site,Camping,Duration_Of_Visit
16,1/1/2017,71,Hamat Tveria,North,32,4,911,NaN,10,Upper Galilee,...,0,1,0,1,1,0,1,0,0,1
30,1/2/2017,71,Hamat Tveria,North,43,0,911,NaN,10,Upper Galilee,...,0,0,0,1,1,0,1,0,0,1
44,1/3/2017,71,Hamat Tveria,North,22,0,911,NaN,10,Upper Galilee,...,0,0,0,1,1,0,1,0,0,1
59,1/4/2017,71,Hamat Tveria,North,42,85,911,NaN,10,Upper Galilee,...,0,0,0,1,1,0,1,0,0,1
74,1/5/2017,71,Hamat Tveria,North,32,0,911,NaN,10,Upper Galilee,...,0,0,0,1,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66098,12/31/2019,153,Tel Afek,Central,284,0,2410,17.2,9,Tel Aviv,...,0,0,0,1,1,0,0,1,1,2
66099,12/31/2019,158,Apolonia,Central,118,7,2410,17.2,9,Tel Aviv,...,0,0,0,0,1,0,0,1,0,1
66100,12/31/2019,159,Beit Govrin,Central,423,151,3541,17.7,9,Tel Aviv,...,0,0,0,0,1,0,0,0,0,2
66101,12/31/2019,162,Ashkelon,Central,6313,5,3541,17.7,9,Tel Aviv,...,0,0,0,1,1,0,0,1,1,3


In [211]:
Metzada_df = FinalTable_df.loc[FinalTable_df['Site_Code'] == 258]

In [212]:
Megido_df = FinalTable_df.loc[FinalTable_df['Site_Code'] == 82]

In [213]:
Gamla_df = FinalTable_df.loc[FinalTable_df['Site_Code'] == 23]

In [214]:
Banias_df = FinalTable_df.loc[FinalTable_df['Site_Code'] == 18]

In [215]:
fig1 = go.Figure(data=go.Scatter(x=Metzada_df.Date, y=Metzada_df.Israeli_Visitors, mode='markers+lines'))
fig1.add_trace(go.Scatter(x=Metzada_df.Date, y=Metzada_df.Israeli_Visitors,
                    mode='lines',
                    name='Israeli Visitors'))

fig1.add_trace(go.Scatter(x=Metzada_df.Date, y=Metzada_df.Tourist_Visitors,
                    mode='markers',
                    marker_line_width=1,
                    marker_size=6,
                    name='Tourist Visitors'))

fig1.update_layout(
    title="Israeli and Tourist Visitors in Metzada site",
    xaxis_title="Date",
    yaxis_title="Visitors",
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="RebeccaPurple"
    )
)

fig1.show()

In [216]:
fig1 = go.Figure(data=go.Scatter(x=Gamla_df.Date, y=Gamla_df.Israeli_Visitors, mode='markers+lines'))
fig1.add_trace(go.Scatter(x=Gamla_df.Date, y=Gamla_df.Israeli_Visitors,
                    mode='lines',
                    name='Israeli Visitors'))

fig1.add_trace(go.Scatter(x=Gamla_df.Date, y=Gamla_df.Tourist_Visitors,
                    mode='markers',
                    marker_line_width=1,
                    marker_size=6,
                    name='Tourist Visitors'))

fig1.update_layout(
    title="Israeli and Tourist Visitors in Gamla site",
    xaxis_title="Date",
    yaxis_title="Visitors",
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="RebeccaPurple"
    )
)

fig1.show()

In [217]:
fig1 = go.Figure(data=go.Scatter(x=Banias_df.Date, y=Banias_df.Israeli_Visitors, mode='markers+lines'))
fig1.add_trace(go.Scatter(x=Banias_df.Date, y=Banias_df.Israeli_Visitors,
                    mode='lines',
                    name='Israeli Visitors'))

fig1.add_trace(go.Scatter(x=Banias_df.Date, y=Banias_df.Tourist_Visitors,
                    mode='markers',
                    marker_line_width=1,
                    marker_size=6,
                    name='Tourist Visitors'))

fig1.update_layout(
    title="Israeli and Tourist Visitors in Banias site",
    xaxis_title="Date",
    yaxis_title="Visitors",
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="RebeccaPurple"
    )
)

fig1.show()

In [218]:
fig1 = go.Figure(data=go.Scatter(x=Megido_df.Date, y=Megido_df.Israeli_Visitors, mode='markers+lines'))
fig1.add_trace(go.Scatter(x=Megido_df.Date, y=Megido_df.Israeli_Visitors,
                    mode='lines',
                    name='Israeli Visitors'))

fig1.add_trace(go.Scatter(x=Megido_df.Date, y=Megido_df.Tourist_Visitors,
                    mode='markers',
                    marker_line_width=1,
                    marker_size=6,
                    name='Tourist Visitors'))

fig1.update_layout(
    title="Israeli and Tourist Visitors in Megido site",
    xaxis_title="Date",
    yaxis_title="Visitors",
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="RebeccaPurple"
    )
)

fig1.show()

In [219]:
pm25_df = FinalTable_df.copy()
pm25_df.loc[pm25_df['pm25'] >= 37.5, 'pm25'] = 1
pm25_df.loc[pm25_df['pm25'] != 1, 'pm25'] = 0

In [220]:
numofcases = pm25_df.pm25.value_counts()
numofcases

1.0    56132
0.0     9971
Name: pm25, dtype: int64

In [221]:
precentage = [str(round(i/numofcases.sum()*100,2)) +'%' for i in numofcases]
precentage

['84.92%', '15.08%']

In [222]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=['Yes', 'No'],
    y=numofcases,
    marker_color=['red','green'],
    text=precentage,
    textposition='outside',
)) 
fig.update_layout(
    title='pm25 Air Pollution Days', 
    yaxis_title='Number of cases',
    )
fig.show()

In [223]:
pm10_df = FinalTable_df.copy()
pm10_df.loc[pm10_df['pm10'] >= 130, 'pm10'] = 1
pm10_df.loc[pm10_df['pm10'] != 1, 'pm10'] = 0

In [224]:
numofcases2 = pm10_df.pm10.value_counts()
numofcases2

0.0    65584
1.0      519
Name: pm10, dtype: int64

In [225]:
precentage2 = [str(round(i/numofcases2.sum()*100,2)) +'%' for i in numofcases2]
precentage2

['99.21%', '0.79%']

In [226]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=['No', 'Yes'],
    y=numofcases2,
    marker_color=['green','red'],
    text=precentage2,
    textposition='outside',
)) 
fig.update_layout(
    title='pm10 Air Pollution Days', 
    yaxis_title='Number of cases',
    )
fig.show()

In [227]:
no2_df = FinalTable_df.copy()
no2_df.loc[no2_df['no2'] >= 200, 'no2'] = 1
no2_df.loc[no2_df['no2'] != 1, 'no2'] = 0

In [228]:
numofcases3 = no2_df.no2.value_counts()
numofcases3

0.0    64176
1.0     1927
Name: no2, dtype: int64

In [229]:
precentage3 = [str(round(i/numofcases3.sum()*100,2)) +'%' for i in numofcases3]
precentage3

['97.08%', '2.92%']

In [230]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=['No', 'Yes'],
    y=numofcases3,
    marker_color=['green','red'],
    text=precentage3,
    textposition='outside',
)) 
fig.update_layout(
    title='no2 Air Pollution Days', 
    yaxis_title='Number of cases',
    )
fig.show()

In [231]:
so2_df = FinalTable_df.copy()
so2_df.loc[so2_df['so2'] >= 50, 'so2'] = 1
so2_df.loc[so2_df['so2'] != 1, 'so2'] = 0

In [232]:
numofcases4 = so2_df.so2.value_counts()
numofcases4

0.0    55762
1.0    10341
Name: so2, dtype: int64

In [233]:
precentage4 = [str(round(i/numofcases4.sum()*100,2)) +'%' for i in numofcases4]
precentage4

['84.36%', '15.64%']

In [234]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=['No', 'Yes'],
    y=numofcases4,
    marker_color=['green','red'],
    text=precentage4,
    textposition='outside',
)) 
fig.update_layout(
    title='so2 Air Pollution Days', 
    yaxis_title='Number of cases',
    )
fig.show()

In [235]:
df1 = FinalTable_df.copy()
df2 = FinalTable_df.copy()
df3 = FinalTable_df.copy()
df4 = FinalTable_df.copy()
df5 = FinalTable_df.copy()
df6 = FinalTable_df.copy()
df7 = FinalTable_df.copy()
df8 = FinalTable_df.copy()
df9 = FinalTable_df.copy()
df10 = FinalTable_df.copy()
#df.dropna(axis=0, inplace=True)
#ForT_Test = df.reset_index()

In [236]:
HotDays_df = FinalTable_df.loc[FinalTable_df['Temp_C'] >= 30]

In [237]:
df1 = df1.loc[df1['pm10'] >= 130]
df2 = df2.loc[df2['pm10'] < 130]
df3 = df3.loc[df3['pm25'] >= 37.5]
df4 = df4.loc[df4['pm25'] < 37.5]
df5 = df5.loc[df5['no2'] >= 200]
df6 = df6.loc[df6['no2'] < 200]
df7 = df7.loc[df7['so2'] >= 50]
df8 = df8.loc[df8['so2'] < 50]
df9 = df9.loc[df9['Temp_C'] >= 30]
df10 = df10.loc[df10['Temp_C'] < 30]

In [238]:
Israeli1 = df1[['Israeli_Visitors']]
Israeli2 = df2[['Israeli_Visitors']]
Tourist1 = df1[['Tourist_Visitors']]
Tourist2 = df2[['Tourist_Visitors']]
Israeli3 = df3[['Israeli_Visitors']]
Israeli4 = df4[['Israeli_Visitors']]
Tourist3 = df3[['Tourist_Visitors']]
Tourist4 = df4[['Tourist_Visitors']]
Israeli5 = df5[['Israeli_Visitors']]
Israeli6 = df6[['Israeli_Visitors']]
Tourist5 = df5[['Tourist_Visitors']]
Tourist6 = df6[['Tourist_Visitors']]
Israeli7 = df7[['Israeli_Visitors']]
Israeli8 = df8[['Israeli_Visitors']]
Tourist7 = df7[['Tourist_Visitors']]
Tourist8 = df8[['Tourist_Visitors']]
Israeli9 = df9[['Israeli_Visitors']]
Israeli10 = df10[['Israeli_Visitors']]
Tourist9 = df9[['Tourist_Visitors']]
Tourist10 = df10[['Tourist_Visitors']]

In [239]:
israeliWithPm10 = Israeli1["Israeli_Visitors"].mean()
israeliWithoutPm10 = Israeli2["Israeli_Visitors"].mean()
touristsWithPm10 = Tourist1["Tourist_Visitors"].mean()
touristsWithoutPm10 = Tourist2["Tourist_Visitors"].mean()

israeliWithPm25 = Israeli3["Israeli_Visitors"].mean()
israeliWithoutPm25 = Israeli4["Israeli_Visitors"].mean()
touristsWithPm25 = Tourist3["Tourist_Visitors"].mean()
touristsWithoutPm25 = Tourist4["Tourist_Visitors"].mean()

israeliWithSo2 = Israeli7["Israeli_Visitors"].mean()
israeliWithoutSo2 = Israeli8["Israeli_Visitors"].mean()
touristsWithSo2 = Tourist7["Tourist_Visitors"].mean()
touristsWithoutSo2 = Tourist8["Tourist_Visitors"].mean()

israeliWithHeat = Israeli9["Israeli_Visitors"].mean()
israeliWithoutHeat = Israeli10["Israeli_Visitors"].mean()
touristsWithHeat = Tourist9["Tourist_Visitors"].mean()
touristsWithoutHeat = Tourist10["Tourist_Visitors"].mean()

israeliWithoutPm25

294.6229185317816

In [240]:
name = ["Israeli_Visitors","Tourist_Visitors"] 

In [241]:
t_value,p_value=stats.ttest_ind(Israeli1,Israeli2)
print('\n'+'1)'+' Test statistic of pm10 is on '+name[0], str((t_value)))
print('p-value for two tailed test is ',str(p_value))
alpha = 0.05
if p_value<=alpha:
  print('Conclusion: Since p-value(=%f)'%p_value,'<','alpha(=%.2f)'%alpha,'We reject the null hypothesis H0.')
else:
  print('Conclusion: Since p-value(=%f)'%p_value,'>','alpha(=%.2f)'%alpha,'We do not reject the null hypothesis H0.')

t_value,p_value=stats.ttest_ind(Tourist1,Tourist2)
print('\n'+'2)'+' Test statistic of pm10 is on '+name[1], str((t_value)))
print('p-value for two tailed test is ',str(p_value))
alpha = 0.05
if p_value<=alpha:
  print('Conclusion: Since p-value(=%f)'%p_value,'<','alpha(=%.2f)'%alpha,'We reject the null hypothesis H0.')
else:
  print('Conclusion: Since p-value(=%f)'%p_value,'>','alpha(=%.2f)'%alpha,'We do not reject the null hypothesis H0.')



1) Test statistic of pm10 is on Israeli_Visitors [-0.67637423]
p-value for two tailed test is  [0.49880712]
Conclusion: Since p-value(=0.498807) > alpha(=0.05) We do not reject the null hypothesis H0.

2) Test statistic of pm10 is on Tourist_Visitors [1.31457311]
p-value for two tailed test is  [0.1886612]
Conclusion: Since p-value(=0.188661) > alpha(=0.05) We do not reject the null hypothesis H0.


In [242]:
t_value,p_value=stats.ttest_ind(Israeli3,Israeli4)
print('\n'+'1)'+' Test statistic of pm2.5 is on '+name[0], str((t_value)))
print('p-value for two tailed test is ',str(p_value))
alpha = 0.05
if p_value<=alpha:
  print('Conclusion: Since p-value(=%f)'%p_value,'<','alpha(=%.2f)'%alpha,'We reject the null hypothesis H0.')
else:
  print('Conclusion: Since p-value(=%f)'%p_value,'>','alpha(=%.2f)'%alpha,'We do not reject the null hypothesis H0.')

t_value,p_value=stats.ttest_ind(Tourist3,Tourist4)
print('\n'+'2)'+' Test statistic of pm.25 is on '+name[1], str((t_value)))
print('p-value for two tailed test is ',str(p_value))
alpha = 0.05
if p_value<=alpha:
  print('Conclusion: Since p-value(=%f)'%p_value,'<','alpha(=%.2f)'%alpha,'We reject the null hypothesis H0.')
else:
  print('Conclusion: Since p-value(=%f)'%p_value,'>','alpha(=%.2f)'%alpha,'We do not reject the null hypothesis H0.')


1) Test statistic of pm2.5 is on Israeli_Visitors [3.88861721]
p-value for two tailed test is  [0.00010092]
Conclusion: Since p-value(=0.000101) < alpha(=0.05) We reject the null hypothesis H0.

2) Test statistic of pm.25 is on Tourist_Visitors [-5.53012435]
p-value for two tailed test is  [3.2129671e-08]
Conclusion: Since p-value(=0.000000) < alpha(=0.05) We reject the null hypothesis H0.


In [243]:
t_value,p_value=stats.ttest_ind(Israeli5,Israeli6)
print('\n'+'1)'+' Test statistic of no2 is on '+name[0], str((t_value)))
print('p-value for two tailed test is ',str(p_value))
alpha = 0.05
if p_value<=alpha:
  print('Conclusion: Since p-value(=%f)'%p_value,'<','alpha(=%.2f)'%alpha,'We reject the null hypothesis H0.')
else:
  print('Conclusion: Since p-value(=%f)'%p_value,'>','alpha(=%.2f)'%alpha,'We do not reject the null hypothesis H0.')

t_value,p_value=stats.ttest_ind(Tourist5,Tourist6)
print('\n'+'2)'+' Test statistic of no2 is on '+name[1], str((t_value)))
print('p-value for two tailed test is ',str(p_value))
alpha = 0.05
if p_value<=alpha:
  print('Conclusion: Since p-value(=%f)'%p_value,'<','alpha(=%.2f)'%alpha,'We reject the null hypothesis H0.')
else:
  print('Conclusion: Since p-value(=%f)'%p_value,'>','alpha(=%.2f)'%alpha,'We do not reject the null hypothesis H0.')


1) Test statistic of no2 is on Israeli_Visitors nan
p-value for two tailed test is  nan
Conclusion: Since p-value(=nan) > alpha(=0.05) We do not reject the null hypothesis H0.

2) Test statistic of no2 is on Tourist_Visitors nan
p-value for two tailed test is  nan
Conclusion: Since p-value(=nan) > alpha(=0.05) We do not reject the null hypothesis H0.


In [244]:
t_value,p_value=stats.ttest_ind(Israeli7,Israeli8)
print('\n'+'1)'+' Test statistic of so2 is on '+name[0], str((t_value)))
print('p-value for two tailed test is ',str(p_value))
alpha = 0.05
if p_value<=alpha:
  print('Conclusion: Since p-value(=%f)'%p_value,'<','alpha(=%.2f)'%alpha,'We reject the null hypothesis H0.')
else:
  print('Conclusion: Since p-value(=%f)'%p_value,'>','alpha(=%.2f)'%alpha,'We do not reject the null hypothesis H0.')

t_value,p_value=stats.ttest_ind(Tourist7,Tourist8)
print('\n'+'2)'+' Test statistic of so2 is on '+name[1], str((t_value)))
print('p-value for two tailed test is ',str(p_value))
alpha = 0.05
if p_value<=alpha:
  print('Conclusion: Since p-value(=%f)'%p_value,'<','alpha(=%.2f)'%alpha,'We reject the null hypothesis H0.')
else:
  print('Conclusion: Since p-value(=%f)'%p_value,'>','alpha(=%.2f)'%alpha,'We do not reject the null hypothesis H0.')


1) Test statistic of so2 is on Israeli_Visitors [-0.29066368]
p-value for two tailed test is  [0.77131088]
Conclusion: Since p-value(=0.771311) > alpha(=0.05) We do not reject the null hypothesis H0.

2) Test statistic of so2 is on Tourist_Visitors [-0.38601056]
p-value for two tailed test is  [0.69949201]
Conclusion: Since p-value(=0.699492) > alpha(=0.05) We do not reject the null hypothesis H0.


In [245]:
t_value,p_value=stats.ttest_ind(Israeli9,Israeli10)
print('\n'+'1)'+' Test statistic of Temp_C is on '+name[0], str((t_value)))
print('p-value for two tailed test is ',str(p_value))
alpha = 0.05
if p_value<=alpha:
  print('Conclusion: Since p-value(=%f)'%p_value,'<','alpha(=%.2f)'%alpha,'We reject the null hypothesis H0.')
else:
  print('Conclusion: Since p-value(=%f)'%p_value,'>','alpha(=%.2f)'%alpha,'We do not reject the null hypothesis H0.')

t_value,p_value=stats.ttest_ind(Tourist9,Tourist10)
print('\n'+'2)'+' Test statistic of Temp_C is on '+name[1], str((t_value)))
print('p-value for two tailed test is ',str(p_value))
alpha = 0.05
if p_value<=alpha:
  print('Conclusion: Since p-value(=%f)'%p_value,'<','alpha(=%.2f)'%alpha,'We reject the null hypothesis H0.')
else:
  print('Conclusion: Since p-value(=%f)'%p_value,'>','alpha(=%.2f)'%alpha,'We do not reject the null hypothesis H0.')


1) Test statistic of Temp_C is on Israeli_Visitors [-2.17192486]
p-value for two tailed test is  [0.02986507]
Conclusion: Since p-value(=0.029865) < alpha(=0.05) We reject the null hypothesis H0.

2) Test statistic of Temp_C is on Tourist_Visitors [-3.16860741]
p-value for two tailed test is  [0.00153245]
Conclusion: Since p-value(=0.001532) < alpha(=0.05) We reject the null hypothesis H0.


In [246]:
EnterPerSite = merge1_df.groupby(['Visitors_type','Site_Code'])['Quantity'].count().unstack()
EnterPerSite


Site_Code,12,13,14,15,16,17,18,19,20,21,...,259,260,264,266,267,269,272,273,301,303
Visitors_type,,,,,,,,,,,,,,,,,,,,,
Israeli,777.0,1050.0,809.0,1056.0,1069.0,1090.0,1092.0,1092.0,1091.0,975.0,...,210.0,140.0,1021.0,1011.0,1084.0,1075.0,1074.0,1064.0,1089.0,1089.0
Tourist,121.0,324.0,85.0,1009.0,358.0,1072.0,1092.0,1088.0,269.0,696.0,...,NaN,NaN,890.0,800.0,1087.0,1062.0,1047.0,36.0,1090.0,877.0


In [247]:
fig = make_subplots(rows=1, cols=4, specs=[[{'type':'domain'},{'type':'domain'},{'type':'domain'},{'type':'domain'}]])
fig.add_trace(go.Pie(labels=['Israeli', 'Tourist'], values=EnterPerSite[82], textinfo='label + percent', name='Megido', title='Megido'), 1, 1)
fig.add_trace(go.Pie(labels=['Israeli', 'Tourist'], values=EnterPerSite[18], textinfo='label + percent', name='Banias', title='Banias'), 1, 2)
fig.add_trace(go.Pie(labels=['Israeli', 'Tourist'], values=EnterPerSite[23], textinfo='label + percent', name='Gamla', title='Gamla'), 1, 3)
fig.add_trace(go.Pie(labels=['Israeli', 'Tourist'], values=EnterPerSite[258], textinfo='label + percent', name='Metzada', title='Metzada'), 1, 4)
fig.show()

### **Build Megido, Banias, Gamla and Metzada Regression Tables**

In [248]:
FinalTable_df['Date'] = pd.to_datetime(FinalTable_df['Date'])

In [249]:
FinalTable_df['month'] = pd.Categorical.from_codes(
    FinalTable_df['Date'].dt.month - 1,
    categories=list(calendar.month_name),
    ordered=True
)

FinalTable_df['season'] = pd.Categorical.from_codes(
    FinalTable_df['Date'].dt.month % 12 // 3,
    categories=['Winter', 'Spring', 'Summer', 'Fall'],
    ordered=True
)

FinalTable_df = pd.get_dummies(FinalTable_df, columns=['season'], prefix_sep='', prefix='')

In [250]:
FinalTable_df.drop(['Fall','month'], axis=1, inplace=True)


In [251]:
MegidoIsraeli_df = FinalTable_df.loc[FinalTable_df['Site_Name'] == 'Megido']
MegidoTourist_df = FinalTable_df.loc[FinalTable_df['Site_Name'] == 'Megido']

BaniasIsraeli_df = FinalTable_df.loc[FinalTable_df['Site_Name'] == 'Banias']
BaniasTourist_df = FinalTable_df.loc[FinalTable_df['Site_Name'] == 'Banias']

GamlaIsraeli_df = FinalTable_df.loc[FinalTable_df['Site_Name'] == 'Gamla']
GamlaTourist_df = FinalTable_df.loc[FinalTable_df['Site_Name'] == 'Gamla']

MetzadaIsraeli_df = FinalTable_df.loc[FinalTable_df['Site_Name'] == 'Metzada']
MetzadaTourist_df = FinalTable_df.loc[FinalTable_df['Site_Name'] == 'Metzada']


In [252]:
MegidoIsraeli_df.drop(['Date','Site_Code','Temp_location_code','Air_quality_Code','Site_Name','Location','Tourist_Visitors','Air_quality_location','Shabbaton','Is_water_area','Is_Reserve','Camping','Is_Jewish_Site'], axis=1, inplace=True)
MegidoTourist_df.drop(['Date','Site_Code','Temp_location_code','Air_quality_Code','Site_Name','Location','Israeli_Visitors','Air_quality_location','Shabbaton','Is_water_area','Is_Reserve','Camping','Is_Jewish_Site'], axis=1, inplace=True)

BaniasIsraeli_df.drop(['Date','Site_Code','Temp_location_code','Air_quality_Code','Site_Name','Location','Tourist_Visitors','Air_quality_location','Shabbaton','Is_National_Park','Camping','Is_Jewish_Site'], axis=1, inplace=True)
BaniasTourist_df.drop(['Date','Site_Code','Temp_location_code','Air_quality_Code','Site_Name','Location','Israeli_Visitors','Air_quality_location','Shabbaton','Is_National_Park','Camping','Is_Jewish_Site'], axis=1, inplace=True)

GamlaIsraeli_df.drop(['Date','Site_Code','Temp_location_code','Air_quality_Code','Site_Name','Location','Tourist_Visitors','Air_quality_location','Shabbaton','Is_water_area','Is_National_Park','Camping','Is_Christian_Site'], axis=1, inplace=True)
GamlaTourist_df.drop(['Date','Site_Code','Temp_location_code','Air_quality_Code','Site_Name','Location','Israeli_Visitors','Air_quality_location','Shabbaton','Is_water_area','Is_National_Park','Camping','Is_Christian_Site'], axis=1, inplace=True)

MetzadaIsraeli_df.drop(['Date','Site_Code','Temp_location_code','Air_quality_Code','Site_Name','Location','Tourist_Visitors','Air_quality_location','Shabbaton','Is_water_area','Is_Reserve','Is_Christian_Site'], axis=1, inplace=True)
MetzadaTourist_df.drop(['Date','Site_Code','Temp_location_code','Air_quality_Code','Site_Name','Location','Israeli_Visitors','Air_quality_location','Shabbaton','Is_water_area','Is_Reserve','Is_Christian_Site'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [253]:
#Megido Israeli
MegidoIsraelipm25_df = MegidoIsraeli_df.copy()
MegidoIsraelino2_df = MegidoIsraeli_df.copy()
#Megido Tourist
MegidoTouristpm25_df = MegidoTourist_df.copy()
MegidoTouristno2_df = MegidoTourist_df.copy()

#Banias Israeli
BaniasIsraelipm25_df = BaniasIsraeli_df.copy()
BaniasIsraelino2_df = BaniasIsraeli_df.copy()
#Banias Tourist
BaniasTouristpm25_df = BaniasTourist_df.copy()
BaniasTouristno2_df = BaniasTourist_df.copy()

#Gamla Israeli
GamlaIsraelipm25_df = GamlaIsraeli_df.copy()
GamlaIsraelino2_df = GamlaIsraeli_df.copy()
#Gamla Tourist
GamlaTouristpm25_df = GamlaTourist_df.copy()
GamlaTouristno2_df = GamlaTourist_df.copy()

#Metzada Israeli
MetzadaIsraelipm25_df = MetzadaIsraeli_df.copy()
MetzadaIsraelipm10_df = MetzadaIsraeli_df.copy()
MetzadaIsraelino2_df = MetzadaIsraeli_df.copy()
#Metzada Tourist
MetzadaTouristpm25_df = MetzadaTourist_df.copy()
MetzadaTouristpm10_df = MetzadaTourist_df.copy()
MetzadaTouristno2_df = MetzadaTourist_df.copy()

In [254]:
#Megido Israeli
MegidoIsraelipm25_df.drop(['pm10','no2','so2'], axis=1, inplace=True)
MegidoIsraelino2_df.drop(['pm25','pm10','so2'], axis=1, inplace=True)
#Megido Tourist
MegidoTouristpm25_df.drop(['pm10','no2','so2'], axis=1, inplace=True)
MegidoTouristno2_df.drop(['pm25','pm10','so2'], axis=1, inplace=True)

#Banias Israeli
BaniasIsraelipm25_df.drop(['pm10','no2','so2'], axis=1, inplace=True)
BaniasIsraelino2_df.drop(['pm25','pm10','so2'], axis=1, inplace=True)
#Banias Tourist
BaniasTouristpm25_df.drop(['pm10','no2','so2'], axis=1, inplace=True)
BaniasTouristno2_df.drop(['pm25','pm10','so2'], axis=1, inplace=True)

#Gamla Israeli
GamlaIsraelipm25_df.drop(['pm10','no2','so2'], axis=1, inplace=True)
GamlaIsraelino2_df.drop(['pm25','pm10','so2'], axis=1, inplace=True)
#Gamla Tourist
GamlaTouristpm25_df.drop(['pm10','no2','so2'], axis=1, inplace=True)
GamlaTouristno2_df.drop(['pm25','pm10','so2'], axis=1, inplace=True)

#Metzada Israeli
MetzadaIsraelipm25_df.drop(['pm10','no2','so2'], axis=1, inplace=True)
MetzadaIsraelipm10_df.drop(['pm25','no2','so2'], axis=1, inplace=True)
MetzadaIsraelino2_df.drop(['pm25','pm10','so2'], axis=1, inplace=True)
#Metzada Tourist
MetzadaTouristpm25_df.drop(['pm10','no2','so2'], axis=1, inplace=True)
MetzadaTouristpm10_df.drop(['pm25','no2','so2'], axis=1, inplace=True)
MetzadaTouristno2_df.drop(['pm25','pm10','so2'], axis=1, inplace=True)

In [255]:
#Megido Israeli
MegidoIsraelipm25_df.dropna(inplace=True)
MegidoIsraelino2_df.dropna(inplace=True)
#Megido Tourist
MegidoTouristpm25_df.dropna(inplace=True)
MegidoTouristno2_df.dropna(inplace=True)

#Banias Israeli
BaniasIsraelipm25_df.dropna(inplace=True)
BaniasIsraelino2_df.dropna(inplace=True)
#Banias Tourist
BaniasTouristpm25_df.dropna(inplace=True)
BaniasTouristno2_df.dropna(inplace=True)

#Gamla Israeli
GamlaIsraelipm25_df.dropna(inplace=True)
GamlaIsraelino2_df.dropna(inplace=True)
#Gamla Tourist
GamlaTouristpm25_df.dropna(inplace=True)
GamlaTouristno2_df.dropna(inplace=True)

#Metzada Israeli
MetzadaIsraelipm25_df.dropna(inplace=True)
MetzadaIsraelipm10_df.dropna(inplace=True)
MetzadaIsraelino2_df.dropna(inplace=True)
#Metzada Tourist
MetzadaTouristpm25_df.dropna(inplace=True)
MetzadaTouristpm10_df.dropna(inplace=True)
MetzadaTouristno2_df.dropna(inplace=True)

In [256]:
#Megido Israeli
MegidoIsraelipm25_df.reset_index(inplace=True, drop=True)
MegidoIsraelino2_df.reset_index(inplace=True, drop=True)
#Megido Tourist
MegidoTouristpm25_df.reset_index(inplace=True, drop=True)
MegidoTouristno2_df.reset_index(inplace=True, drop=True)

#Banias Israeli
BaniasIsraelipm25_df.reset_index(inplace=True, drop=True)
BaniasIsraelino2_df.reset_index(inplace=True, drop=True)
#Banias Tourist
BaniasTouristpm25_df.reset_index(inplace=True, drop=True)
BaniasTouristno2_df.reset_index(inplace=True, drop=True)

#Gamla Israeli
GamlaIsraelipm25_df.reset_index(inplace=True, drop=True)
GamlaIsraelino2_df.reset_index(inplace=True, drop=True)
#Gamla Tourist
GamlaTouristpm25_df.reset_index(inplace=True, drop=True)
GamlaTouristno2_df.reset_index(inplace=True, drop=True)

#Metzada Israeli
MetzadaIsraelipm25_df.reset_index(inplace=True, drop=True)
MetzadaIsraelipm10_df.reset_index(inplace=True, drop=True)
MetzadaIsraelino2_df.reset_index(inplace=True, drop=True)
#Metzada Tourist
MetzadaTouristpm25_df.reset_index(inplace=True, drop=True)
MetzadaTouristpm10_df.reset_index(inplace=True, drop=True)
MetzadaTouristno2_df.reset_index(inplace=True, drop=True)

In [257]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#Megido Israeli
scaled_MegidoIsraelipm25_df = pd.DataFrame(scaler.fit_transform(MegidoIsraelipm25_df), columns=MegidoIsraelipm25_df.columns)
scaled_MegidoIsraelino2_df = pd.DataFrame(scaler.fit_transform(MegidoIsraelino2_df), columns=MegidoIsraelino2_df.columns)
#Megido Tourist
scaled_MegidoTouristpm25_df = pd.DataFrame(scaler.fit_transform(MegidoTouristpm25_df), columns=MegidoTouristpm25_df.columns)
scaled_MegidoTouristno2_df = pd.DataFrame(scaler.fit_transform(MegidoTouristno2_df), columns=MegidoTouristno2_df.columns)

#Banias Israeli
scaled_BaniasIsraelipm25_df = pd.DataFrame(scaler.fit_transform(BaniasIsraelipm25_df), columns=BaniasIsraelipm25_df.columns)
scaled_BaniasIsraelino2_df = pd.DataFrame(scaler.fit_transform(BaniasIsraelino2_df), columns=BaniasIsraelino2_df.columns)
#Banias Tourist
scaled_BaniasTouristpm25_df = pd.DataFrame(scaler.fit_transform(BaniasTouristpm25_df), columns=BaniasTouristpm25_df.columns)
scaled_BaniasTouristno2_df = pd.DataFrame(scaler.fit_transform(BaniasTouristno2_df), columns=BaniasTouristno2_df.columns)

#Gamla Israeli
scaled_GamlaIsraelipm25_df = pd.DataFrame(scaler.fit_transform(GamlaIsraelipm25_df), columns=GamlaIsraelipm25_df.columns)
scaled_GamlaIsraelino2_df = pd.DataFrame(scaler.fit_transform(GamlaIsraelino2_df), columns=GamlaIsraelino2_df.columns)
#Gamla Tourist
scaled_GamlaTouristpm25_df = pd.DataFrame(scaler.fit_transform(GamlaTouristpm25_df), columns=GamlaTouristpm25_df.columns)
scaled_GamlaTouristno2_df = pd.DataFrame(scaler.fit_transform(GamlaTouristno2_df), columns=GamlaTouristno2_df.columns)

#Metzada Israeli
scaled_MetzadaIsraelipm25_df = pd.DataFrame(scaler.fit_transform(MetzadaIsraelipm25_df), columns=MetzadaIsraelipm25_df.columns)
scaled_MetzadaIsraelipm10_df = pd.DataFrame(scaler.fit_transform(MetzadaIsraelipm10_df), columns=MetzadaIsraelipm10_df.columns)
scaled_MetzadaIsraelino2_df = pd.DataFrame(scaler.fit_transform(MetzadaIsraelino2_df), columns=MetzadaIsraelino2_df.columns)
#Metzada Tourist
scaled_MetzadaTouristpm25_df = pd.DataFrame(scaler.fit_transform(MetzadaTouristpm25_df), columns=MetzadaTouristpm25_df.columns)
scaled_MetzadaTouristpm10_df = pd.DataFrame(scaler.fit_transform(MetzadaTouristpm10_df), columns=MetzadaTouristpm10_df.columns)
scaled_MetzadaTouristno2_df = pd.DataFrame(scaler.fit_transform(MetzadaTouristno2_df), columns=MetzadaTouristno2_df.columns)

In [258]:
#Megido Israeli
corr_MegidoIsraelipm25_df = MegidoIsraelipm25_df.corr()
correlated_MegidoIsraelipm25_df = corr_MegidoIsraelipm25_df.Israeli_Visitors.index.tolist()

corr_MegidoIsraelino2_df = MegidoIsraelino2_df.corr()
correlated_MegidoIsraelino2_df = corr_MegidoIsraelino2_df.Israeli_Visitors.index.tolist()
#Megido Tourist
corr_MegidoTouristpm25_df = MegidoTouristpm25_df.corr()
correlated_MegidoTouristpm25_df = corr_MegidoTouristpm25_df.Tourist_Visitors.index.tolist()

corr_MegidoTouristno2_df = MegidoTouristno2_df.corr()
correlated_MegidoTouristno2_df = corr_MegidoTouristno2_df.Tourist_Visitors.index.tolist()

#-------------------------------

#Banias Israeli
corr_BaniasIsraelipm25_df = BaniasIsraelipm25_df.corr()
correlated_BaniasIsraelipm25_df = corr_BaniasIsraelipm25_df.Israeli_Visitors.index.tolist()

corr_BaniasIsraelino2_df = BaniasIsraelino2_df.corr()
correlated_BaniasIsraelino2_df = corr_BaniasIsraelino2_df.Israeli_Visitors.index.tolist()
#Banias Tourist
corr_BaniasTouristpm25_df = BaniasTouristpm25_df.corr()
correlated_BaniasTouristpm25_df = corr_BaniasTouristpm25_df.Tourist_Visitors.index.tolist()

corr_BaniasTouristno2_df = BaniasTouristno2_df.corr()
correlated_BaniasTouristno2_df = corr_BaniasTouristno2_df.Tourist_Visitors.index.tolist()

#-------------------------------

#Gamla Israeli
corr_GamlaIsraelipm25_df = GamlaIsraelipm25_df.corr()
correlated_GamlaIsraelipm25_df = corr_GamlaIsraelipm25_df.Israeli_Visitors.index.tolist()

corr_GamlaIsraelino2_df = GamlaIsraelino2_df.corr()
correlated_GamlaIsraelino2_df = corr_GamlaIsraelino2_df.Israeli_Visitors.index.tolist()
#Gamla Tourist
corr_GamlaTouristpm25_df = GamlaTouristpm25_df.corr()
correlated_GamlaTouristpm25_df = corr_GamlaTouristpm25_df.Tourist_Visitors.index.tolist()

corr_GamlaTouristno2_df = GamlaTouristno2_df.corr()
correlated_GamlaTouristno2_df = corr_GamlaTouristno2_df.Tourist_Visitors.index.tolist()

#-------------------------------

#Metzada Israeli
corr_MetzadaIsraelipm25_df = MetzadaIsraelipm25_df.corr()
correlated_MetzadaIsraelipm25_df = corr_MetzadaIsraelipm25_df.Israeli_Visitors.index.tolist()

corr_MetzadaIsraelipm10_df = MetzadaIsraelipm10_df.corr()
correlated_MetzadaIsraelipm10_df = corr_MetzadaIsraelipm10_df.Israeli_Visitors.index.tolist()

corr_MetzadaIsraelino2_df = MetzadaIsraelino2_df.corr()
correlated_MetzadaIsraelino2_df = corr_MetzadaIsraelino2_df.Israeli_Visitors.index.tolist()
#Metzada Tourist
corr_MetzadaTouristpm25_df = MetzadaTouristpm25_df.corr()
correlated_MetzadaTouristpm25_df = corr_MetzadaTouristpm25_df.Tourist_Visitors.index.tolist()

corr_MetzadaTouristpm10_df = MetzadaTouristpm10_df.corr()
correlated_MetzadaTouristpm10_df = corr_MetzadaTouristpm10_df.Tourist_Visitors.index.tolist()

corr_MetzadaTouristno2_df = MetzadaTouristno2_df.corr()
correlated_MetzadaTouristno2_df = corr_MetzadaTouristno2_df.Tourist_Visitors.index.tolist()

In [259]:
#Megido Israeli
correlated_MegidoIsraelipm25_df = correlated_MegidoIsraelipm25_df[1:]
correlated_MegidoIsraelino2_df = correlated_MegidoIsraelino2_df[1:]
#Megido Tourist
correlated_MegidoTouristpm25_df = correlated_MegidoTouristpm25_df[1:]
correlated_MegidoTouristno2_df = correlated_MegidoTouristno2_df[1:]

#Banias Israeli
correlated_BaniasIsraelipm25_df = correlated_BaniasIsraelipm25_df[1:]
correlated_BaniasIsraelino2_df = correlated_BaniasIsraelino2_df[1:]
#Banias Tourist
correlated_BaniasTouristpm25_df = correlated_BaniasTouristpm25_df[1:]
correlated_BaniasTouristno2_df = correlated_BaniasTouristno2_df[1:]

#Gamla Israeli
correlated_GamlaIsraelipm25_df = correlated_GamlaIsraelipm25_df[1:]
correlated_GamlaIsraelino2_df = correlated_GamlaIsraelino2_df[1:]
#Gamla Tourist
correlated_GamlaTouristpm25_df = correlated_GamlaTouristpm25_df[1:]
correlated_GamlaTouristno2_df = correlated_GamlaTouristno2_df[1:]

#Metzada Israeli
correlated_MetzadaIsraelipm25_df = correlated_MetzadaIsraelipm25_df[1:]
correlated_MetzadaIsraelipm10_df = correlated_MetzadaIsraelipm10_df[1:]
correlated_MetzadaIsraelino2_df = correlated_MetzadaIsraelino2_df[1:]
#Metzada Tourist
correlated_MetzadaTouristpm25_df = correlated_MetzadaTouristpm25_df[1:]
correlated_MetzadaTouristpm10_df = correlated_MetzadaTouristpm10_df[1:]
correlated_MetzadaTouristno2_df = correlated_MetzadaTouristno2_df[1:]

### **Build Megido Weekend Regression Tables**

In [260]:
MegidoIsraeliWeekend_df = MegidoIsraeli_df.loc[MegidoIsraeli_df['Is_Weekend'] == 1]

In [261]:
MegidoIsraeliWeekendpm25_df = MegidoIsraeliWeekend_df.copy()
MegidoIsraeliWeekendno2_df = MegidoIsraeliWeekend_df.copy()
MegidoIsraeliWeekendTemp_df = MegidoIsraeliWeekend_df.copy()

In [262]:
MegidoIsraeliWeekendpm25_df.drop(['pm10','no2','so2','Temp_C'], axis=1, inplace=True)
MegidoIsraeliWeekendno2_df.drop(['pm25','pm10','so2','Temp_C'], axis=1, inplace=True)
MegidoIsraeliWeekendTemp_df.drop(['pm25','pm10','so2','no2'], axis=1, inplace=True)

In [263]:
MegidoIsraeliWeekendpm25_df.dropna(inplace=True)
MegidoIsraeliWeekendno2_df.dropna(inplace=True)
MegidoIsraeliWeekendTemp_df.dropna(inplace=True)

In [264]:
MegidoIsraeliWeekendpm25_df.reset_index(inplace=True, drop=True)
MegidoIsraeliWeekendno2_df.reset_index(inplace=True, drop=True)
MegidoIsraeliWeekendTemp_df.reset_index(inplace=True, drop=True)

In [265]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaled_MegidoIsraeliWeekendpm25_df = pd.DataFrame(scaler.fit_transform(MegidoIsraeliWeekendpm25_df), columns=MegidoIsraeliWeekendpm25_df.columns)
scaled_MegidoIsraeliWeekendno2_df = pd.DataFrame(scaler.fit_transform(MegidoIsraeliWeekendno2_df), columns=MegidoIsraeliWeekendno2_df.columns)
scaled_MegidoIsraeliWeekendTemp_df = pd.DataFrame(scaler.fit_transform(MegidoIsraeliWeekendTemp_df), columns=MegidoIsraeliWeekendTemp_df.columns)

In [266]:
corr_MegidoIsraeliWeekendpm25_df = MegidoIsraeliWeekendpm25_df.corr()
correlated_MegidoIsraeliWeekendpm25_df = corr_MegidoIsraeliWeekendpm25_df.Israeli_Visitors.index.tolist()

corr_MegidoIsraeliWeekendno2_df = MegidoIsraeliWeekendno2_df.corr()
correlated_MegidoIsraeliWeekendno2_df = corr_MegidoIsraeliWeekendno2_df.Israeli_Visitors.index.tolist()

corr_MegidoIsraeliWeekendTemp_df = MegidoIsraeliWeekendTemp_df.corr()
correlated_MegidoIsraeliWeekendTemp_df = corr_MegidoIsraeliWeekendTemp_df.Israeli_Visitors.index.tolist()

In [267]:
correlated_MegidoIsraeliWeekendpm25_df = correlated_MegidoIsraeliWeekendpm25_df[1:]
correlated_MegidoIsraeliWeekendno2_df = correlated_MegidoIsraeliWeekendno2_df[1:]
correlated_MegidoIsraeliWeekendTemp_df = correlated_MegidoIsraeliWeekendTemp_df[1:]

### **Build Full Table Summer Regression Tables**

In [268]:
FullTableSummerIsraeli_df = FinalTable_df.loc[FinalTable_df['Summer'] == 1]
FullTableSummerTourist_df = FinalTable_df.loc[FinalTable_df['Summer'] == 1]

In [269]:
FullTableSummerIsraeli_df.drop(['Date','Site_Code','Temp_location_code','Air_quality_Code','Site_Name','Location','Tourist_Visitors','Air_quality_location','Shabbaton','Winter','Spring'], axis=1, inplace=True)
FullTableSummerTourist_df.drop(['Date','Site_Code','Temp_location_code','Air_quality_Code','Site_Name','Location','Israeli_Visitors','Air_quality_location','Shabbaton','Winter','Spring'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [270]:
#Israeli
FullTableSummerIsraelipm25_df = FullTableSummerIsraeli_df.copy()
FullTableSummerIsraelino2_df = FullTableSummerIsraeli_df.copy()
#Tourist
FullTableSummerTouristpm25_df = FullTableSummerTourist_df.copy()
FullTableSummerTouristno2_df = FullTableSummerTourist_df.copy()

In [271]:
#Israeli
FullTableSummerIsraelipm25_df.drop(['pm10','no2','so2'], axis=1, inplace=True)
FullTableSummerIsraelino2_df.drop(['pm25','pm10','so2'], axis=1, inplace=True)
#Tourist
FullTableSummerTouristpm25_df.drop(['pm10','no2','so2'], axis=1, inplace=True)
FullTableSummerTouristno2_df.drop(['pm25','pm10','so2'], axis=1, inplace=True)

In [272]:
#Israeli
FullTableSummerIsraelipm25_df.dropna(inplace=True)
FullTableSummerIsraelino2_df.dropna(inplace=True)
#Tourist
FullTableSummerTouristpm25_df.dropna(inplace=True)
FullTableSummerTouristno2_df.dropna(inplace=True)

In [273]:
#Israeli
FullTableSummerIsraelipm25_df.reset_index(inplace=True, drop=True)
FullTableSummerIsraelino2_df.reset_index(inplace=True, drop=True)
#Tourist
FullTableSummerTouristpm25_df.reset_index(inplace=True, drop=True)
FullTableSummerTouristno2_df.reset_index(inplace=True, drop=True)

In [274]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#Israeli
scaled_FullTableSummerIsraelipm25_df = pd.DataFrame(scaler.fit_transform(FullTableSummerIsraelipm25_df), columns=FullTableSummerIsraelipm25_df.columns)
scaled_FullTableSummerIsraelino2_df = pd.DataFrame(scaler.fit_transform(FullTableSummerIsraelino2_df), columns=FullTableSummerIsraelino2_df.columns)
#Tourist
scaled_FullTableSummerTouristpm25_df = pd.DataFrame(scaler.fit_transform(FullTableSummerTouristpm25_df), columns=FullTableSummerTouristpm25_df.columns)
scaled_FullTableSummerTouristno2_df = pd.DataFrame(scaler.fit_transform(FullTableSummerTouristno2_df), columns=FullTableSummerTouristno2_df.columns)

In [275]:
#Israeli
corr_FullTableSummerIsraelipm25_df = FullTableSummerIsraelipm25_df.corr()
correlated_FullTableSummerIsraelipm25_df = corr_FullTableSummerIsraelipm25_df.Israeli_Visitors.index.tolist()

corr_FullTableSummerIsraelino2_df = FullTableSummerIsraelino2_df.corr()
correlated_FullTableSummerIsraelino2_df = corr_FullTableSummerIsraelino2_df.Israeli_Visitors.index.tolist()

#Tourist
corr_FullTableSummerTouristpm25_df = FullTableSummerTouristpm25_df.corr()
correlated_FullTableSummerTouristpm25_df = corr_FullTableSummerTouristpm25_df.Tourist_Visitors.index.tolist()

corr_FullTableSummerTouristno2_df = FullTableSummerTouristno2_df.corr()
correlated_FullTableSummerTouristno2_df = corr_FullTableSummerTouristno2_df.Tourist_Visitors.index.tolist()

In [276]:
#Israeli
correlated_FullTableSummerIsraelipm25_df = correlated_FullTableSummerIsraelipm25_df[1:]
correlated_FullTableSummerIsraelino2_df = correlated_FullTableSummerIsraelino2_df[1:]
#Tourist
correlated_FullTableSummerTouristpm25_df = correlated_FullTableSummerTouristpm25_df[1:]
correlated_FullTableSummerTouristno2_df = correlated_FullTableSummerTouristno2_df[1:]

### **Megido**

**Megido Israeli pm25**

In [277]:
MegidoIsraelipm25_df.std()

Israeli_Visitors        91.700693
Temp_C                   6.880385
pm25                    17.214486
Is_Weekend               0.450671
Is_Jewish_Holiday        0.299762
Is_Muslem_Holiday        0.179344
Is_Christian_Holiday     0.118556
Is_National_Park         0.000000
Is_Christian_Site        0.000000
Duration_Of_Visit        0.000000
Winter                   0.430300
Spring                   0.435793
Summer                   0.438436
dtype: float64

In [278]:
from sklearn.model_selection import train_test_split
y_Megido_Israelipm25 = MegidoIsraelipm25_df.Israeli_Visitors
X_Megido_Israelipm25 = MegidoIsraelipm25_df[correlated_MegidoIsraelipm25_df]

X_train_Megido_Israelipm25, X_test_Megido_Israelipm25, y_train_Megido_Israelipm25, y_test_Megido_Israelipm25 = train_test_split(X_Megido_Israelipm25, y_Megido_Israelipm25, test_size=0.3, random_state=1234)
train_df_Megido_Israelipm25 = pd.merge(left=X_train_Megido_Israelipm25, right=y_train_Megido_Israelipm25, left_index=True, right_index=True)
test_df_Megido_Israelipm25 = pd.merge(left=X_test_Megido_Israelipm25, right=y_test_Megido_Israelipm25, left_index=True, right_index=True)
#test_df_Megido_Israelipm25.head(10)

In [279]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Megido_Israelipm25 = StandardScaler()
x_test_scaler_Megido_Israelipm25 = StandardScaler()
y_train_scaler_Megido_Israelipm25 = StandardScaler()
y_test_scaler_Megido_Israelipm25 = StandardScaler()

X_train_scaled_Megido_Israelipm25 = x_train_scaler_Megido_Israelipm25.fit_transform(X_train_Megido_Israelipm25)
X_test_scaled_Megido_Israelipm25 = x_test_scaler_Megido_Israelipm25.fit_transform(X_test_Megido_Israelipm25)
y_train_scaled_Megido_Israelipm25 = y_train_scaler_Megido_Israelipm25.fit_transform(pd.DataFrame(y_train_Megido_Israelipm25))
y_test_scaled_Megido_Israelipm25 = y_test_scaler_Megido_Israelipm25.fit_transform(pd.DataFrame(y_test_Megido_Israelipm25))

In [280]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Megido_Israelipm25,y_train_scaled_Megido_Israelipm25)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_MegidoIsraelipm25_df[0],
      " + ",coeff[1]," * ",correlated_MegidoIsraelipm25_df[1]," + ",coeff[2]," * ",correlated_MegidoIsraelipm25_df[2]," + ",coeff[3]," * ",correlated_MegidoIsraelipm25_df[3]," + ",coeff[4]," * ",correlated_MegidoIsraelipm25_df[4]," + ",coeff[5]," * ",correlated_MegidoIsraelipm25_df[5]," + ",coeff[6]," * ",correlated_MegidoIsraelipm25_df[6]," + ",coeff[7]," * ",correlated_MegidoIsraelipm25_df[7]," + ",coeff[8]," * ",correlated_MegidoIsraelipm25_df[8]," + ",coeff[9]," * ",correlated_MegidoIsraelipm25_df[9]," + ",coeff[10]," * ",correlated_MegidoIsraelipm25_df[10]," + ",coeff[11]," * ",correlated_MegidoIsraelipm25_df[11])

Coefficients: 
 coeff = [-1.28127112e-01  6.01119797e-02  3.71335726e-01  1.47740626e-01
  5.74495117e-04  8.97361438e-02  2.08166817e-17 -2.77555756e-17
 -5.55111512e-17  1.89609059e-02  8.35858233e-02 -9.21429639e-02] ,  Intercept= 2.4006292305152068e-17 

The regression equation is: Israeli Visitors =  2.4006292305152068e-17  +  -0.12812711217193384  *  Temp_C  +  0.06011197973159829  *  pm25  +  0.3713357262913503  *  Is_Weekend  +  0.14774062626844545  *  Is_Jewish_Holiday  +  0.0005744951168719522  *  Is_Muslem_Holiday  +  0.0897361438119515  *  Is_Christian_Holiday  +  2.0816681711721685e-17  *  Is_National_Park  +  -2.7755575615628914e-17  *  Is_Christian_Site  +  -5.551115123125783e-17  *  Duration_Of_Visit  +  0.018960905884376968  *  Winter  +  0.08358582330323121  *  Spring  +  -0.09214296387820729  *  Summer


In [281]:
fitted_scaled_visitors_Megido_Israelipm25 = lm.predict(X_train_scaled_Megido_Israelipm25)
fitted_visitors_Megido_Israelipm25 = y_train_scaler_Megido_Israelipm25.inverse_transform(fitted_scaled_visitors_Megido_Israelipm25)
predicted_train_visitors_Israelipm25 = round(pd.Series(fitted_visitors_Megido_Israelipm25[:,0], index= y_train_Megido_Israelipm25.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Megido_Israelipm25 = pd.merge(left=train_df_Megido_Israelipm25, right=predicted_train_visitors_Israelipm25 , left_index=True, right_index=True)

#train_df_Megido_Israelipm25.head(10)

In [282]:
fitted_scaled_visitors_Megido_Israelipm25 = lm.predict(X_test_scaled_Megido_Israelipm25)
fitted_visitors_Megido_Israelipm25 = y_test_scaler_Megido_Israelipm25.inverse_transform(fitted_scaled_visitors_Megido_Israelipm25)
predicted_test_visitors_Israelipm25 = round(pd.Series(fitted_visitors_Megido_Israelipm25[:,0], index= y_test_Megido_Israelipm25.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Megido_Israelipm25 = pd.merge(left=test_df_Megido_Israelipm25, right=predicted_test_visitors_Israelipm25 , left_index=True, right_index=True)

#test_df_Megido_Israelipm25.head(10)

In [283]:
from sklearn import metrics
print("std of the objective function: 91.7")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Megido_Israelipm25.Israeli_Visitors, train_df_Megido_Israelipm25.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Megido_Israelipm25.Israeli_Visitors, test_df_Megido_Israelipm25.predicted_test_Visitors)))

std of the objective function: 91.7

------ TRAIN DATA ------
RMSE: 80.29955769118702

------ TEST DATA ------
RMSE: 80.60431890186103


In [284]:
from sklearn.metrics import r2_score
r2_score(train_df_Megido_Israelipm25.Israeli_Visitors, train_df_Megido_Israelipm25.predicted_train_Visitors)

0.2372316226275113

In [285]:
#get R2 for test set
r2_score(test_df_Megido_Israelipm25.Israeli_Visitors, test_df_Megido_Israelipm25.predicted_test_Visitors)

0.21506095295382865

**Megido Israeli no2**

In [286]:
MegidoIsraelino2_df.std()

Israeli_Visitors        92.023210
Temp_C                   6.903651
no2                      3.735243
Is_Weekend               0.451969
Is_Jewish_Holiday        0.300143
Is_Muslem_Holiday        0.179591
Is_Christian_Holiday     0.126268
Is_National_Park         0.000000
Is_Christian_Site        0.000000
Duration_Of_Visit        0.000000
Winter                   0.431276
Spring                   0.436203
Summer                   0.436736
dtype: float64

In [287]:
y_Megido_Israelino2 = MegidoIsraelino2_df.Israeli_Visitors
X_Megido_Israelino2 = MegidoIsraelino2_df[correlated_MegidoIsraelino2_df]

X_train_Megido_Israelino2, X_test_Megido_Israelino2, y_train_Megido_Israelino2, y_test_Megido_Israelino2 = train_test_split(X_Megido_Israelino2, y_Megido_Israelino2, test_size=0.3, random_state=1234)
train_df_Megido_Israelino2 = pd.merge(left=X_train_Megido_Israelino2, right=y_train_Megido_Israelino2, left_index=True, right_index=True)
test_df_Megido_Israelino2 = pd.merge(left=X_test_Megido_Israelino2, right=y_test_Megido_Israelino2, left_index=True, right_index=True)
#test_df_Megido_Israelino2.head(10)

In [288]:
x_train_scaler_Megido_Israelino2 = StandardScaler()
x_test_scaler_Megido_Israelino2 = StandardScaler()
y_train_scaler_Megido_Israelino2 = StandardScaler()
y_test_scaler_Megido_Israelino2 = StandardScaler()

X_train_scaled_Megido_Israelino2 = x_train_scaler_Megido_Israelino2.fit_transform(X_train_Megido_Israelino2)
X_test_scaled_Megido_Israelino2 = x_test_scaler_Megido_Israelino2.fit_transform(X_test_Megido_Israelino2)
y_train_scaled_Megido_Israelino2 = y_train_scaler_Megido_Israelino2.fit_transform(pd.DataFrame(y_train_Megido_Israelino2))
y_test_scaled_Megido_Israelino2 = y_test_scaler_Megido_Israelino2.fit_transform(pd.DataFrame(y_test_Megido_Israelino2))

In [289]:
lm = LinearRegression()
lm.fit(X_train_scaled_Megido_Israelino2,y_train_scaled_Megido_Israelino2)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_MegidoIsraelino2_df[0],
      " + ",coeff[1]," * ",correlated_MegidoIsraelino2_df[1]," + ",coeff[2]," * ",correlated_MegidoIsraelino2_df[2]," + ",coeff[3]," * ",correlated_MegidoIsraelino2_df[3]," + ",coeff[4]," * ",correlated_MegidoIsraelino2_df[4]," + ",coeff[5]," * ",correlated_MegidoIsraelino2_df[5]," + ",coeff[6]," * ",correlated_MegidoIsraelino2_df[6]," + ",coeff[7]," * ",correlated_MegidoIsraelino2_df[7]," + ",coeff[8]," * ",correlated_MegidoIsraelino2_df[8]," + ",coeff[9]," * ",correlated_MegidoIsraelino2_df[9]," + ",coeff[10]," * ",correlated_MegidoIsraelino2_df[10]," + ",coeff[11]," * ",correlated_MegidoIsraelino2_df[11])

Coefficients: 
 coeff = [-1.40105212e-01  1.02200256e-01  3.55170348e-01  2.03051560e-01
 -6.38326677e-03  5.63473460e-02 -6.93889390e-18  2.77555756e-17
  2.77555756e-17 -3.02439464e-03  9.44568251e-02 -8.27774672e-02] ,  Intercept= -7.700926218434425e-17 

The regression equation is: Israeli Visitors =  -7.700926218434425e-17  +  -0.14010521232401854  *  Temp_C  +  0.10220025586608457  *  no2  +  0.3551703478210547  *  Is_Weekend  +  0.2030515604913208  *  Is_Jewish_Holiday  +  -0.006383266767031444  *  Is_Muslem_Holiday  +  0.056347345950921765  *  Is_Christian_Holiday  +  -6.938893903907228e-18  *  Is_National_Park  +  2.7755575615628914e-17  *  Is_Christian_Site  +  2.7755575615628914e-17  *  Duration_Of_Visit  +  -0.0030243946356744433  *  Winter  +  0.09445682508150721  *  Spring  +  -0.08277746715710083  *  Summer


In [290]:
fitted_scaled_visitors_Megido_Israelino2 = lm.predict(X_train_scaled_Megido_Israelino2)
fitted_visitors_Megido_Israelino2 = y_train_scaler_Megido_Israelino2.inverse_transform(fitted_scaled_visitors_Megido_Israelino2)
predicted_train_visitors_Israelino2 = round(pd.Series(fitted_visitors_Megido_Israelino2[:,0], index= y_train_Megido_Israelino2.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Megido_Israelino2 = pd.merge(left=train_df_Megido_Israelino2, right=predicted_train_visitors_Israelino2 , left_index=True, right_index=True)

#train_df_Megido_Israelino2.head(10)

In [291]:
fitted_scaled_visitors_Megido_Israelino2 = lm.predict(X_test_scaled_Megido_Israelino2)
fitted_visitors_Megido_Israelino2 = y_test_scaler_Megido_Israelino2.inverse_transform(fitted_scaled_visitors_Megido_Israelino2)
predicted_test_visitors_Israelino2 = round(pd.Series(fitted_visitors_Megido_Israelino2[:,0], index= y_test_Megido_Israelino2.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Megido_Israelino2 = pd.merge(left=test_df_Megido_Israelino2, right=predicted_test_visitors_Israelino2 , left_index=True, right_index=True)

#test_df_Megido_Israelino2.head(10)

In [292]:
from sklearn import metrics
print("std of the objective function: 92.02")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Megido_Israelino2.Israeli_Visitors, train_df_Megido_Israelino2.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Megido_Israelino2.Israeli_Visitors, test_df_Megido_Israelino2.predicted_test_Visitors)))

std of the objective function: 92.02

------ TRAIN DATA ------
RMSE: 76.27517780633326

------ TEST DATA ------
RMSE: 88.62504342220264


In [293]:
from sklearn.metrics import r2_score
r2_score(train_df_Megido_Israelino2.Israeli_Visitors, train_df_Megido_Israelino2.predicted_train_Visitors)

0.2467495761580616

In [294]:
#get R2 for test set
r2_score(test_df_Megido_Israelino2.Israeli_Visitors, test_df_Megido_Israelino2.predicted_test_Visitors)

0.22583950024332988

**Megido Tourist pm25**

In [295]:
MegidoTouristpm25_df.std()

Tourist_Visitors        322.098494
Temp_C                    6.880385
pm25                     17.214486
Is_Weekend                0.450671
Is_Jewish_Holiday         0.299762
Is_Muslem_Holiday         0.179344
Is_Christian_Holiday      0.118556
Is_National_Park          0.000000
Is_Christian_Site         0.000000
Duration_Of_Visit         0.000000
Winter                    0.430300
Spring                    0.435793
Summer                    0.438436
dtype: float64

In [296]:
from sklearn.model_selection import train_test_split
y_Megido_Touristpm25 = MegidoTouristpm25_df.Tourist_Visitors
X_Megido_Touristpm25 = MegidoTouristpm25_df[correlated_MegidoTouristpm25_df]

X_train_Megido_Touristpm25, X_test_Megido_Touristpm25, y_train_Megido_Touristpm25, y_test_Megido_Touristpm25 = train_test_split(X_Megido_Touristpm25, y_Megido_Touristpm25, test_size=0.3, random_state=1234)
train_df_Megido_Touristpm25 = pd.merge(left=X_train_Megido_Touristpm25, right=y_train_Megido_Touristpm25, left_index=True, right_index=True)
test_df_Megido_Touristpm25 = pd.merge(left=X_test_Megido_Touristpm25, right=y_test_Megido_Touristpm25, left_index=True, right_index=True)
#test_df_Megido_Touristpm25.head(10)

In [297]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Megido_Touristpm25 = StandardScaler()
x_test_scaler_Megido_Touristpm25 = StandardScaler()
y_train_scaler_Megido_Touristpm25 = StandardScaler()
y_test_scaler_Megido_Touristpm25 = StandardScaler()

X_train_scaled_Megido_Touristpm25 = x_train_scaler_Megido_Touristpm25.fit_transform(X_train_Megido_Touristpm25)
X_test_scaled_Megido_Touristpm25 = x_test_scaler_Megido_Touristpm25.fit_transform(X_test_Megido_Touristpm25)
y_train_scaled_Megido_Touristpm25 = y_train_scaler_Megido_Touristpm25.fit_transform(pd.DataFrame(y_train_Megido_Touristpm25))
y_test_scaled_Megido_Touristpm25 = y_test_scaler_Megido_Touristpm25.fit_transform(pd.DataFrame(y_test_Megido_Touristpm25))

In [298]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Megido_Touristpm25,y_train_scaled_Megido_Touristpm25)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Tourist Visitors = " ,intercept," + ",coeff[0]," * ",correlated_MegidoTouristpm25_df[0],
      " + ",coeff[1]," * ",correlated_MegidoTouristpm25_df[1]," + ",coeff[2]," * ",correlated_MegidoTouristpm25_df[2]," + ",coeff[3]," * ",correlated_MegidoTouristpm25_df[3]," + ",coeff[4]," * ",correlated_MegidoTouristpm25_df[4]," + ",coeff[5]," * ",correlated_MegidoTouristpm25_df[5]," + ",coeff[6]," * ",correlated_MegidoTouristpm25_df[6]," + ",coeff[7]," * ",correlated_MegidoTouristpm25_df[7]," + ",coeff[8]," * ",correlated_MegidoTouristpm25_df[8]," + ",coeff[9]," * ",correlated_MegidoTouristpm25_df[9]," + ",coeff[10]," * ",correlated_MegidoTouristpm25_df[10]," + ",coeff[11]," * ",correlated_MegidoTouristpm25_df[11])

Coefficients: 
 coeff = [-4.00780994e-01  1.15317798e-01 -5.57274800e-02 -1.59575390e-01
 -1.33562682e-02 -9.86569131e-02 -2.77555756e-17  3.68628739e-18
  2.77555756e-17 -5.50891206e-01 -8.58959611e-02 -4.04970942e-01] ,  Intercept= 8.992437118594715e-17 

The regression equation is: Tourist Visitors =  8.992437118594715e-17  +  -0.4007809944940048  *  Temp_C  +  0.11531779814672433  *  pm25  +  -0.05572748004995539  *  Is_Weekend  +  -0.15957538995983414  *  Is_Jewish_Holiday  +  -0.01335626817455933  *  Is_Muslem_Holiday  +  -0.0986569131219307  *  Is_Christian_Holiday  +  -2.7755575615628914e-17  *  Is_National_Park  +  3.686287386450715e-18  *  Is_Christian_Site  +  2.7755575615628914e-17  *  Duration_Of_Visit  +  -0.550891206399727  *  Winter  +  -0.08589596111189465  *  Spring  +  -0.40497094199421  *  Summer


In [299]:
fitted_scaled_visitors_Megido_Touristpm25 = lm.predict(X_train_scaled_Megido_Touristpm25)
fitted_visitors_Megido_Touristpm25 = y_train_scaler_Megido_Touristpm25.inverse_transform(fitted_scaled_visitors_Megido_Touristpm25)
predicted_train_visitors_Touristpm25 = round(pd.Series(fitted_visitors_Megido_Touristpm25[:,0], index= y_train_Megido_Touristpm25.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Megido_Touristpm25 = pd.merge(left=train_df_Megido_Touristpm25, right=predicted_train_visitors_Touristpm25 , left_index=True, right_index=True)

#train_df_Megido_Touristpm25.head(10)

In [300]:
fitted_scaled_visitors_Megido_Touristpm25 = lm.predict(X_test_scaled_Megido_Touristpm25)
fitted_visitors_Megido_Touristpm25 = y_test_scaler_Megido_Touristpm25.inverse_transform(fitted_scaled_visitors_Megido_Touristpm25)
predicted_test_visitors_Touristpm25 = round(pd.Series(fitted_visitors_Megido_Touristpm25[:,0], index= y_test_Megido_Touristpm25.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Megido_Touristpm25 = pd.merge(left=test_df_Megido_Touristpm25, right=predicted_test_visitors_Touristpm25 , left_index=True, right_index=True)

#test_df_Megido_Touristpm25.head(10)

In [301]:
from sklearn import metrics
print("std of the objective function: 322.09")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Megido_Touristpm25.Tourist_Visitors, train_df_Megido_Touristpm25.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Megido_Touristpm25.Tourist_Visitors, test_df_Megido_Touristpm25.predicted_test_Visitors)))

std of the objective function: 322.09

------ TRAIN DATA ------
RMSE: 271.4396639444565

------ TEST DATA ------
RMSE: 266.3913410220366


In [302]:
from sklearn.metrics import r2_score
r2_score(train_df_Megido_Touristpm25.Tourist_Visitors, train_df_Megido_Touristpm25.predicted_train_Visitors)

0.3007300225652181

In [303]:
#get R2 for test set
r2_score(test_df_Megido_Touristpm25.Tourist_Visitors, test_df_Megido_Touristpm25.predicted_test_Visitors)

0.28680240216291863

**Megido Tourist no2**

In [304]:
MegidoTouristno2_df.std()

Tourist_Visitors        323.799495
Temp_C                    6.903651
no2                       3.735243
Is_Weekend                0.451969
Is_Jewish_Holiday         0.300143
Is_Muslem_Holiday         0.179591
Is_Christian_Holiday      0.126268
Is_National_Park          0.000000
Is_Christian_Site         0.000000
Duration_Of_Visit         0.000000
Winter                    0.431276
Spring                    0.436203
Summer                    0.436736
dtype: float64

In [305]:
from sklearn.model_selection import train_test_split
y_Megido_Touristno2 = MegidoTouristno2_df.Tourist_Visitors
X_Megido_Touristno2 = MegidoTouristno2_df[correlated_MegidoTouristno2_df]

X_train_Megido_Touristno2, X_test_Megido_Touristno2, y_train_Megido_Touristno2, y_test_Megido_Touristno2 = train_test_split(X_Megido_Touristno2, y_Megido_Touristno2, test_size=0.3, random_state=1234)
train_df_Megido_Touristno2 = pd.merge(left=X_train_Megido_Touristno2, right=y_train_Megido_Touristno2, left_index=True, right_index=True)
test_df_Megido_Touristno2 = pd.merge(left=X_test_Megido_Touristno2, right=y_test_Megido_Touristno2, left_index=True, right_index=True)
#test_df_Megido_Touristno2.head(10)

In [306]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Megido_Touristno2 = StandardScaler()
x_test_scaler_Megido_Touristno2 = StandardScaler()
y_train_scaler_Megido_Touristno2 = StandardScaler()
y_test_scaler_Megido_Touristno2 = StandardScaler()

X_train_scaled_Megido_Touristno2 = x_train_scaler_Megido_Touristno2.fit_transform(X_train_Megido_Touristno2)
X_test_scaled_Megido_Touristno2 = x_test_scaler_Megido_Touristno2.fit_transform(X_test_Megido_Touristno2)
y_train_scaled_Megido_Touristno2 = y_train_scaler_Megido_Touristno2.fit_transform(pd.DataFrame(y_train_Megido_Touristno2))
y_test_scaled_Megido_Touristno2 = y_test_scaler_Megido_Touristno2.fit_transform(pd.DataFrame(y_test_Megido_Touristno2))

In [307]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Megido_Touristno2,y_train_scaled_Megido_Touristno2)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Tourist Visitors = " ,intercept," + ",coeff[0]," * ",correlated_MegidoTouristno2_df[0],
      " + ",coeff[1]," * ",correlated_MegidoTouristno2_df[1]," + ",coeff[2]," * ",correlated_MegidoTouristno2_df[2]," + ",coeff[3]," * ",correlated_MegidoTouristno2_df[3]," + ",coeff[4]," * ",correlated_MegidoTouristno2_df[4]," + ",coeff[5]," * ",correlated_MegidoTouristno2_df[5]," + ",coeff[6]," * ",correlated_MegidoTouristno2_df[6]," + ",coeff[7]," * ",correlated_MegidoTouristno2_df[7]," + ",coeff[8]," * ",correlated_MegidoTouristno2_df[8]," + ",coeff[9]," * ",correlated_MegidoTouristno2_df[9]," + ",coeff[10]," * ",correlated_MegidoTouristno2_df[10]," + ",coeff[11]," * ",correlated_MegidoTouristno2_df[11])

Coefficients: 
 coeff = [-3.22376378e-01 -5.50896011e-03 -7.69627858e-02 -1.79715129e-01
 -1.26870575e-02 -1.07126247e-01  0.00000000e+00  2.08166817e-17
  0.00000000e+00 -4.88340886e-01 -6.05905619e-02 -4.24397823e-01] ,  Intercept= -2.5356234812319996e-16 

The regression equation is: Tourist Visitors =  -2.5356234812319996e-16  +  -0.3223763784400662  *  Temp_C  +  -0.005508960110543665  *  no2  +  -0.07696278577447493  *  Is_Weekend  +  -0.17971512902592157  *  Is_Jewish_Holiday  +  -0.012687057503489333  *  Is_Muslem_Holiday  +  -0.10712624696044885  *  Is_Christian_Holiday  +  0.0  *  Is_National_Park  +  2.0816681711721685e-17  *  Is_Christian_Site  +  0.0  *  Duration_Of_Visit  +  -0.48834088593000957  *  Winter  +  -0.06059056186099165  *  Spring  +  -0.4243978232560564  *  Summer


In [308]:
fitted_scaled_visitors_Megido_Touristno2 = lm.predict(X_train_scaled_Megido_Touristno2)
fitted_visitors_Megido_Touristno2 = y_train_scaler_Megido_Touristno2.inverse_transform(fitted_scaled_visitors_Megido_Touristno2)
predicted_train_visitors_Touristno2 = round(pd.Series(fitted_visitors_Megido_Touristno2[:,0], index= y_train_Megido_Touristno2.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Megido_Touristno2 = pd.merge(left=train_df_Megido_Touristno2, right=predicted_train_visitors_Touristno2 , left_index=True, right_index=True)

#train_df_Megido_Touristno2.head(10)

In [309]:
fitted_scaled_visitors_Megido_Touristno2 = lm.predict(X_test_scaled_Megido_Touristno2)
fitted_visitors_Megido_Touristno2 = y_test_scaler_Megido_Touristno2.inverse_transform(fitted_scaled_visitors_Megido_Touristno2)
predicted_test_visitors_Touristno2 = round(pd.Series(fitted_visitors_Megido_Touristno2[:,0], index= y_test_Megido_Touristno2.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Megido_Touristno2 = pd.merge(left=test_df_Megido_Touristno2, right=predicted_test_visitors_Touristno2 , left_index=True, right_index=True)

#test_df_Megido_Touristno2.head(10)

In [310]:
from sklearn import metrics
print("std of the objective function: 323.79")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Megido_Touristno2.Tourist_Visitors, train_df_Megido_Touristno2.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Megido_Touristno2.Tourist_Visitors, test_df_Megido_Touristno2.predicted_test_Visitors)))

std of the objective function: 323.79

------ TRAIN DATA ------
RMSE: 276.2757263465238

------ TEST DATA ------
RMSE: 262.10785953587157


In [311]:
from sklearn.metrics import r2_score
r2_score(train_df_Megido_Touristno2.Tourist_Visitors, train_df_Megido_Touristno2.predicted_train_Visitors)

0.30321348553074134

In [312]:
#get R2 for test set
r2_score(test_df_Megido_Touristno2.Tourist_Visitors, test_df_Megido_Touristno2.predicted_test_Visitors)

0.2636164321286496

### **Megido Weekend**

**Megido Israeli Weekend no2**

In [313]:
MegidoIsraeliWeekendno2_df.std()

Israeli_Visitors        132.320092
no2                       4.283697
Is_Weekend                0.000000
Is_Jewish_Holiday         0.276847
Is_Muslem_Holiday         0.170872
Is_Christian_Holiday      0.081513
Is_National_Park          0.000000
Is_Christian_Site         0.000000
Duration_Of_Visit         0.000000
Winter                    0.431791
Spring                    0.433736
Summer                    0.441178
dtype: float64

In [314]:
from sklearn.model_selection import train_test_split
y_Megido_IsraeliWeekendno2 = MegidoIsraeliWeekendno2_df.Israeli_Visitors
X_Megido_IsraeliWeekendno2 = MegidoIsraeliWeekendno2_df[correlated_MegidoIsraeliWeekendno2_df]

X_train_Megido_IsraeliWeekendno2, X_test_Megido_IsraeliWeekendno2, y_train_Megido_IsraeliWeekendno2, y_test_Megido_IsraeliWeekendno2 = train_test_split(X_Megido_IsraeliWeekendno2, y_Megido_IsraeliWeekendno2, test_size=0.3, random_state=1234)
train_df_Megido_IsraeliWeekendno2 = pd.merge(left=X_train_Megido_IsraeliWeekendno2, right=y_train_Megido_IsraeliWeekendno2, left_index=True, right_index=True)
test_df_Megido_IsraeliWeekendno2 = pd.merge(left=X_test_Megido_IsraeliWeekendno2, right=y_test_Megido_IsraeliWeekendno2, left_index=True, right_index=True)
#test_df_Megido_IsraeliWeekendno2.head(10)

In [315]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Megido_IsraeliWeekendno2 = StandardScaler()
x_test_scaler_Megido_IsraeliWeekendno2 = StandardScaler()
y_train_scaler_Megido_IsraeliWeekendno2 = StandardScaler()
y_test_scaler_Megido_IsraeliWeekendno2 = StandardScaler()

X_train_scaled_Megido_IsraeliWeekendno2 = x_train_scaler_Megido_IsraeliWeekendno2.fit_transform(X_train_Megido_IsraeliWeekendno2)
X_test_scaled_Megido_IsraeliWeekendno2 = x_test_scaler_Megido_IsraeliWeekendno2.fit_transform(X_test_Megido_IsraeliWeekendno2)
y_train_scaled_Megido_IsraeliWeekendno2 = y_train_scaler_Megido_IsraeliWeekendno2.fit_transform(pd.DataFrame(y_train_Megido_IsraeliWeekendno2))
y_test_scaled_Megido_IsraeliWeekendno2 = y_test_scaler_Megido_IsraeliWeekendno2.fit_transform(pd.DataFrame(y_test_Megido_IsraeliWeekendno2))

In [316]:
MegidoIsraeliWeekendno2_df

,Israeli_Visitors,no2,Is_Weekend,Is_Jewish_Holiday,Is_Muslem_Holiday,Is_Christian_Holiday,Is_National_Park,Is_Christian_Site,Duration_Of_Visit,Winter,Spring,Summer
0,148,4.0,1,0,0,0,1,1,1,1,0,0
1,286,4.0,1,0,0,0,1,1,1,1,0,0
2,102,7.0,1,0,0,0,1,1,1,1,0,0
3,147,8.0,1,0,0,0,1,1,1,1,0,0
4,79,3.0,1,0,0,0,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
295,151,6.0,1,0,0,0,1,1,1,1,0,0
296,77,2.0,1,0,0,0,1,1,1,1,0,0
297,298,12.0,1,0,0,0,1,1,1,1,0,0
298,46,2.0,1,1,0,0,1,1,1,1,0,0


In [317]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Megido_IsraeliWeekendno2,y_train_scaled_Megido_IsraeliWeekendno2)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_MegidoIsraeliWeekendno2_df[0],
      " + ",coeff[1]," * ",correlated_MegidoIsraeliWeekendno2_df[1]," + ",coeff[2]," * ",correlated_MegidoIsraeliWeekendno2_df[2]," + ",coeff[3]," * ",correlated_MegidoIsraeliWeekendno2_df[3]," + ",coeff[4]," * ",correlated_MegidoIsraeliWeekendno2_df[4]," + ",coeff[5]," * ",correlated_MegidoIsraeliWeekendno2_df[5]," + ",coeff[6]," * ",correlated_MegidoIsraeliWeekendno2_df[6]," + ",coeff[7]," * ",correlated_MegidoIsraeliWeekendno2_df[7]," + ",coeff[8]," * ",correlated_MegidoIsraeliWeekendno2_df[8]," + ",coeff[9]," * ",correlated_MegidoIsraeliWeekendno2_df[9]," + ",coeff[10]," * ",correlated_MegidoIsraeliWeekendno2_df[10])

Coefficients: 
 coeff = [ 2.01899824e-01  0.00000000e+00 -8.49379052e-02 -3.57379373e-02
  1.77089279e-01  0.00000000e+00 -6.93889390e-18  0.00000000e+00
  2.09517853e-01  8.94673635e-02 -2.56241808e-01] ,  Intercept= 8.633307280172421e-17 

The regression equation is: Israeli Visitors =  8.633307280172421e-17  +  0.20189982380437235  *  no2  +  0.0  *  Is_Weekend  +  -0.0849379052125972  *  Is_Jewish_Holiday  +  -0.035737937309317167  *  Is_Muslem_Holiday  +  0.17708927859837964  *  Is_Christian_Holiday  +  0.0  *  Is_National_Park  +  -6.938893903907228e-18  *  Is_Christian_Site  +  0.0  *  Duration_Of_Visit  +  0.20951785273542814  *  Winter  +  0.08946736347710095  *  Spring  +  -0.25624180837024746  *  Summer


In [318]:
fitted_scaled_visitors_Megido_IsraeliWeekendno2 = lm.predict(X_train_scaled_Megido_IsraeliWeekendno2)
fitted_visitors_Megido_IsraeliWeekendno2 = y_train_scaler_Megido_IsraeliWeekendno2.inverse_transform(fitted_scaled_visitors_Megido_IsraeliWeekendno2)
predicted_train_visitors_IsraeliWeekendno2 = round(pd.Series(fitted_visitors_Megido_IsraeliWeekendno2[:,0], index= y_train_Megido_IsraeliWeekendno2.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Megido_IsraeliWeekendno2 = pd.merge(left=train_df_Megido_IsraeliWeekendno2, right=predicted_train_visitors_IsraeliWeekendno2 , left_index=True, right_index=True)

#train_df_Megido_IsraeliWeekendno2.head(10)

In [319]:
fitted_scaled_visitors_Megido_IsraeliWeekendno2 = lm.predict(X_test_scaled_Megido_IsraeliWeekendno2)
fitted_visitors_Megido_IsraeliWeekendno2 = y_test_scaler_Megido_IsraeliWeekendno2.inverse_transform(fitted_scaled_visitors_Megido_IsraeliWeekendno2)
predicted_test_visitors_IsraeliWeekendno2 = round(pd.Series(fitted_visitors_Megido_IsraeliWeekendno2[:,0], index= y_test_Megido_IsraeliWeekendno2.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Megido_IsraeliWeekendno2 = pd.merge(left=test_df_Megido_IsraeliWeekendno2, right=predicted_test_visitors_IsraeliWeekendno2 , left_index=True, right_index=True)

#test_df_Megido_IsraeliWeekendno2.head(10)

In [320]:
from sklearn import metrics
print("std of the objective function: 132.32")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Megido_IsraeliWeekendno2.Israeli_Visitors, train_df_Megido_IsraeliWeekendno2.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Megido_IsraeliWeekendno2.Israeli_Visitors, test_df_Megido_IsraeliWeekendno2.predicted_test_Visitors)))


std of the objective function: 132.32

------ TRAIN DATA ------
RMSE: 113.48316551260764

------ TEST DATA ------
RMSE: 123.79879024269806


In [321]:
from sklearn.metrics import r2_score
r2_score(train_df_Megido_IsraeliWeekendno2.Israeli_Visitors, train_df_Megido_IsraeliWeekendno2.predicted_train_Visitors)

0.2512895809569129

In [322]:
#get R2 for test set
r2_score(test_df_Megido_IsraeliWeekendno2.Israeli_Visitors, test_df_Megido_IsraeliWeekendno2.predicted_test_Visitors)

0.1499793545857705

**Megido Israeli Weekend pm25**

In [323]:
MegidoIsraeliWeekendpm25_df.std()

Israeli_Visitors        131.819344
pm25                     17.477385
Is_Weekend                0.000000
Is_Jewish_Holiday         0.272580
Is_Muslem_Holiday         0.171429
Is_Christian_Holiday      0.081785
Is_National_Park          0.000000
Is_Christian_Site         0.000000
Duration_Of_Visit         0.000000
Winter                    0.432766
Spring                    0.432766
Summer                    0.443902
dtype: float64

In [324]:
from sklearn.model_selection import train_test_split
y_Megido_IsraeliWeekendpm25 = MegidoIsraeliWeekendpm25_df.Israeli_Visitors
X_Megido_IsraeliWeekendpm25 = MegidoIsraeliWeekendpm25_df[correlated_MegidoIsraeliWeekendpm25_df]

X_train_Megido_IsraeliWeekendpm25, X_test_Megido_IsraeliWeekendpm25, y_train_Megido_IsraeliWeekendpm25, y_test_Megido_IsraeliWeekendpm25 = train_test_split(X_Megido_IsraeliWeekendpm25, y_Megido_IsraeliWeekendpm25, test_size=0.3, random_state=1234)
train_df_Megido_IsraeliWeekendpm25 = pd.merge(left=X_train_Megido_IsraeliWeekendpm25, right=y_train_Megido_IsraeliWeekendpm25, left_index=True, right_index=True)
test_df_Megido_IsraeliWeekendpm25 = pd.merge(left=X_test_Megido_IsraeliWeekendpm25, right=y_test_Megido_IsraeliWeekendpm25, left_index=True, right_index=True)
#test_df_Megido_IsraeliWeekendpm25.head(10)

In [325]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Megido_IsraeliWeekendpm25 = StandardScaler()
x_test_scaler_Megido_IsraeliWeekendpm25 = StandardScaler()
y_train_scaler_Megido_IsraeliWeekendpm25 = StandardScaler()
y_test_scaler_Megido_IsraeliWeekendpm25 = StandardScaler()

X_train_scaled_Megido_IsraeliWeekendpm25 = x_train_scaler_Megido_IsraeliWeekendpm25.fit_transform(X_train_Megido_IsraeliWeekendpm25)
X_test_scaled_Megido_IsraeliWeekendpm25 = x_test_scaler_Megido_IsraeliWeekendpm25.fit_transform(X_test_Megido_IsraeliWeekendpm25)
y_train_scaled_Megido_IsraeliWeekendpm25 = y_train_scaler_Megido_IsraeliWeekendpm25.fit_transform(pd.DataFrame(y_train_Megido_IsraeliWeekendpm25))
y_test_scaled_Megido_IsraeliWeekendpm25 = y_test_scaler_Megido_IsraeliWeekendpm25.fit_transform(pd.DataFrame(y_test_Megido_IsraeliWeekendpm25))

In [326]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Megido_IsraeliWeekendpm25,y_train_scaled_Megido_IsraeliWeekendpm25)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_MegidoIsraeliWeekendpm25_df[0],
      " + ",coeff[1]," * ",correlated_MegidoIsraeliWeekendpm25_df[1]," + ",coeff[2]," * ",correlated_MegidoIsraeliWeekendpm25_df[2]," + ",coeff[3]," * ",correlated_MegidoIsraeliWeekendpm25_df[3]," + ",coeff[4]," * ",correlated_MegidoIsraeliWeekendpm25_df[4]," + ",coeff[5]," * ",correlated_MegidoIsraeliWeekendpm25_df[5]," + ",coeff[6]," * ",correlated_MegidoIsraeliWeekendpm25_df[6]," + ",coeff[7]," * ",correlated_MegidoIsraeliWeekendpm25_df[7]," + ",coeff[8]," * ",correlated_MegidoIsraeliWeekendpm25_df[8]," + ",coeff[9]," * ",correlated_MegidoIsraeliWeekendpm25_df[9]," + ",coeff[10])

Coefficients: 
 coeff = [-9.24258513e-02  0.00000000e+00 -4.48052424e-02 -1.67749427e-02
  1.86440638e-01 -5.20417043e-18 -2.77555756e-17  0.00000000e+00
  2.24446754e-01  1.62371069e-01 -2.44384944e-01] ,  Intercept= 9.66324450050407e-17 

The regression equation is: Israeli Visitors =  9.66324450050407e-17  +  -0.09242585126700968  *  pm25  +  0.0  *  Is_Weekend  +  -0.04480524239202284  *  Is_Jewish_Holiday  +  -0.016774942658526283  *  Is_Muslem_Holiday  +  0.1864406380397719  *  Is_Christian_Holiday  +  -5.204170427930421e-18  *  Is_National_Park  +  -2.7755575615628914e-17  *  Is_Christian_Site  +  0.0  *  Duration_Of_Visit  +  0.2244467537476553  *  Winter  +  0.1623710691189509  *  Spring  +  -0.2443849438316018


In [327]:
fitted_scaled_visitors_Megido_IsraeliWeekendpm25 = lm.predict(X_train_scaled_Megido_IsraeliWeekendpm25)
fitted_visitors_Megido_IsraeliWeekendpm25 = y_train_scaler_Megido_IsraeliWeekendpm25.inverse_transform(fitted_scaled_visitors_Megido_IsraeliWeekendpm25)
predicted_train_visitors_IsraeliWeekendpm25 = round(pd.Series(fitted_visitors_Megido_IsraeliWeekendpm25[:,0], index= y_train_Megido_IsraeliWeekendpm25.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Megido_IsraeliWeekendpm25 = pd.merge(left=train_df_Megido_IsraeliWeekendpm25, right=predicted_train_visitors_IsraeliWeekendpm25 , left_index=True, right_index=True)

#train_df_Megido_IsraeliWeekendpm25.head(10)

In [328]:
fitted_scaled_visitors_Megido_IsraeliWeekendpm25 = lm.predict(X_test_scaled_Megido_IsraeliWeekendpm25)
fitted_visitors_Megido_IsraeliWeekendpm25 = y_test_scaler_Megido_IsraeliWeekendpm25.inverse_transform(fitted_scaled_visitors_Megido_IsraeliWeekendpm25)
predicted_test_visitors_IsraeliWeekendpm25 = round(pd.Series(fitted_visitors_Megido_IsraeliWeekendpm25[:,0], index= y_test_Megido_IsraeliWeekendpm25.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Megido_IsraeliWeekendpm25 = pd.merge(left=test_df_Megido_IsraeliWeekendpm25, right=predicted_test_visitors_IsraeliWeekendpm25 , left_index=True, right_index=True)

#test_df_Megido_IsraeliWeekendpm25.head(10)

In [329]:
from sklearn import metrics
print("std of the objective function: 131.81")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Megido_IsraeliWeekendpm25.Israeli_Visitors, train_df_Megido_IsraeliWeekendpm25.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Megido_IsraeliWeekendpm25.Israeli_Visitors, test_df_Megido_IsraeliWeekendpm25.predicted_test_Visitors)))


std of the objective function: 131.81

------ TRAIN DATA ------
RMSE: 117.90327423154639

------ TEST DATA ------
RMSE: 117.6079342325357


In [330]:
from sklearn.metrics import r2_score
r2_score(train_df_Megido_IsraeliWeekendpm25.Israeli_Visitors, train_df_Megido_IsraeliWeekendpm25.predicted_train_Visitors)

0.23372451578681563

In [331]:
#get R2 for test set
r2_score(test_df_Megido_IsraeliWeekendpm25.Israeli_Visitors, test_df_Megido_IsraeliWeekendpm25.predicted_test_Visitors)

0.09725637399345255

**Megido Israeli Weekend Temp**

In [332]:
MegidoIsraeliWeekendTemp_df.std()

Israeli_Visitors        130.927889
Temp_C                    7.013568
Is_Weekend                0.000000
Is_Jewish_Holiday         0.286689
Is_Muslem_Holiday         0.167905
Is_Christian_Holiday      0.097898
Is_National_Park          0.000000
Is_Christian_Site         0.000000
Duration_Of_Visit         0.000000
Winter                    0.428475
Spring                    0.436010
Summer                    0.437814
dtype: float64

In [333]:
from sklearn.model_selection import train_test_split
y_Megido_IsraeliWeekendTemp = MegidoIsraeliWeekendTemp_df.Israeli_Visitors
X_Megido_IsraeliWeekendTemp = MegidoIsraeliWeekendTemp_df[correlated_MegidoIsraeliWeekendTemp_df]

X_train_Megido_IsraeliWeekendTemp, X_test_Megido_IsraeliWeekendTemp, y_train_Megido_IsraeliWeekendTemp, y_test_Megido_IsraeliWeekendTemp = train_test_split(X_Megido_IsraeliWeekendTemp, y_Megido_IsraeliWeekendTemp, test_size=0.3, random_state=1234)
train_df_Megido_IsraeliWeekendTemp = pd.merge(left=X_train_Megido_IsraeliWeekendTemp, right=y_train_Megido_IsraeliWeekendTemp, left_index=True, right_index=True)
test_df_Megido_IsraeliWeekendTemp = pd.merge(left=X_test_Megido_IsraeliWeekendTemp, right=y_test_Megido_IsraeliWeekendTemp, left_index=True, right_index=True)
#test_df_Megido_IsraeliWeekendTemp.head(10)

In [334]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Megido_IsraeliWeekendTemp = StandardScaler()
x_test_scaler_Megido_IsraeliWeekendTemp = StandardScaler()
y_train_scaler_Megido_IsraeliWeekendTemp = StandardScaler()
y_test_scaler_Megido_IsraeliWeekendTemp = StandardScaler()

X_train_scaled_Megido_IsraeliWeekendTemp = x_train_scaler_Megido_IsraeliWeekendTemp.fit_transform(X_train_Megido_IsraeliWeekendTemp)
X_test_scaled_Megido_IsraeliWeekendTemp = x_test_scaler_Megido_IsraeliWeekendTemp.fit_transform(X_test_Megido_IsraeliWeekendTemp)
y_train_scaled_Megido_IsraeliWeekendTemp = y_train_scaler_Megido_IsraeliWeekendTemp.fit_transform(pd.DataFrame(y_train_Megido_IsraeliWeekendTemp))
y_test_scaled_Megido_IsraeliWeekendTemp = y_test_scaler_Megido_IsraeliWeekendTemp.fit_transform(pd.DataFrame(y_test_Megido_IsraeliWeekendTemp))

In [335]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Megido_IsraeliWeekendTemp,y_train_scaled_Megido_IsraeliWeekendTemp)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_MegidoIsraeliWeekendTemp_df[0],
      " + ",coeff[1]," * ",correlated_MegidoIsraeliWeekendTemp_df[1]," + ",coeff[2]," * ",correlated_MegidoIsraeliWeekendTemp_df[2]," + ",coeff[3]," * ",correlated_MegidoIsraeliWeekendTemp_df[3]," + ",coeff[4]," * ",correlated_MegidoIsraeliWeekendTemp_df[4]," + ",coeff[5]," * ",correlated_MegidoIsraeliWeekendTemp_df[5]," + ",coeff[6]," * ",correlated_MegidoIsraeliWeekendTemp_df[6]," + ",coeff[7]," * ",correlated_MegidoIsraeliWeekendTemp_df[7]," + ",coeff[8]," * ",correlated_MegidoIsraeliWeekendTemp_df[8]," + ",coeff[9]," * ",correlated_MegidoIsraeliWeekendTemp_df[9]," + ",coeff[10]," * ",correlated_MegidoIsraeliWeekendTemp_df[10])

Coefficients: 
 coeff = [-6.56403016e-02  0.00000000e+00 -3.96301302e-02  6.74876513e-04
  3.18866775e-02 -6.93889390e-18  0.00000000e+00  0.00000000e+00
  2.26408960e-01  1.47832355e-01 -2.17945233e-01] ,  Intercept= -4.941370592481264e-18 

The regression equation is: Israeli Visitors =  -4.941370592481264e-18  +  -0.06564030160361492  *  Temp_C  +  0.0  *  Is_Weekend  +  -0.03963013024072373  *  Is_Jewish_Holiday  +  0.0006748765127321671  *  Is_Muslem_Holiday  +  0.03188667750247753  *  Is_Christian_Holiday  +  -6.938893903907228e-18  *  Is_National_Park  +  0.0  *  Is_Christian_Site  +  0.0  *  Duration_Of_Visit  +  0.2264089602900782  *  Winter  +  0.1478323550782608  *  Spring  +  -0.21794523316309738  *  Summer


In [336]:
fitted_scaled_visitors_Megido_IsraeliWeekendTemp = lm.predict(X_train_scaled_Megido_IsraeliWeekendTemp)
fitted_visitors_Megido_IsraeliWeekendTemp = y_train_scaler_Megido_IsraeliWeekendTemp.inverse_transform(fitted_scaled_visitors_Megido_IsraeliWeekendTemp)
predicted_train_visitors_IsraeliWeekendTemp = round(pd.Series(fitted_visitors_Megido_IsraeliWeekendTemp[:,0], index= y_train_Megido_IsraeliWeekendTemp.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Megido_IsraeliWeekendTemp = pd.merge(left=train_df_Megido_IsraeliWeekendTemp, right=predicted_train_visitors_IsraeliWeekendTemp , left_index=True, right_index=True)

#train_df_Megido_IsraeliWeekendTemp.head(10)

In [337]:
fitted_scaled_visitors_Megido_IsraeliWeekendTemp = lm.predict(X_test_scaled_Megido_IsraeliWeekendTemp)
fitted_visitors_Megido_IsraeliWeekendTemp = y_test_scaler_Megido_IsraeliWeekendTemp.inverse_transform(fitted_scaled_visitors_Megido_IsraeliWeekendTemp)
predicted_test_visitors_IsraeliWeekendTemp = round(pd.Series(fitted_visitors_Megido_IsraeliWeekendTemp[:,0], index= y_test_Megido_IsraeliWeekendTemp.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Megido_IsraeliWeekendTemp = pd.merge(left=test_df_Megido_IsraeliWeekendTemp, right=predicted_test_visitors_IsraeliWeekendTemp , left_index=True, right_index=True)

#test_df_Megido_IsraeliWeekendTemp.head(10)

In [338]:
from sklearn import metrics
print("std of the objective function: 130.92")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Megido_IsraeliWeekendTemp.Israeli_Visitors, train_df_Megido_IsraeliWeekendTemp.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Megido_IsraeliWeekendTemp.Israeli_Visitors, test_df_Megido_IsraeliWeekendTemp.predicted_test_Visitors)))


std of the objective function: 130.92

------ TRAIN DATA ------
RMSE: 115.74435983312414

------ TEST DATA ------
RMSE: 123.91095950790718


In [339]:
from sklearn.metrics import r2_score
r2_score(train_df_Megido_IsraeliWeekendTemp.Israeli_Visitors, train_df_Megido_IsraeliWeekendTemp.predicted_train_Visitors)

0.19412085000731094

In [340]:
#get R2 for test set
r2_score(test_df_Megido_IsraeliWeekendTemp.Israeli_Visitors, test_df_Megido_IsraeliWeekendTemp.predicted_test_Visitors)

0.15429424587873553

### **Banias**

**Banias Israeli pm25**

In [341]:
BaniasIsraelipm25_df.std()

Israeli_Visitors        716.244751
Temp_C                    7.309618
pm25                     24.723764
Is_Weekend                0.447182
Is_Jewish_Holiday         0.320140
Is_Muslem_Holiday         0.184358
Is_Christian_Holiday      0.125125
Is_water_area             0.000000
Is_Reserve                0.000000
Is_Christian_Site         0.000000
Duration_Of_Visit         0.000000
Winter                    0.404258
Spring                    0.462236
Summer                    0.446610
dtype: float64

In [342]:
from sklearn.model_selection import train_test_split
y_Banias_Israelipm25 = BaniasIsraelipm25_df.Israeli_Visitors
X_Banias_Israelipm25 = BaniasIsraelipm25_df[correlated_BaniasIsraelipm25_df]

X_train_Banias_Israelipm25, X_test_Banias_Israelipm25, y_train_Banias_Israelipm25, y_test_Banias_Israelipm25 = train_test_split(X_Banias_Israelipm25, y_Banias_Israelipm25, test_size=0.3, random_state=1234)
train_df_Banias_Israelipm25 = pd.merge(left=X_train_Banias_Israelipm25, right=y_train_Banias_Israelipm25, left_index=True, right_index=True)
test_df_Banias_Israelipm25 = pd.merge(left=X_test_Banias_Israelipm25, right=y_test_Banias_Israelipm25, left_index=True, right_index=True)
#test_df_Banias_Israelipm25.head(10)

In [343]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Banias_Israelipm25 = StandardScaler()
x_test_scaler_Banias_Israelipm25 = StandardScaler()
y_train_scaler_Banias_Israelipm25 = StandardScaler()
y_test_scaler_Banias_Israelipm25 = StandardScaler()

X_train_scaled_Banias_Israelipm25 = x_train_scaler_Banias_Israelipm25.fit_transform(X_train_Banias_Israelipm25)
X_test_scaled_Banias_Israelipm25 = x_test_scaler_Banias_Israelipm25.fit_transform(X_test_Banias_Israelipm25)
y_train_scaled_Banias_Israelipm25 = y_train_scaler_Banias_Israelipm25.fit_transform(pd.DataFrame(y_train_Banias_Israelipm25))
y_test_scaled_Banias_Israelipm25 = y_test_scaler_Banias_Israelipm25.fit_transform(pd.DataFrame(y_test_Banias_Israelipm25))

In [344]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Banias_Israelipm25,y_train_scaled_Banias_Israelipm25)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_BaniasIsraelipm25_df[0],
      " + ",coeff[1]," * ",correlated_BaniasIsraelipm25_df[1]," + ",coeff[2]," * ",correlated_BaniasIsraelipm25_df[2]," + ",coeff[3]," * ",correlated_BaniasIsraelipm25_df[3]," + ",coeff[4]," * ",correlated_BaniasIsraelipm25_df[4]," + ",coeff[5]," * ",correlated_BaniasIsraelipm25_df[5]," + ",coeff[6]," * ",correlated_BaniasIsraelipm25_df[6]," + ",coeff[7]," * ",correlated_BaniasIsraelipm25_df[7]," + ",coeff[8]," * ",correlated_BaniasIsraelipm25_df[8]," + ",coeff[9]," * ",correlated_BaniasIsraelipm25_df[9]," + ",coeff[10]," * ",correlated_BaniasIsraelipm25_df[10]," + ",coeff[11]," * ",correlated_BaniasIsraelipm25_df[11]," + ",coeff[12]," * ",correlated_BaniasIsraelipm25_df[12])

Coefficients: 
 coeff = [ 5.66895526e-02 -4.70035397e-02  1.04131101e-01  2.36920354e-01
  3.36942165e-02  4.68031820e-02  1.73472348e-17  0.00000000e+00
 -5.55111512e-17  0.00000000e+00  6.63989975e-02  4.79119397e-01
  1.00825131e-01] ,  Intercept= 1.5994441815129707e-17 

The regression equation is: Israeli Visitors =  1.5994441815129707e-17  +  0.05668955264206505  *  Temp_C  +  -0.047003539662839475  *  pm25  +  0.10413110107771581  *  Is_Weekend  +  0.23692035372357148  *  Is_Jewish_Holiday  +  0.033694216533031784  *  Is_Muslem_Holiday  +  0.04680318200420509  *  Is_Christian_Holiday  +  1.734723475976807e-17  *  Is_water_area  +  0.0  *  Is_Reserve  +  -5.551115123125783e-17  *  Is_Christian_Site  +  0.0  *  Duration_Of_Visit  +  0.06639899748480099  *  Winter  +  0.47911939717132923  *  Spring  +  0.10082513120079523  *  Summer


In [345]:
fitted_scaled_visitors_Banias_Israelipm25 = lm.predict(X_train_scaled_Banias_Israelipm25)
fitted_visitors_Banias_Israelipm25 = y_train_scaler_Banias_Israelipm25.inverse_transform(fitted_scaled_visitors_Banias_Israelipm25)
predicted_train_visitors_Israelipm25 = round(pd.Series(fitted_visitors_Banias_Israelipm25[:,0], index= y_train_Banias_Israelipm25.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Banias_Israelipm25 = pd.merge(left=train_df_Banias_Israelipm25, right=predicted_train_visitors_Israelipm25 , left_index=True, right_index=True)

#train_df_Banias_Israelipm25.head(10)

In [346]:
fitted_scaled_visitors_Banias_Israelipm25 = lm.predict(X_test_scaled_Banias_Israelipm25)
fitted_visitors_Banias_Israelipm25 = y_test_scaler_Banias_Israelipm25.inverse_transform(fitted_scaled_visitors_Banias_Israelipm25)
predicted_test_visitors_Israelipm25 = round(pd.Series(fitted_visitors_Banias_Israelipm25[:,0], index= y_test_Banias_Israelipm25.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Banias_Israelipm25 = pd.merge(left=test_df_Banias_Israelipm25, right=predicted_test_visitors_Israelipm25 , left_index=True, right_index=True)

#test_df_Banias_Israelipm25.head(10)

In [347]:
from sklearn import metrics
print("std of the objective function: 716.24")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Banias_Israelipm25.Israeli_Visitors, train_df_Banias_Israelipm25.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Banias_Israelipm25.Israeli_Visitors, test_df_Banias_Israelipm25.predicted_test_Visitors)))

std of the objective function: 716.24

------ TRAIN DATA ------
RMSE: 625.3067938213916

------ TEST DATA ------
RMSE: 586.6467942628408


In [348]:
from sklearn.metrics import r2_score
r2_score(train_df_Banias_Israelipm25.Israeli_Visitors, train_df_Banias_Israelipm25.predicted_train_Visitors)

0.27327321694055096

In [349]:
#get R2 for test set
r2_score(test_df_Banias_Israelipm25.Israeli_Visitors, test_df_Banias_Israelipm25.predicted_test_Visitors)

0.23990121562119004

**Banias Israeli no2**

In [350]:
BaniasIsraelino2_df.std()

Israeli_Visitors        640.995090
Temp_C                    7.400320
no2                       5.064207
Is_Weekend                0.450025
Is_Jewish_Holiday         0.301510
Is_Muslem_Holiday         0.185672
Is_Christian_Holiday      0.130627
Is_water_area             0.000000
Is_Reserve                0.000000
Is_Christian_Site         0.000000
Duration_Of_Visit         0.000000
Winter                    0.415420
Spring                    0.442801
Summer                    0.448019
dtype: float64

In [351]:
from sklearn.model_selection import train_test_split
y_Banias_Israelino2 = BaniasIsraelino2_df.Israeli_Visitors
X_Banias_Israelino2 = BaniasIsraelino2_df[correlated_BaniasIsraelino2_df]

X_train_Banias_Israelino2, X_test_Banias_Israelino2, y_train_Banias_Israelino2, y_test_Banias_Israelino2 = train_test_split(X_Banias_Israelino2, y_Banias_Israelino2, test_size=0.3, random_state=1234)
train_df_Banias_Israelino2 = pd.merge(left=X_train_Banias_Israelino2, right=y_train_Banias_Israelino2, left_index=True, right_index=True)
test_df_Banias_Israelino2 = pd.merge(left=X_test_Banias_Israelino2, right=y_test_Banias_Israelino2, left_index=True, right_index=True)
#test_df_Banias_Israelino2.head(10)

In [352]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Banias_Israelino2 = StandardScaler()
x_test_scaler_Banias_Israelino2 = StandardScaler()
y_train_scaler_Banias_Israelino2 = StandardScaler()
y_test_scaler_Banias_Israelino2 = StandardScaler()

X_train_scaled_Banias_Israelino2 = x_train_scaler_Banias_Israelino2.fit_transform(X_train_Banias_Israelino2)
X_test_scaled_Banias_Israelino2 = x_test_scaler_Banias_Israelino2.fit_transform(X_test_Banias_Israelino2)
y_train_scaled_Banias_Israelino2 = y_train_scaler_Banias_Israelino2.fit_transform(pd.DataFrame(y_train_Banias_Israelino2))
y_test_scaled_Banias_Israelino2 = y_test_scaler_Banias_Israelino2.fit_transform(pd.DataFrame(y_test_Banias_Israelino2))

In [353]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Banias_Israelino2,y_train_scaled_Banias_Israelino2)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_BaniasIsraelino2_df[0],
      " + ",coeff[1]," * ",correlated_BaniasIsraelino2_df[1]," + ",coeff[2]," * ",correlated_BaniasIsraelino2_df[2]," + ",coeff[3]," * ",correlated_BaniasIsraelino2_df[3]," + ",coeff[4]," * ",correlated_BaniasIsraelino2_df[4]," + ",coeff[5]," * ",correlated_BaniasIsraelino2_df[5]," + ",coeff[6]," * ",correlated_BaniasIsraelino2_df[6]," + ",coeff[7]," * ",correlated_BaniasIsraelino2_df[7]," + ",coeff[8]," * ",correlated_BaniasIsraelino2_df[8]," + ",coeff[9]," * ",correlated_BaniasIsraelino2_df[9]," + ",coeff[10]," * ",correlated_BaniasIsraelino2_df[10]," + ",coeff[11]," * ",correlated_BaniasIsraelino2_df[11]," + ",coeff[12]," * ",correlated_BaniasIsraelino2_df[12])

Coefficients: 
 coeff = [ 3.60552061e-02  7.51637897e-03  1.37710602e-01  1.41575512e-01
  8.53917121e-02  4.55827209e-02  0.00000000e+00  0.00000000e+00
 -1.38777878e-17  0.00000000e+00  4.94242387e-04  4.55390821e-01
  6.28706467e-02] ,  Intercept= 7.004593199934933e-18 

The regression equation is: Israeli Visitors =  7.004593199934933e-18  +  0.0360552061172725  *  Temp_C  +  0.007516378968329188  *  no2  +  0.1377106022344623  *  Is_Weekend  +  0.14157551168584578  *  Is_Jewish_Holiday  +  0.08539171205022768  *  Is_Muslem_Holiday  +  0.04558272092329096  *  Is_Christian_Holiday  +  0.0  *  Is_water_area  +  0.0  *  Is_Reserve  +  -1.3877787807814457e-17  *  Is_Christian_Site  +  0.0  *  Duration_Of_Visit  +  0.0004942423874447072  *  Winter  +  0.4553908208512864  *  Spring  +  0.0628706467259667  *  Summer


In [354]:
fitted_scaled_visitors_Banias_Israelino2 = lm.predict(X_train_scaled_Banias_Israelino2)
fitted_visitors_Banias_Israelino2 = y_train_scaler_Banias_Israelino2.inverse_transform(fitted_scaled_visitors_Banias_Israelino2)
predicted_train_visitors_Israelino2 = round(pd.Series(fitted_visitors_Banias_Israelino2[:,0], index= y_train_Banias_Israelino2.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Banias_Israelino2 = pd.merge(left=train_df_Banias_Israelino2, right=predicted_train_visitors_Israelino2 , left_index=True, right_index=True)

#train_df_Banias_Israelino2.head(10)

In [355]:
fitted_scaled_visitors_Banias_Israelino2 = lm.predict(X_test_scaled_Banias_Israelino2)
fitted_visitors_Banias_Israelino2 = y_test_scaler_Banias_Israelino2.inverse_transform(fitted_scaled_visitors_Banias_Israelino2)
predicted_test_visitors_Israelino2 = round(pd.Series(fitted_visitors_Banias_Israelino2[:,0], index= y_test_Banias_Israelino2.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Banias_Israelino2 = pd.merge(left=test_df_Banias_Israelino2, right=predicted_test_visitors_Israelino2 , left_index=True, right_index=True)

#test_df_Banias_Israelino2.head(10)

In [356]:
from sklearn import metrics
print("std of the objective function: 640.99")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Banias_Israelino2.Israeli_Visitors, train_df_Banias_Israelino2.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Banias_Israelino2.Israeli_Visitors, test_df_Banias_Israelino2.predicted_test_Visitors)))

std of the objective function: 640.99

------ TRAIN DATA ------
RMSE: 552.153852151265

------ TEST DATA ------
RMSE: 590.7640029901208


In [357]:
from sklearn.metrics import r2_score
r2_score(train_df_Banias_Israelino2.Israeli_Visitors, train_df_Banias_Israelino2.predicted_train_Visitors)

0.24741225588528504

In [358]:
#get R2 for test set
r2_score(test_df_Banias_Israelino2.Israeli_Visitors, test_df_Banias_Israelino2.predicted_test_Visitors)

0.17464349031456805

**Banias Tourist pm25**

In [359]:
BaniasTouristpm25_df.std()

Tourist_Visitors        476.404374
Temp_C                    7.309618
pm25                     24.723764
Is_Weekend                0.447182
Is_Jewish_Holiday         0.320140
Is_Muslem_Holiday         0.184358
Is_Christian_Holiday      0.125125
Is_water_area             0.000000
Is_Reserve                0.000000
Is_Christian_Site         0.000000
Duration_Of_Visit         0.000000
Winter                    0.404258
Spring                    0.462236
Summer                    0.446610
dtype: float64

In [360]:
from sklearn.model_selection import train_test_split
y_Banias_Touristpm25 = BaniasTouristpm25_df.Tourist_Visitors
X_Banias_Touristpm25 = BaniasTouristpm25_df[correlated_BaniasTouristpm25_df]

X_train_Banias_Touristpm25, X_test_Banias_Touristpm25, y_train_Banias_Touristpm25, y_test_Banias_Touristpm25 = train_test_split(X_Banias_Touristpm25, y_Banias_Touristpm25, test_size=0.3, random_state=1234)
train_df_Banias_Touristpm25 = pd.merge(left=X_train_Banias_Touristpm25, right=y_train_Banias_Touristpm25, left_index=True, right_index=True)
test_df_Banias_Touristpm25 = pd.merge(left=X_test_Banias_Touristpm25, right=y_test_Banias_Touristpm25, left_index=True, right_index=True)
#test_df_Banias_Touristpm25.head(10)

In [361]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Banias_Touristpm25 = StandardScaler()
x_test_scaler_Banias_Touristpm25 = StandardScaler()
y_train_scaler_Banias_Touristpm25 = StandardScaler()
y_test_scaler_Banias_Touristpm25 = StandardScaler()

X_train_scaled_Banias_Touristpm25 = x_train_scaler_Banias_Touristpm25.fit_transform(X_train_Banias_Touristpm25)
X_test_scaled_Banias_Touristpm25 = x_test_scaler_Banias_Touristpm25.fit_transform(X_test_Banias_Touristpm25)
y_train_scaled_Banias_Touristpm25 = y_train_scaler_Banias_Touristpm25.fit_transform(pd.DataFrame(y_train_Banias_Touristpm25))
y_test_scaled_Banias_Touristpm25 = y_test_scaler_Banias_Touristpm25.fit_transform(pd.DataFrame(y_test_Banias_Touristpm25))

In [362]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Banias_Touristpm25,y_train_scaled_Banias_Touristpm25)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Tourist Visitors = " ,intercept," + ",coeff[0]," * ",correlated_BaniasTouristpm25_df[0],
      " + ",coeff[1]," * ",correlated_BaniasTouristpm25_df[1]," + ",coeff[2]," * ",correlated_BaniasTouristpm25_df[2]," + ",coeff[3]," * ",correlated_BaniasTouristpm25_df[3]," + ",coeff[4]," * ",correlated_BaniasTouristpm25_df[4]," + ",coeff[5]," * ",correlated_BaniasTouristpm25_df[5]," + ",coeff[6]," * ",correlated_BaniasTouristpm25_df[6]," + ",coeff[7]," * ",correlated_BaniasTouristpm25_df[7]," + ",coeff[8]," * ",correlated_BaniasTouristpm25_df[8]," + ",coeff[9]," * ",correlated_BaniasTouristpm25_df[9]," + ",coeff[10]," * ",correlated_BaniasTouristpm25_df[10]," + ",coeff[11]," * ",correlated_BaniasTouristpm25_df[11]," + ",coeff[12]," * ",correlated_BaniasTouristpm25_df[12])

Coefficients: 
 coeff = [-1.29217552e-01  4.00771556e-03 -4.44428898e-02 -2.14920478e-01
 -2.91479927e-02 -1.22186810e-01 -2.77555756e-17 -2.77555756e-17
  4.16333634e-17  0.00000000e+00 -2.41760217e-01  1.77246902e-01
 -2.66985063e-01] ,  Intercept= -2.073814139114585e-16 

The regression equation is: Tourist Visitors =  -2.073814139114585e-16  +  -0.129217551743191  *  Temp_C  +  0.004007715555206527  *  pm25  +  -0.04444288979802495  *  Is_Weekend  +  -0.2149204775674209  *  Is_Jewish_Holiday  +  -0.029147992693401768  *  Is_Muslem_Holiday  +  -0.12218680983525962  *  Is_Christian_Holiday  +  -2.7755575615628914e-17  *  Is_water_area  +  -2.7755575615628914e-17  *  Is_Reserve  +  4.163336342344337e-17  *  Is_Christian_Site  +  0.0  *  Duration_Of_Visit  +  -0.24176021718802515  *  Winter  +  0.1772469017039482  *  Spring  +  -0.2669850633708432  *  Summer


In [363]:
fitted_scaled_visitors_Banias_Touristpm25 = lm.predict(X_train_scaled_Banias_Touristpm25)
fitted_visitors_Banias_Touristpm25 = y_train_scaler_Banias_Touristpm25.inverse_transform(fitted_scaled_visitors_Banias_Touristpm25)
predicted_train_visitors_Touristpm25 = round(pd.Series(fitted_visitors_Banias_Touristpm25[:,0], index= y_train_Banias_Touristpm25.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Banias_Touristpm25 = pd.merge(left=train_df_Banias_Touristpm25, right=predicted_train_visitors_Touristpm25 , left_index=True, right_index=True)

#train_df_Banias_Touristpm25.head(10)

In [364]:
fitted_scaled_visitors_Banias_Touristpm25 = lm.predict(X_test_scaled_Banias_Touristpm25)
fitted_visitors_Banias_Touristpm25 = y_test_scaler_Banias_Touristpm25.inverse_transform(fitted_scaled_visitors_Banias_Touristpm25)
predicted_test_visitors_Touristpm25 = round(pd.Series(fitted_visitors_Banias_Touristpm25[:,0], index= y_test_Banias_Touristpm25.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Banias_Touristpm25 = pd.merge(left=test_df_Banias_Touristpm25, right=predicted_test_visitors_Touristpm25 , left_index=True, right_index=True)

#test_df_Banias_Touristpm25.head(10)

In [365]:
from sklearn import metrics
print("std of the objective function: 476.4")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Banias_Touristpm25.Tourist_Visitors, train_df_Banias_Touristpm25.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Banias_Touristpm25.Tourist_Visitors, test_df_Banias_Touristpm25.predicted_test_Visitors)))

std of the objective function: 476.4

------ TRAIN DATA ------
RMSE: 413.28553591210505

------ TEST DATA ------
RMSE: 416.5723656130503


In [366]:
from sklearn.metrics import r2_score
r2_score(train_df_Banias_Touristpm25.Tourist_Visitors, train_df_Banias_Touristpm25.predicted_train_Visitors)

0.24155291752523356

In [367]:
#get R2 for test set
r2_score(test_df_Banias_Touristpm25.Tourist_Visitors, test_df_Banias_Touristpm25.predicted_test_Visitors)

0.24440279322410952

**Banias Tourist no2**

In [368]:
BaniasTouristno2_df.std()

Tourist_Visitors        480.603638
Temp_C                    7.400320
no2                       5.064207
Is_Weekend                0.450025
Is_Jewish_Holiday         0.301510
Is_Muslem_Holiday         0.185672
Is_Christian_Holiday      0.130627
Is_water_area             0.000000
Is_Reserve                0.000000
Is_Christian_Site         0.000000
Duration_Of_Visit         0.000000
Winter                    0.415420
Spring                    0.442801
Summer                    0.448019
dtype: float64

In [369]:
from sklearn.model_selection import train_test_split
y_Banias_Touristno2 = BaniasTouristno2_df.Tourist_Visitors
X_Banias_Touristno2 = BaniasTouristno2_df[correlated_BaniasTouristno2_df]

X_train_Banias_Touristno2, X_test_Banias_Touristno2, y_train_Banias_Touristno2, y_test_Banias_Touristno2 = train_test_split(X_Banias_Touristno2, y_Banias_Touristno2, test_size=0.3, random_state=1234)
train_df_Banias_Touristno2 = pd.merge(left=X_train_Banias_Touristno2, right=y_train_Banias_Touristno2, left_index=True, right_index=True)
test_df_Banias_Touristno2 = pd.merge(left=X_test_Banias_Touristno2, right=y_test_Banias_Touristno2, left_index=True, right_index=True)
#test_df_Banias_Touristno2.head(10)

In [370]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Banias_Touristno2 = StandardScaler()
x_test_scaler_Banias_Touristno2 = StandardScaler()
y_train_scaler_Banias_Touristno2 = StandardScaler()
y_test_scaler_Banias_Touristno2 = StandardScaler()

X_train_scaled_Banias_Touristno2 = x_train_scaler_Banias_Touristno2.fit_transform(X_train_Banias_Touristno2)
X_test_scaled_Banias_Touristno2 = x_test_scaler_Banias_Touristno2.fit_transform(X_test_Banias_Touristno2)
y_train_scaled_Banias_Touristno2 = y_train_scaler_Banias_Touristno2.fit_transform(pd.DataFrame(y_train_Banias_Touristno2))
y_test_scaled_Banias_Touristno2 = y_test_scaler_Banias_Touristno2.fit_transform(pd.DataFrame(y_test_Banias_Touristno2))

In [371]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Banias_Touristno2,y_train_scaled_Banias_Touristno2)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Tourist Visitors = " ,intercept," + ",coeff[0]," * ",correlated_BaniasTouristno2_df[0],
      " + ",coeff[1]," * ",correlated_BaniasTouristno2_df[1]," + ",coeff[2]," * ",correlated_BaniasTouristno2_df[2]," + ",coeff[3]," * ",correlated_BaniasTouristno2_df[3]," + ",coeff[4]," * ",correlated_BaniasTouristno2_df[4]," + ",coeff[5]," * ",correlated_BaniasTouristno2_df[5]," + ",coeff[6]," * ",correlated_BaniasTouristno2_df[6]," + ",coeff[7]," * ",correlated_BaniasTouristno2_df[7]," + ",coeff[8]," * ",correlated_BaniasTouristno2_df[8]," + ",coeff[9]," * ",correlated_BaniasTouristno2_df[9]," + ",coeff[10]," * ",correlated_BaniasTouristno2_df[10]," + ",coeff[11]," * ",correlated_BaniasTouristno2_df[11]," + ",coeff[12]," * ",correlated_BaniasTouristno2_df[12])

Coefficients: 
 coeff = [-1.41058816e-01  5.68159026e-03 -5.23663766e-02 -2.18754115e-01
 -2.56606430e-02 -7.77669362e-02  0.00000000e+00 -1.38777878e-17
  5.55111512e-17  0.00000000e+00 -3.67818233e-01  1.01830945e-01
 -3.70664394e-01] ,  Intercept= -3.0793132290898014e-17 

The regression equation is: Tourist Visitors =  -3.0793132290898014e-17  +  -0.14105881590959107  *  Temp_C  +  0.005681590257457604  *  no2  +  -0.05236637660989467  *  Is_Weekend  +  -0.2187541154026806  *  Is_Jewish_Holiday  +  -0.02566064301143113  *  Is_Muslem_Holiday  +  -0.07776693622598654  *  Is_Christian_Holiday  +  0.0  *  Is_water_area  +  -1.3877787807814457e-17  *  Is_Reserve  +  5.551115123125783e-17  *  Is_Christian_Site  +  0.0  *  Duration_Of_Visit  +  -0.36781823305580436  *  Winter  +  0.1018309453885394  *  Spring  +  -0.3706643940580203  *  Summer


In [372]:
fitted_scaled_visitors_Banias_Touristno2 = lm.predict(X_train_scaled_Banias_Touristno2)
fitted_visitors_Banias_Touristno2 = y_train_scaler_Banias_Touristno2.inverse_transform(fitted_scaled_visitors_Banias_Touristno2)
predicted_train_visitors_Touristno2 = round(pd.Series(fitted_visitors_Banias_Touristno2[:,0], index= y_train_Banias_Touristno2.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Banias_Touristno2 = pd.merge(left=train_df_Banias_Touristno2, right=predicted_train_visitors_Touristno2 , left_index=True, right_index=True)

#train_df_Banias_Touristno2.head(10)

In [373]:
fitted_scaled_visitors_Banias_Touristno2 = lm.predict(X_test_scaled_Banias_Touristno2)
fitted_visitors_Banias_Touristno2 = y_test_scaler_Banias_Touristno2.inverse_transform(fitted_scaled_visitors_Banias_Touristno2)
predicted_test_visitors_Touristno2 = round(pd.Series(fitted_visitors_Banias_Touristno2[:,0], index= y_test_Banias_Touristno2.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Banias_Touristno2 = pd.merge(left=test_df_Banias_Touristno2, right=predicted_test_visitors_Touristno2 , left_index=True, right_index=True)

#test_df_Banias_Touristno2.head(10)

In [374]:
from sklearn import metrics
print("std of the objective function: 480.6")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Banias_Touristno2.Tourist_Visitors, train_df_Banias_Touristno2.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Banias_Touristno2.Tourist_Visitors, test_df_Banias_Touristno2.predicted_test_Visitors)))

std of the objective function: 480.6

------ TRAIN DATA ------
RMSE: 417.88544522692735

------ TEST DATA ------
RMSE: 394.6544703048892


In [375]:
from sklearn.metrics import r2_score
r2_score(train_df_Banias_Touristno2.Tourist_Visitors, train_df_Banias_Touristno2.predicted_train_Visitors)

0.28271423576802546

In [376]:
#get R2 for test set
r2_score(test_df_Banias_Touristno2.Tourist_Visitors, test_df_Banias_Touristno2.predicted_test_Visitors)

0.2252444784427462

### **Gamla**

**Gamla Israeli pm25**

In [377]:
GamlaIsraelipm25_df.std()

Israeli_Visitors        210.895199
Temp_C                    7.560160
pm25                     24.596082
Is_Weekend                0.449325
Is_Jewish_Holiday         0.320852
Is_Muslem_Holiday         0.179789
Is_Christian_Holiday      0.126173
Is_Reserve                0.000000
Is_Jewish_Site            0.000000
Duration_Of_Visit         0.000000
Winter                    0.415804
Spring                    0.454939
Summer                    0.439295
dtype: float64

In [378]:
from sklearn.model_selection import train_test_split
y_Gamla_Israelipm25 = GamlaIsraelipm25_df.Israeli_Visitors
X_Gamla_Israelipm25 = GamlaIsraelipm25_df[correlated_GamlaIsraelipm25_df]

X_train_Gamla_Israelipm25, X_test_Gamla_Israelipm25, y_train_Gamla_Israelipm25, y_test_Gamla_Israelipm25 = train_test_split(X_Gamla_Israelipm25, y_Gamla_Israelipm25, test_size=0.3, random_state=1234)
train_df_Gamla_Israelipm25 = pd.merge(left=X_train_Gamla_Israelipm25, right=y_train_Gamla_Israelipm25, left_index=True, right_index=True)
test_df_Gamla_Israelipm25 = pd.merge(left=X_test_Gamla_Israelipm25, right=y_test_Gamla_Israelipm25, left_index=True, right_index=True)
#test_df_Gamla_Israelipm25.head(10)

In [379]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Gamla_Israelipm25 = StandardScaler()
x_test_scaler_Gamla_Israelipm25 = StandardScaler()
y_train_scaler_Gamla_Israelipm25 = StandardScaler()
y_test_scaler_Gamla_Israelipm25 = StandardScaler()

X_train_scaled_Gamla_Israelipm25 = x_train_scaler_Gamla_Israelipm25.fit_transform(X_train_Gamla_Israelipm25)
X_test_scaled_Gamla_Israelipm25 = x_test_scaler_Gamla_Israelipm25.fit_transform(X_test_Gamla_Israelipm25)
y_train_scaled_Gamla_Israelipm25 = y_train_scaler_Gamla_Israelipm25.fit_transform(pd.DataFrame(y_train_Gamla_Israelipm25))
y_test_scaled_Gamla_Israelipm25 = y_test_scaler_Gamla_Israelipm25.fit_transform(pd.DataFrame(y_test_Gamla_Israelipm25))

In [380]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Gamla_Israelipm25,y_train_scaled_Gamla_Israelipm25)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_GamlaIsraelipm25_df[0],
      " + ",coeff[1]," * ",correlated_GamlaIsraelipm25_df[1]," + ",coeff[2]," * ",correlated_GamlaIsraelipm25_df[2]," + ",coeff[3]," * ",correlated_GamlaIsraelipm25_df[3]," + ",coeff[4]," * ",correlated_GamlaIsraelipm25_df[4]," + ",coeff[5]," * ",correlated_GamlaIsraelipm25_df[5]," + ",coeff[6]," * ",correlated_GamlaIsraelipm25_df[6]," + ",coeff[7]," * ",correlated_GamlaIsraelipm25_df[7]," + ",coeff[8]," * ",correlated_GamlaIsraelipm25_df[8]," + ",coeff[9]," * ",correlated_GamlaIsraelipm25_df[9]," + ",coeff[10]," * ",correlated_GamlaIsraelipm25_df[10]," + ",coeff[11]," * ",correlated_GamlaIsraelipm25_df[11])

Coefficients: 
 coeff = [-2.76168132e-01  1.08385321e-03  4.68595213e-02  1.68890528e-01
 -4.91676470e-04  1.05369840e-01 -8.67361738e-19  6.93889390e-18
  1.38777878e-17 -1.52479662e-01  2.99914507e-01 -3.91041443e-02] ,  Intercept= 2.209738157935718e-16 

The regression equation is: Israeli Visitors =  2.209738157935718e-16  +  -0.2761681319643893  *  Temp_C  +  0.0010838532078093072  *  pm25  +  0.04685952129455123  *  Is_Weekend  +  0.1688905280672471  *  Is_Jewish_Holiday  +  -0.0004916764696808931  *  Is_Muslem_Holiday  +  0.10536984049572122  *  Is_Christian_Holiday  +  -8.673617379884035e-19  *  Is_Reserve  +  6.938893903907228e-18  *  Is_Jewish_Site  +  1.3877787807814457e-17  *  Duration_Of_Visit  +  -0.15247966150727033  *  Winter  +  0.2999145070510516  *  Spring  +  -0.039104144257851796  *  Summer


In [381]:
fitted_scaled_visitors_Gamla_Israelipm25 = lm.predict(X_train_scaled_Gamla_Israelipm25)
fitted_visitors_Gamla_Israelipm25 = y_train_scaler_Gamla_Israelipm25.inverse_transform(fitted_scaled_visitors_Gamla_Israelipm25)
predicted_train_visitors_Israelipm25 = round(pd.Series(fitted_visitors_Gamla_Israelipm25[:,0], index= y_train_Gamla_Israelipm25.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Gamla_Israelipm25 = pd.merge(left=train_df_Gamla_Israelipm25, right=predicted_train_visitors_Israelipm25 , left_index=True, right_index=True)

#train_df_Gamla_Israelipm25.head(10)

In [382]:
fitted_scaled_visitors_Gamla_Israelipm25 = lm.predict(X_test_scaled_Gamla_Israelipm25)
fitted_visitors_Gamla_Israelipm25 = y_test_scaler_Gamla_Israelipm25.inverse_transform(fitted_scaled_visitors_Gamla_Israelipm25)
predicted_test_visitors_Israelipm25 = round(pd.Series(fitted_visitors_Gamla_Israelipm25[:,0], index= y_test_Gamla_Israelipm25.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Gamla_Israelipm25 = pd.merge(left=test_df_Gamla_Israelipm25, right=predicted_test_visitors_Israelipm25 , left_index=True, right_index=True)

#test_df_Gamla_Israelipm25.head(10)

In [383]:
from sklearn import metrics
print("std of the objective function: 210.89")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Gamla_Israelipm25.Israeli_Visitors, train_df_Gamla_Israelipm25.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Gamla_Israelipm25.Israeli_Visitors, test_df_Gamla_Israelipm25.predicted_test_Visitors)))

std of the objective function: 210.89

------ TRAIN DATA ------
RMSE: 177.7115737841987

------ TEST DATA ------
RMSE: 180.54226120000678


In [384]:
from sklearn.metrics import r2_score
r2_score(train_df_Gamla_Israelipm25.Israeli_Visitors, train_df_Gamla_Israelipm25.predicted_train_Visitors)

0.27126582427844714

In [385]:
#get R2 for test set
r2_score(test_df_Gamla_Israelipm25.Israeli_Visitors, test_df_Gamla_Israelipm25.predicted_test_Visitors)

0.30153061370726875

**Gamla Israeli no2**

In [386]:
GamlaIsraelino2_df.std()

Israeli_Visitors        196.986104
Temp_C                    7.685817
no2                       5.026052
Is_Weekend                0.451843
Is_Jewish_Holiday         0.303273
Is_Muslem_Holiday         0.179935
Is_Christian_Holiday      0.135533
Is_Reserve                0.000000
Is_Jewish_Site            0.000000
Duration_Of_Visit         0.000000
Winter                    0.425578
Spring                    0.437133
Summer                    0.442987
dtype: float64

In [387]:
from sklearn.model_selection import train_test_split
y_Gamla_Israelino2 = GamlaIsraelino2_df.Israeli_Visitors
X_Gamla_Israelino2 = GamlaIsraelino2_df[correlated_GamlaIsraelino2_df]

X_train_Gamla_Israelino2, X_test_Gamla_Israelino2, y_train_Gamla_Israelino2, y_test_Gamla_Israelino2 = train_test_split(X_Gamla_Israelino2, y_Gamla_Israelino2, test_size=0.3, random_state=1234)
train_df_Gamla_Israelino2 = pd.merge(left=X_train_Gamla_Israelino2, right=y_train_Gamla_Israelino2, left_index=True, right_index=True)
test_df_Gamla_Israelino2 = pd.merge(left=X_test_Gamla_Israelino2, right=y_test_Gamla_Israelino2, left_index=True, right_index=True)
#test_df_Gamla_Israelino2.head(10)

In [388]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Gamla_Israelino2 = StandardScaler()
x_test_scaler_Gamla_Israelino2 = StandardScaler()
y_train_scaler_Gamla_Israelino2 = StandardScaler()
y_test_scaler_Gamla_Israelino2 = StandardScaler()

X_train_scaled_Gamla_Israelino2 = x_train_scaler_Gamla_Israelino2.fit_transform(X_train_Gamla_Israelino2)
X_test_scaled_Gamla_Israelino2 = x_test_scaler_Gamla_Israelino2.fit_transform(X_test_Gamla_Israelino2)
y_train_scaled_Gamla_Israelino2 = y_train_scaler_Gamla_Israelino2.fit_transform(pd.DataFrame(y_train_Gamla_Israelino2))
y_test_scaled_Gamla_Israelino2 = y_test_scaler_Gamla_Israelino2.fit_transform(pd.DataFrame(y_test_Gamla_Israelino2))

In [389]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Gamla_Israelino2,y_train_scaled_Gamla_Israelino2)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_GamlaIsraelino2_df[0],
      " + ",coeff[1]," * ",correlated_GamlaIsraelino2_df[1]," + ",coeff[2]," * ",correlated_GamlaIsraelino2_df[2]," + ",coeff[3]," * ",correlated_GamlaIsraelino2_df[3]," + ",coeff[4]," * ",correlated_GamlaIsraelino2_df[4]," + ",coeff[5]," * ",correlated_GamlaIsraelino2_df[5]," + ",coeff[6]," * ",correlated_GamlaIsraelino2_df[6]," + ",coeff[7]," * ",correlated_GamlaIsraelino2_df[7]," + ",coeff[8]," * ",correlated_GamlaIsraelino2_df[8]," + ",coeff[9]," * ",correlated_GamlaIsraelino2_df[9]," + ",coeff[10]," * ",correlated_GamlaIsraelino2_df[10]," + ",coeff[11]," * ",correlated_GamlaIsraelino2_df[11])

Coefficients: 
 coeff = [-2.91323641e-01 -1.68709179e-02  5.83828147e-02  1.08609339e-01
 -6.60747365e-03  5.16977336e-02 -7.80625564e-18 -1.95156391e-18
  6.93889390e-18 -1.74271871e-01  3.16395798e-01 -4.83105234e-02] ,  Intercept= -1.3563536062148417e-17 

The regression equation is: Israeli Visitors =  -1.3563536062148417e-17  +  -0.2913236414763191  *  Temp_C  +  -0.01687091791104016  *  no2  +  0.05838281469828483  *  Is_Weekend  +  0.10860933905119882  *  Is_Jewish_Holiday  +  -0.006607473653170581  *  Is_Muslem_Holiday  +  0.05169773364033608  *  Is_Christian_Holiday  +  -7.806255641895632e-18  *  Is_Reserve  +  -1.951563910473908e-18  *  Is_Jewish_Site  +  6.938893903907228e-18  *  Duration_Of_Visit  +  -0.17427187091180846  *  Winter  +  0.31639579793506795  *  Spring  +  -0.048310523419508526  *  Summer


In [390]:
fitted_scaled_visitors_Gamla_Israelino2 = lm.predict(X_train_scaled_Gamla_Israelino2)
fitted_visitors_Gamla_Israelino2 = y_train_scaler_Gamla_Israelino2.inverse_transform(fitted_scaled_visitors_Gamla_Israelino2)
predicted_train_visitors_Israelino2 = round(pd.Series(fitted_visitors_Gamla_Israelino2[:,0], index= y_train_Gamla_Israelino2.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Gamla_Israelino2 = pd.merge(left=train_df_Gamla_Israelino2, right=predicted_train_visitors_Israelino2 , left_index=True, right_index=True)

#train_df_Gamla_Israelino2.head(10)

In [391]:
fitted_scaled_visitors_Gamla_Israelino2 = lm.predict(X_test_scaled_Gamla_Israelino2)
fitted_visitors_Gamla_Israelino2 = y_test_scaler_Gamla_Israelino2.inverse_transform(fitted_scaled_visitors_Gamla_Israelino2)
predicted_test_visitors_Israelino2 = round(pd.Series(fitted_visitors_Gamla_Israelino2[:,0], index= y_test_Gamla_Israelino2.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Gamla_Israelino2 = pd.merge(left=test_df_Gamla_Israelino2, right=predicted_test_visitors_Israelino2 , left_index=True, right_index=True)

#test_df_Gamla_Israelino2.head(10)

In [392]:
from sklearn import metrics
print("std of the objective function: 196.98")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Gamla_Israelino2.Israeli_Visitors, train_df_Gamla_Israelino2.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Gamla_Israelino2.Israeli_Visitors, test_df_Gamla_Israelino2.predicted_test_Visitors)))

std of the objective function: 196.98

------ TRAIN DATA ------
RMSE: 179.7682788716267

------ TEST DATA ------
RMSE: 149.31540320990902


In [393]:
from sklearn.metrics import r2_score
r2_score(train_df_Gamla_Israelino2.Israeli_Visitors, train_df_Gamla_Israelino2.predicted_train_Visitors)

0.24774913607535032

In [394]:
#get R2 for test set
r2_score(test_df_Gamla_Israelino2.Israeli_Visitors, test_df_Gamla_Israelino2.predicted_test_Visitors)

0.2270085684346954

**Gamla Tourist pm25**

In [395]:
GamlaTouristpm25_df.std()

Tourist_Visitors        48.593681
Temp_C                   7.560160
pm25                    24.596082
Is_Weekend               0.449325
Is_Jewish_Holiday        0.320852
Is_Muslem_Holiday        0.179789
Is_Christian_Holiday     0.126173
Is_Reserve               0.000000
Is_Jewish_Site           0.000000
Duration_Of_Visit        0.000000
Winter                   0.415804
Spring                   0.454939
Summer                   0.439295
dtype: float64

In [396]:
from sklearn.model_selection import train_test_split
y_Gamla_Touristpm25 = GamlaTouristpm25_df.Tourist_Visitors
X_Gamla_Touristpm25 = GamlaTouristpm25_df[correlated_GamlaTouristpm25_df]

X_train_Gamla_Touristpm25, X_test_Gamla_Touristpm25, y_train_Gamla_Touristpm25, y_test_Gamla_Touristpm25 = train_test_split(X_Gamla_Touristpm25, y_Gamla_Touristpm25, test_size=0.3, random_state=1234)
train_df_Gamla_Touristpm25 = pd.merge(left=X_train_Gamla_Touristpm25, right=y_train_Gamla_Touristpm25, left_index=True, right_index=True)
test_df_Gamla_Touristpm25 = pd.merge(left=X_test_Gamla_Touristpm25, right=y_test_Gamla_Touristpm25, left_index=True, right_index=True)
#test_df_Gamla_Touristpm25.head(10)

In [397]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Gamla_Touristpm25 = StandardScaler()
x_test_scaler_Gamla_Touristpm25 = StandardScaler()
y_train_scaler_Gamla_Touristpm25 = StandardScaler()
y_test_scaler_Gamla_Touristpm25 = StandardScaler()

X_train_scaled_Gamla_Touristpm25 = x_train_scaler_Gamla_Touristpm25.fit_transform(X_train_Gamla_Touristpm25)
X_test_scaled_Gamla_Touristpm25 = x_test_scaler_Gamla_Touristpm25.fit_transform(X_test_Gamla_Touristpm25)
y_train_scaled_Gamla_Touristpm25 = y_train_scaler_Gamla_Touristpm25.fit_transform(pd.DataFrame(y_train_Gamla_Touristpm25))
y_test_scaled_Gamla_Touristpm25 = y_test_scaler_Gamla_Touristpm25.fit_transform(pd.DataFrame(y_test_Gamla_Touristpm25))

In [398]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Gamla_Touristpm25,y_train_scaled_Gamla_Touristpm25)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Tourist Visitors = " ,intercept," + ",coeff[0]," * ",correlated_GamlaTouristpm25_df[0],
      " + ",coeff[1]," * ",correlated_GamlaTouristpm25_df[1]," + ",coeff[2]," * ",correlated_GamlaTouristpm25_df[2]," + ",coeff[3]," * ",correlated_GamlaTouristpm25_df[3]," + ",coeff[4]," * ",correlated_GamlaTouristpm25_df[4]," + ",coeff[5]," * ",correlated_GamlaTouristpm25_df[5]," + ",coeff[6]," * ",correlated_GamlaTouristpm25_df[6]," + ",coeff[7]," * ",correlated_GamlaTouristpm25_df[7]," + ",coeff[8]," * ",correlated_GamlaTouristpm25_df[8]," + ",coeff[9]," * ",correlated_GamlaTouristpm25_df[9]," + ",coeff[10]," * ",correlated_GamlaTouristpm25_df[10]," + ",coeff[11]," * ",correlated_GamlaTouristpm25_df[11])

Coefficients: 
 coeff = [-1.03875283e-01  1.24194399e-02 -1.50739202e-01 -1.42557469e-01
 -2.53597684e-02  4.22700914e-02  2.77555756e-17  0.00000000e+00
 -2.77555756e-17 -7.10590606e-02  2.46297778e-01 -2.73542211e-02] ,  Intercept= 1.094299585017343e-16 

The regression equation is: Tourist Visitors =  1.094299585017343e-16  +  -0.10387528334478494  *  Temp_C  +  0.01241943985610135  *  pm25  +  -0.15073920176835762  *  Is_Weekend  +  -0.14255746902263944  *  Is_Jewish_Holiday  +  -0.02535976839183992  *  Is_Muslem_Holiday  +  0.0422700913522592  *  Is_Christian_Holiday  +  2.7755575615628914e-17  *  Is_Reserve  +  0.0  *  Is_Jewish_Site  +  -2.7755575615628914e-17  *  Duration_Of_Visit  +  -0.07105906058991615  *  Winter  +  0.24629777833067848  *  Spring  +  -0.027354221088516213  *  Summer


In [399]:
fitted_scaled_visitors_Gamla_Touristpm25 = lm.predict(X_train_scaled_Gamla_Touristpm25)
fitted_visitors_Gamla_Touristpm25 = y_train_scaler_Gamla_Touristpm25.inverse_transform(fitted_scaled_visitors_Gamla_Touristpm25)
predicted_train_visitors_Touristpm25 = round(pd.Series(fitted_visitors_Gamla_Touristpm25[:,0], index= y_train_Gamla_Touristpm25.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Gamla_Touristpm25 = pd.merge(left=train_df_Gamla_Touristpm25, right=predicted_train_visitors_Touristpm25 , left_index=True, right_index=True)

#train_df_Gamla_Touristpm25.head(10)

In [400]:
fitted_scaled_visitors_Gamla_Touristpm25 = lm.predict(X_test_scaled_Gamla_Touristpm25)
fitted_visitors_Gamla_Touristpm25 = y_test_scaler_Gamla_Touristpm25.inverse_transform(fitted_scaled_visitors_Gamla_Touristpm25)
predicted_test_visitors_Touristpm25 = round(pd.Series(fitted_visitors_Gamla_Touristpm25[:,0], index= y_test_Gamla_Touristpm25.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Gamla_Touristpm25 = pd.merge(left=test_df_Gamla_Touristpm25, right=predicted_test_visitors_Touristpm25 , left_index=True, right_index=True)

#test_df_Gamla_Touristpm25.head(10)

In [401]:
from sklearn import metrics
print("std of the objective function: 48.59")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Gamla_Touristpm25.Tourist_Visitors, train_df_Gamla_Touristpm25.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Gamla_Touristpm25.Tourist_Visitors, test_df_Gamla_Touristpm25.predicted_test_Visitors)))

std of the objective function: 48.59

------ TRAIN DATA ------
RMSE: 45.59527051204166

------ TEST DATA ------
RMSE: 46.55685238054898


In [402]:
from sklearn.metrics import r2_score
r2_score(train_df_Gamla_Touristpm25.Tourist_Visitors, train_df_Gamla_Touristpm25.predicted_train_Visitors)

0.12616182057254988

In [403]:
#get R2 for test set
r2_score(test_df_Gamla_Touristpm25.Tourist_Visitors, test_df_Gamla_Touristpm25.predicted_test_Visitors)

0.06179272555826709

**Gamla Tourist no2**

In [404]:
GamlaTouristno2_df.std()

Tourist_Visitors        50.742154
Temp_C                   7.685817
no2                      5.026052
Is_Weekend               0.451843
Is_Jewish_Holiday        0.303273
Is_Muslem_Holiday        0.179935
Is_Christian_Holiday     0.135533
Is_Reserve               0.000000
Is_Jewish_Site           0.000000
Duration_Of_Visit        0.000000
Winter                   0.425578
Spring                   0.437133
Summer                   0.442987
dtype: float64

In [405]:
from sklearn.model_selection import train_test_split
y_Gamla_Touristno2 = GamlaTouristno2_df.Tourist_Visitors
X_Gamla_Touristno2 = GamlaTouristno2_df[correlated_GamlaTouristno2_df]

X_train_Gamla_Touristno2, X_test_Gamla_Touristno2, y_train_Gamla_Touristno2, y_test_Gamla_Touristno2 = train_test_split(X_Gamla_Touristno2, y_Gamla_Touristno2, test_size=0.3, random_state=1234)
train_df_Gamla_Touristno2 = pd.merge(left=X_train_Gamla_Touristno2, right=y_train_Gamla_Touristno2, left_index=True, right_index=True)
test_df_Gamla_Touristno2 = pd.merge(left=X_test_Gamla_Touristno2, right=y_test_Gamla_Touristno2, left_index=True, right_index=True)
#test_df_Gamla_Touristno2.head(10)

In [406]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Gamla_Touristno2 = StandardScaler()
x_test_scaler_Gamla_Touristno2 = StandardScaler()
y_train_scaler_Gamla_Touristno2 = StandardScaler()
y_test_scaler_Gamla_Touristno2 = StandardScaler()

X_train_scaled_Gamla_Touristno2 = x_train_scaler_Gamla_Touristno2.fit_transform(X_train_Gamla_Touristno2)
X_test_scaled_Gamla_Touristno2 = x_test_scaler_Gamla_Touristno2.fit_transform(X_test_Gamla_Touristno2)
y_train_scaled_Gamla_Touristno2 = y_train_scaler_Gamla_Touristno2.fit_transform(pd.DataFrame(y_train_Gamla_Touristno2))
y_test_scaled_Gamla_Touristno2 = y_test_scaler_Gamla_Touristno2.fit_transform(pd.DataFrame(y_test_Gamla_Touristno2))

In [407]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Gamla_Touristno2,y_train_scaled_Gamla_Touristno2)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Tourist Visitors = " ,intercept," + ",coeff[0]," * ",correlated_GamlaTouristno2_df[0],
      " + ",coeff[1]," * ",correlated_GamlaTouristno2_df[1]," + ",coeff[2]," * ",correlated_GamlaTouristno2_df[2]," + ",coeff[3]," * ",correlated_GamlaTouristno2_df[3]," + ",coeff[4]," * ",correlated_GamlaTouristno2_df[4]," + ",coeff[5]," * ",correlated_GamlaTouristno2_df[5]," + ",coeff[6]," * ",correlated_GamlaTouristno2_df[6]," + ",coeff[7]," * ",correlated_GamlaTouristno2_df[7]," + ",coeff[8]," * ",correlated_GamlaTouristno2_df[8]," + ",coeff[9]," * ",correlated_GamlaTouristno2_df[9]," + ",coeff[10]," * ",correlated_GamlaTouristno2_df[10]," + ",coeff[11]," * ",correlated_GamlaTouristno2_df[11])

Coefficients: 
 coeff = [-1.78191824e-01  3.05904171e-02 -1.17255857e-01 -1.13712713e-01
 -2.33267694e-02  1.18113310e-01  6.93889390e-18  0.00000000e+00
  0.00000000e+00 -1.45848498e-01  1.56387883e-01 -5.76595895e-02] ,  Intercept= 4.482225106863394e-17 

The regression equation is: Tourist Visitors =  4.482225106863394e-17  +  -0.17819182397258568  *  Temp_C  +  0.03059041709982025  *  no2  +  -0.11725585696229862  *  Is_Weekend  +  -0.11371271295255886  *  Is_Jewish_Holiday  +  -0.02332676936243205  *  Is_Muslem_Holiday  +  0.11811330959194098  *  Is_Christian_Holiday  +  6.938893903907228e-18  *  Is_Reserve  +  0.0  *  Is_Jewish_Site  +  0.0  *  Duration_Of_Visit  +  -0.14584849849970516  *  Winter  +  0.15638788323501154  *  Spring  +  -0.05765958950870048  *  Summer


In [408]:
fitted_scaled_visitors_Gamla_Touristno2 = lm.predict(X_train_scaled_Gamla_Touristno2)
fitted_visitors_Gamla_Touristno2 = y_train_scaler_Gamla_Touristno2.inverse_transform(fitted_scaled_visitors_Gamla_Touristno2)
predicted_train_visitors_Touristno2 = round(pd.Series(fitted_visitors_Gamla_Touristno2[:,0], index= y_train_Gamla_Touristno2.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Gamla_Touristno2 = pd.merge(left=train_df_Gamla_Touristno2, right=predicted_train_visitors_Touristno2 , left_index=True, right_index=True)

#train_df_Gamla_Touristno2.head(10)

In [409]:
fitted_scaled_visitors_Gamla_Touristno2 = lm.predict(X_test_scaled_Gamla_Touristno2)
fitted_visitors_Gamla_Touristno2 = y_test_scaler_Gamla_Touristno2.inverse_transform(fitted_scaled_visitors_Gamla_Touristno2)
predicted_test_visitors_Touristno2 = round(pd.Series(fitted_visitors_Gamla_Touristno2[:,0], index= y_test_Gamla_Touristno2.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Gamla_Touristno2 = pd.merge(left=test_df_Gamla_Touristno2, right=predicted_test_visitors_Touristno2 , left_index=True, right_index=True)

#test_df_Gamla_Touristno2.head(10)

In [410]:
from sklearn import metrics
print("std of the objective function: 50.74")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Gamla_Touristno2.Tourist_Visitors, train_df_Gamla_Touristno2.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Gamla_Touristno2.Tourist_Visitors, test_df_Gamla_Touristno2.predicted_test_Visitors)))

std of the objective function: 50.74

------ TRAIN DATA ------
RMSE: 48.80026343712759

------ TEST DATA ------
RMSE: 45.32508800819718


In [411]:
from sklearn.metrics import r2_score
r2_score(train_df_Gamla_Touristno2.Tourist_Visitors, train_df_Gamla_Touristno2.predicted_train_Visitors)

0.11384781711658809

In [412]:
#get R2 for test set
r2_score(test_df_Gamla_Touristno2.Tourist_Visitors, test_df_Gamla_Touristno2.predicted_test_Visitors)

0.10600286870851516

### **Metzada**

**Metzada Israeli pm25**

In [413]:
MetzadaIsraelipm25_df.std()

Israeli_Visitors        855.517398
Temp_C                    7.864350
pm25                     20.921156
Is_Weekend                0.451041
Is_Jewish_Holiday         0.306312
Is_Muslem_Holiday         0.179424
Is_Christian_Holiday      0.141935
Is_National_Park          0.000000
Is_Jewish_Site            0.000000
Camping                   0.000000
Duration_Of_Visit         0.000000
Winter                    0.437382
Spring                    0.441638
Summer                    0.436289
dtype: float64

In [414]:
from sklearn.model_selection import train_test_split
y_Metzada_Israelipm25 = MetzadaIsraelipm25_df.Israeli_Visitors
X_Metzada_Israelipm25 = MetzadaIsraelipm25_df[correlated_MetzadaIsraelipm25_df]

X_train_Metzada_Israelipm25, X_test_Metzada_Israelipm25, y_train_Metzada_Israelipm25, y_test_Metzada_Israelipm25 = train_test_split(X_Metzada_Israelipm25, y_Metzada_Israelipm25, test_size=0.3, random_state=1234)
train_df_Metzada_Israelipm25 = pd.merge(left=X_train_Metzada_Israelipm25, right=y_train_Metzada_Israelipm25, left_index=True, right_index=True)
test_df_Metzada_Israelipm25 = pd.merge(left=X_test_Metzada_Israelipm25, right=y_test_Metzada_Israelipm25, left_index=True, right_index=True)
#test_df_Metzada_Israelipm25.head(10)

In [415]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Metzada_Israelipm25 = StandardScaler()
x_test_scaler_Metzada_Israelipm25 = StandardScaler()
y_train_scaler_Metzada_Israelipm25 = StandardScaler()
y_test_scaler_Metzada_Israelipm25 = StandardScaler()

X_train_scaled_Metzada_Israelipm25 = x_train_scaler_Metzada_Israelipm25.fit_transform(X_train_Metzada_Israelipm25)
X_test_scaled_Metzada_Israelipm25 = x_test_scaler_Metzada_Israelipm25.fit_transform(X_test_Metzada_Israelipm25)
y_train_scaled_Metzada_Israelipm25 = y_train_scaler_Metzada_Israelipm25.fit_transform(pd.DataFrame(y_train_Metzada_Israelipm25))
y_test_scaled_Metzada_Israelipm25 = y_test_scaler_Metzada_Israelipm25.fit_transform(pd.DataFrame(y_test_Metzada_Israelipm25))

In [416]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Metzada_Israelipm25,y_train_scaled_Metzada_Israelipm25)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_MetzadaIsraelipm25_df[0],
      " + ",coeff[1]," * ",correlated_MetzadaIsraelipm25_df[1]," + ",coeff[2]," * ",correlated_MetzadaIsraelipm25_df[2]," + ",coeff[3]," * ",correlated_MetzadaIsraelipm25_df[3]," + ",coeff[4]," * ",correlated_MetzadaIsraelipm25_df[4]," + ",coeff[5]," * ",correlated_MetzadaIsraelipm25_df[5]," + ",coeff[6]," * ",correlated_MetzadaIsraelipm25_df[6]," + ",coeff[7]," * ",correlated_MetzadaIsraelipm25_df[7]," + ",coeff[8]," * ",correlated_MetzadaIsraelipm25_df[8]," + ",coeff[9]," * ",correlated_MetzadaIsraelipm25_df[9]," + ",coeff[10]," * ",correlated_MetzadaIsraelipm25_df[10]," + ",coeff[11]," * ",correlated_MetzadaIsraelipm25_df[11]," + ",coeff[12]," * ",correlated_MetzadaIsraelipm25_df[12])

Coefficients: 
 coeff = [-3.18624099e-01  3.15714119e-03 -7.05055991e-02  5.55311980e-02
  1.22418508e-02  5.07037210e-02 -6.93889390e-18  1.38777878e-17
  3.46944695e-18  0.00000000e+00 -3.26546824e-01  1.62535102e-01
 -2.76335713e-01] ,  Intercept= -3.251727138478969e-17 

The regression equation is: Israeli Visitors =  -3.251727138478969e-17  +  -0.3186240986624511  *  Temp_C  +  0.003157141189537062  *  pm25  +  -0.0705055990625575  *  Is_Weekend  +  0.055531198000253426  *  Is_Jewish_Holiday  +  0.012241850810471694  *  Is_Muslem_Holiday  +  0.05070372096037583  *  Is_Christian_Holiday  +  -6.938893903907228e-18  *  Is_National_Park  +  1.3877787807814457e-17  *  Is_Jewish_Site  +  3.469446951953614e-18  *  Camping  +  0.0  *  Duration_Of_Visit  +  -0.32654682385997846  *  Winter  +  0.16253510236988628  *  Spring  +  -0.2763357129582461  *  Summer


In [417]:
fitted_scaled_visitors_Metzada_Israelipm25 = lm.predict(X_train_scaled_Metzada_Israelipm25)
fitted_visitors_Metzada_Israelipm25 = y_train_scaler_Metzada_Israelipm25.inverse_transform(fitted_scaled_visitors_Metzada_Israelipm25)
predicted_train_visitors_Israelipm25 = round(pd.Series(fitted_visitors_Metzada_Israelipm25[:,0], index= y_train_Metzada_Israelipm25.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Metzada_Israelipm25 = pd.merge(left=train_df_Metzada_Israelipm25, right=predicted_train_visitors_Israelipm25 , left_index=True, right_index=True)

#train_df_Metzada_Israelipm25.head(10)

In [418]:
fitted_scaled_visitors_Metzada_Israelipm25 = lm.predict(X_test_scaled_Metzada_Israelipm25)
fitted_visitors_Metzada_Israelipm25 = y_test_scaler_Metzada_Israelipm25.inverse_transform(fitted_scaled_visitors_Metzada_Israelipm25)
predicted_test_visitors_Israelipm25 = round(pd.Series(fitted_visitors_Metzada_Israelipm25[:,0], index= y_test_Metzada_Israelipm25.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Metzada_Israelipm25 = pd.merge(left=test_df_Metzada_Israelipm25, right=predicted_test_visitors_Israelipm25 , left_index=True, right_index=True)

#test_df_Metzada_Israelipm25.head(10)

In [419]:
from sklearn import metrics
print("std of the objective function: 855.51")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Metzada_Israelipm25.Israeli_Visitors, train_df_Metzada_Israelipm25.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Metzada_Israelipm25.Israeli_Visitors, test_df_Metzada_Israelipm25.predicted_test_Visitors)))

std of the objective function: 855.51

------ TRAIN DATA ------
RMSE: 716.6509598167672

------ TEST DATA ------
RMSE: 749.9033717175533


In [420]:
from sklearn.metrics import r2_score
r2_score(train_df_Metzada_Israelipm25.Israeli_Visitors, train_df_Metzada_Israelipm25.predicted_train_Visitors)

0.29970177923585173

In [421]:
#get R2 for test set
r2_score(test_df_Metzada_Israelipm25.Israeli_Visitors, test_df_Metzada_Israelipm25.predicted_test_Visitors)

0.22427975173760828

**Metzada Israeli pm10**

In [422]:
MetzadaIsraelipm10_df.std()

Israeli_Visitors        852.165802
Temp_C                    7.858730
pm10                     27.147154
Is_Weekend                0.452977
Is_Jewish_Holiday         0.305227
Is_Muslem_Holiday         0.180364
Is_Christian_Holiday      0.142689
Is_National_Park          0.000000
Is_Jewish_Site            0.000000
Camping                   0.000000
Duration_Of_Visit         0.000000
Winter                    0.437291
Spring                    0.440537
Summer                    0.435068
dtype: float64

In [423]:
from sklearn.model_selection import train_test_split
y_Metzada_Israelipm10 = MetzadaIsraelipm10_df.Israeli_Visitors
X_Metzada_Israelipm10 = MetzadaIsraelipm10_df[correlated_MetzadaIsraelipm10_df]

X_train_Metzada_Israelipm10, X_test_Metzada_Israelipm10, y_train_Metzada_Israelipm10, y_test_Metzada_Israelipm10 = train_test_split(X_Metzada_Israelipm10, y_Metzada_Israelipm10, test_size=0.3, random_state=1234)
train_df_Metzada_Israelipm10 = pd.merge(left=X_train_Metzada_Israelipm10, right=y_train_Metzada_Israelipm10, left_index=True, right_index=True)
test_df_Metzada_Israelipm10 = pd.merge(left=X_test_Metzada_Israelipm10, right=y_test_Metzada_Israelipm10, left_index=True, right_index=True)
#test_df_Metzada_Israelipm10.head(10)

In [424]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Metzada_Israelipm10 = StandardScaler()
x_test_scaler_Metzada_Israelipm10 = StandardScaler()
y_train_scaler_Metzada_Israelipm10 = StandardScaler()
y_test_scaler_Metzada_Israelipm10 = StandardScaler()

X_train_scaled_Metzada_Israelipm10 = x_train_scaler_Metzada_Israelipm10.fit_transform(X_train_Metzada_Israelipm10)
X_test_scaled_Metzada_Israelipm10 = x_test_scaler_Metzada_Israelipm10.fit_transform(X_test_Metzada_Israelipm10)
y_train_scaled_Metzada_Israelipm10 = y_train_scaler_Metzada_Israelipm10.fit_transform(pd.DataFrame(y_train_Metzada_Israelipm10))
y_test_scaled_Metzada_Israelipm10 = y_test_scaler_Metzada_Israelipm10.fit_transform(pd.DataFrame(y_test_Metzada_Israelipm10))

In [425]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Metzada_Israelipm10,y_train_scaled_Metzada_Israelipm10)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_MetzadaIsraelipm10_df[0],
      " + ",coeff[1]," * ",correlated_MetzadaIsraelipm10_df[1]," + ",coeff[2]," * ",correlated_MetzadaIsraelipm10_df[2]," + ",coeff[3]," * ",correlated_MetzadaIsraelipm10_df[3]," + ",coeff[4]," * ",correlated_MetzadaIsraelipm10_df[4]," + ",coeff[5]," * ",correlated_MetzadaIsraelipm10_df[5]," + ",coeff[6]," * ",correlated_MetzadaIsraelipm10_df[6]," + ",coeff[7]," * ",correlated_MetzadaIsraelipm10_df[7]," + ",coeff[8]," * ",correlated_MetzadaIsraelipm10_df[8]," + ",coeff[9]," * ",correlated_MetzadaIsraelipm10_df[9]," + ",coeff[10]," * ",correlated_MetzadaIsraelipm10_df[10]," + ",coeff[11]," * ",correlated_MetzadaIsraelipm10_df[11]," + ",coeff[12]," * ",correlated_MetzadaIsraelipm10_df[12])

Coefficients: 
 coeff = [-3.24260942e-01  4.35966992e-02 -6.24908068e-02  1.37653675e-02
  6.16480080e-03  3.18039095e-02  0.00000000e+00 -2.77555756e-17
 -2.77555756e-17  0.00000000e+00 -3.93661025e-01  1.32461344e-01
 -2.88126799e-01] ,  Intercept= 1.7211561932216256e-17 

The regression equation is: Israeli Visitors =  1.7211561932216256e-17  +  -0.32426094150172663  *  Temp_C  +  0.04359669918570311  *  pm10  +  -0.062490806797004285  *  Is_Weekend  +  0.013765367534091301  *  Is_Jewish_Holiday  +  0.00616480079512269  *  Is_Muslem_Holiday  +  0.03180390947083193  *  Is_Christian_Holiday  +  0.0  *  Is_National_Park  +  -2.7755575615628914e-17  *  Is_Jewish_Site  +  -2.7755575615628914e-17  *  Camping  +  0.0  *  Duration_Of_Visit  +  -0.3936610250553905  *  Winter  +  0.1324613441956559  *  Spring  +  -0.2881267994346251  *  Summer


In [426]:
fitted_scaled_visitors_Metzada_Israelipm10 = lm.predict(X_train_scaled_Metzada_Israelipm10)
fitted_visitors_Metzada_Israelipm10 = y_train_scaler_Metzada_Israelipm10.inverse_transform(fitted_scaled_visitors_Metzada_Israelipm10)
predicted_train_visitors_Israelipm10 = round(pd.Series(fitted_visitors_Metzada_Israelipm10[:,0], index= y_train_Metzada_Israelipm10.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Metzada_Israelipm10 = pd.merge(left=train_df_Metzada_Israelipm10, right=predicted_train_visitors_Israelipm10 , left_index=True, right_index=True)

#train_df_Metzada_Israelipm10.head(10)

In [427]:
fitted_scaled_visitors_Metzada_Israelipm10 = lm.predict(X_test_scaled_Metzada_Israelipm10)
fitted_visitors_Metzada_Israelipm10 = y_test_scaler_Metzada_Israelipm10.inverse_transform(fitted_scaled_visitors_Metzada_Israelipm10)
predicted_test_visitors_Israelipm10 = round(pd.Series(fitted_visitors_Metzada_Israelipm10[:,0], index= y_test_Metzada_Israelipm10.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Metzada_Israelipm10 = pd.merge(left=test_df_Metzada_Israelipm10, right=predicted_test_visitors_Israelipm10 , left_index=True, right_index=True)

#test_df_Metzada_Israelipm10.head(10)

In [428]:
from sklearn import metrics
print("std of the objective function: 852.16")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Metzada_Israelipm10.Israeli_Visitors, train_df_Metzada_Israelipm10.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Metzada_Israelipm10.Israeli_Visitors, test_df_Metzada_Israelipm10.predicted_test_Visitors)))

std of the objective function: 852.16

------ TRAIN DATA ------
RMSE: 716.1344646409183

------ TEST DATA ------
RMSE: 744.4132073167641


In [429]:
from sklearn.metrics import r2_score
r2_score(train_df_Metzada_Israelipm10.Israeli_Visitors, train_df_Metzada_Israelipm10.predicted_train_Visitors)

0.2845834467343883

In [430]:
#get R2 for test set
r2_score(test_df_Metzada_Israelipm10.Israeli_Visitors, test_df_Metzada_Israelipm10.predicted_test_Visitors)

0.2560209822484595

**Metzada Israeli no2**

In [431]:
MetzadaIsraelino2_df.std()

Israeli_Visitors        863.970252
Temp_C                    7.866354
no2                       3.725900
Is_Weekend                0.450850
Is_Jewish_Holiday         0.295708
Is_Muslem_Holiday         0.183403
Is_Christian_Holiday      0.134369
Is_National_Park          0.000000
Is_Jewish_Site            0.000000
Camping                   0.000000
Duration_Of_Visit         0.000000
Winter                    0.436793
Spring                    0.435707
Summer                    0.434610
dtype: float64

In [432]:
from sklearn.model_selection import train_test_split
y_Metzada_Israelino2 = MetzadaIsraelino2_df.Israeli_Visitors
X_Metzada_Israelino2 = MetzadaIsraelino2_df[correlated_MetzadaIsraelino2_df]

X_train_Metzada_Israelino2, X_test_Metzada_Israelino2, y_train_Metzada_Israelino2, y_test_Metzada_Israelino2 = train_test_split(X_Metzada_Israelino2, y_Metzada_Israelino2, test_size=0.3, random_state=1234)
train_df_Metzada_Israelino2 = pd.merge(left=X_train_Metzada_Israelino2, right=y_train_Metzada_Israelino2, left_index=True, right_index=True)
test_df_Metzada_Israelino2 = pd.merge(left=X_test_Metzada_Israelino2, right=y_test_Metzada_Israelino2, left_index=True, right_index=True)
#test_df_Metzada_Israelino2.head(10)

In [433]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Metzada_Israelino2 = StandardScaler()
x_test_scaler_Metzada_Israelino2 = StandardScaler()
y_train_scaler_Metzada_Israelino2 = StandardScaler()
y_test_scaler_Metzada_Israelino2 = StandardScaler()

X_train_scaled_Metzada_Israelino2 = x_train_scaler_Metzada_Israelino2.fit_transform(X_train_Metzada_Israelino2)
X_test_scaled_Metzada_Israelino2 = x_test_scaler_Metzada_Israelino2.fit_transform(X_test_Metzada_Israelino2)
y_train_scaled_Metzada_Israelino2 = y_train_scaler_Metzada_Israelino2.fit_transform(pd.DataFrame(y_train_Metzada_Israelino2))
y_test_scaled_Metzada_Israelino2 = y_test_scaler_Metzada_Israelino2.fit_transform(pd.DataFrame(y_test_Metzada_Israelino2))

In [434]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Metzada_Israelino2,y_train_scaled_Metzada_Israelino2)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_MetzadaIsraelino2_df[0],
      " + ",coeff[1]," * ",correlated_MetzadaIsraelino2_df[1]," + ",coeff[2]," * ",correlated_MetzadaIsraelino2_df[2]," + ",coeff[3]," * ",correlated_MetzadaIsraelino2_df[3]," + ",coeff[4]," * ",correlated_MetzadaIsraelino2_df[4]," + ",coeff[5]," * ",correlated_MetzadaIsraelino2_df[5]," + ",coeff[6]," * ",correlated_MetzadaIsraelino2_df[6]," + ",coeff[7]," * ",correlated_MetzadaIsraelino2_df[7]," + ",coeff[8]," * ",correlated_MetzadaIsraelino2_df[8]," + ",coeff[9]," * ",correlated_MetzadaIsraelino2_df[9]," + ",coeff[10]," * ",correlated_MetzadaIsraelino2_df[10]," + ",coeff[11]," * ",correlated_MetzadaIsraelino2_df[11]," + ",coeff[12]," * ",correlated_MetzadaIsraelino2_df[12])

Coefficients: 
 coeff = [-2.08441196e-01  1.56347710e-01 -2.33529996e-02  6.40292391e-02
 -1.67968139e-02  4.24392762e-02  0.00000000e+00 -1.73472348e-17
 -2.77555756e-17  0.00000000e+00 -3.00941091e-01  1.62904665e-01
 -2.51857886e-01] ,  Intercept= 2.1903708410640619e-16 

The regression equation is: Israeli Visitors =  2.1903708410640619e-16  +  -0.20844119637156008  *  Temp_C  +  0.15634770995995423  *  no2  +  -0.02335299963241681  *  Is_Weekend  +  0.06402923908271074  *  Is_Jewish_Holiday  +  -0.016796813886848526  *  Is_Muslem_Holiday  +  0.0424392761885205  *  Is_Christian_Holiday  +  0.0  *  Is_National_Park  +  -1.734723475976807e-17  *  Is_Jewish_Site  +  -2.7755575615628914e-17  *  Camping  +  0.0  *  Duration_Of_Visit  +  -0.3009410912253287  *  Winter  +  0.16290466495349518  *  Spring  +  -0.251857885718183  *  Summer


In [435]:
fitted_scaled_visitors_Metzada_Israelino2 = lm.predict(X_train_scaled_Metzada_Israelino2)
fitted_visitors_Metzada_Israelino2 = y_train_scaler_Metzada_Israelino2.inverse_transform(fitted_scaled_visitors_Metzada_Israelino2)
predicted_train_visitors_Israelino2 = round(pd.Series(fitted_visitors_Metzada_Israelino2[:,0], index= y_train_Metzada_Israelino2.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Metzada_Israelino2 = pd.merge(left=train_df_Metzada_Israelino2, right=predicted_train_visitors_Israelino2 , left_index=True, right_index=True)

#train_df_Metzada_Israelino2.head(10)

In [436]:
fitted_scaled_visitors_Metzada_Israelino2 = lm.predict(X_test_scaled_Metzada_Israelino2)
fitted_visitors_Metzada_Israelino2 = y_test_scaler_Metzada_Israelino2.inverse_transform(fitted_scaled_visitors_Metzada_Israelino2)
predicted_test_visitors_Israelino2 = round(pd.Series(fitted_visitors_Metzada_Israelino2[:,0], index= y_test_Metzada_Israelino2.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Metzada_Israelino2 = pd.merge(left=test_df_Metzada_Israelino2, right=predicted_test_visitors_Israelino2 , left_index=True, right_index=True)

#test_df_Metzada_Israelino2.head(10)

In [437]:
from sklearn import metrics
print("std of the objective function: 863.97")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Metzada_Israelino2.Israeli_Visitors, train_df_Metzada_Israelino2.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Metzada_Israelino2.Israeli_Visitors, test_df_Metzada_Israelino2.predicted_test_Visitors)))

std of the objective function: 863.97

------ TRAIN DATA ------
RMSE: 731.2968211552761

------ TEST DATA ------
RMSE: 732.01848729861


In [438]:
from sklearn.metrics import r2_score
r2_score(train_df_Metzada_Israelino2.Israeli_Visitors, train_df_Metzada_Israelino2.predicted_train_Visitors)

0.2795569024271868

In [439]:
#get R2 for test set
r2_score(test_df_Metzada_Israelino2.Israeli_Visitors, test_df_Metzada_Israelino2.predicted_test_Visitors)

0.2876515661274084

**Metzada Tourist pm25**

In [440]:
MetzadaTouristpm25_df.std()

Tourist_Visitors        922.222155
Temp_C                    7.864350
pm25                     20.921156
Is_Weekend                0.451041
Is_Jewish_Holiday         0.306312
Is_Muslem_Holiday         0.179424
Is_Christian_Holiday      0.141935
Is_National_Park          0.000000
Is_Jewish_Site            0.000000
Camping                   0.000000
Duration_Of_Visit         0.000000
Winter                    0.437382
Spring                    0.441638
Summer                    0.436289
dtype: float64

In [441]:
from sklearn.model_selection import train_test_split
y_Metzada_Touristpm25 = MetzadaTouristpm25_df.Tourist_Visitors
X_Metzada_Touristpm25 = MetzadaTouristpm25_df[correlated_MetzadaTouristpm25_df]

X_train_Metzada_Touristpm25, X_test_Metzada_Touristpm25, y_train_Metzada_Touristpm25, y_test_Metzada_Touristpm25 = train_test_split(X_Metzada_Touristpm25, y_Metzada_Touristpm25, test_size=0.3, random_state=1234)
train_df_Metzada_Touristpm25 = pd.merge(left=X_train_Metzada_Touristpm25, right=y_train_Metzada_Touristpm25, left_index=True, right_index=True)
test_df_Metzada_Touristpm25 = pd.merge(left=X_test_Metzada_Touristpm25, right=y_test_Metzada_Touristpm25, left_index=True, right_index=True)
#test_df_Metzada_Touristpm25.head(10)

In [442]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Metzada_Touristpm25 = StandardScaler()
x_test_scaler_Metzada_Touristpm25 = StandardScaler()
y_train_scaler_Metzada_Touristpm25 = StandardScaler()
y_test_scaler_Metzada_Touristpm25 = StandardScaler()

X_train_scaled_Metzada_Touristpm25 = x_train_scaler_Metzada_Touristpm25.fit_transform(X_train_Metzada_Touristpm25)
X_test_scaled_Metzada_Touristpm25 = x_test_scaler_Metzada_Touristpm25.fit_transform(X_test_Metzada_Touristpm25)
y_train_scaled_Metzada_Touristpm25 = y_train_scaler_Metzada_Touristpm25.fit_transform(pd.DataFrame(y_train_Metzada_Touristpm25))
y_test_scaled_Metzada_Touristpm25 = y_test_scaler_Metzada_Touristpm25.fit_transform(pd.DataFrame(y_test_Metzada_Touristpm25))

In [443]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Metzada_Touristpm25,y_train_scaled_Metzada_Touristpm25)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Tourist Visitors = " ,intercept," + ",coeff[0]," * ",correlated_MetzadaTouristpm25_df[0],
      " + ",coeff[1]," * ",correlated_MetzadaTouristpm25_df[1]," + ",coeff[2]," * ",correlated_MetzadaTouristpm25_df[2]," + ",coeff[3]," * ",correlated_MetzadaTouristpm25_df[3]," + ",coeff[4]," * ",correlated_MetzadaTouristpm25_df[4]," + ",coeff[5]," * ",correlated_MetzadaTouristpm25_df[5]," + ",coeff[6]," * ",correlated_MetzadaTouristpm25_df[6]," + ",coeff[7]," * ",correlated_MetzadaTouristpm25_df[7]," + ",coeff[8]," * ",correlated_MetzadaTouristpm25_df[8]," + ",coeff[9]," * ",correlated_MetzadaTouristpm25_df[9]," + ",coeff[10]," * ",correlated_MetzadaTouristpm25_df[10]," + ",coeff[11]," * ",correlated_MetzadaTouristpm25_df[11]," + ",coeff[12]," * ",correlated_MetzadaTouristpm25_df[12])

Coefficients: 
 coeff = [-1.29972653e-01 -3.27337770e-02  3.07064517e-02 -1.35446849e-01
 -2.67021003e-03  1.16525557e-01 -1.38777878e-17 -6.93889390e-18
  0.00000000e+00  0.00000000e+00 -3.91311727e-01  4.45956499e-02
 -2.51215350e-01] ,  Intercept= -1.207582539763835e-16 

The regression equation is: Tourist Visitors =  -1.207582539763835e-16  +  -0.12997265287187665  *  Temp_C  +  -0.032733777026252964  *  pm25  +  0.03070645169226194  *  Is_Weekend  +  -0.13544684898261455  *  Is_Jewish_Holiday  +  -0.0026702100299166986  *  Is_Muslem_Holiday  +  0.11652555730882436  *  Is_Christian_Holiday  +  -1.3877787807814457e-17  *  Is_National_Park  +  -6.938893903907228e-18  *  Is_Jewish_Site  +  0.0  *  Camping  +  0.0  *  Duration_Of_Visit  +  -0.39131172722608587  *  Winter  +  0.04459564990874224  *  Spring  +  -0.25121534987309324  *  Summer


In [444]:
fitted_scaled_visitors_Metzada_Touristpm25 = lm.predict(X_train_scaled_Metzada_Touristpm25)
fitted_visitors_Metzada_Touristpm25 = y_train_scaler_Metzada_Touristpm25.inverse_transform(fitted_scaled_visitors_Metzada_Touristpm25)
predicted_train_visitors_Touristpm25 = round(pd.Series(fitted_visitors_Metzada_Touristpm25[:,0], index= y_train_Metzada_Touristpm25.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Metzada_Touristpm25 = pd.merge(left=train_df_Metzada_Touristpm25, right=predicted_train_visitors_Touristpm25 , left_index=True, right_index=True)

#train_df_Metzada_Touristpm25.head(10)

In [445]:
fitted_scaled_visitors_Metzada_Touristpm25 = lm.predict(X_test_scaled_Metzada_Touristpm25)
fitted_visitors_Metzada_Touristpm25 = y_test_scaler_Metzada_Touristpm25.inverse_transform(fitted_scaled_visitors_Metzada_Touristpm25)
predicted_test_visitors_Touristpm25 = round(pd.Series(fitted_visitors_Metzada_Touristpm25[:,0], index= y_test_Metzada_Touristpm25.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Metzada_Touristpm25 = pd.merge(left=test_df_Metzada_Touristpm25, right=predicted_test_visitors_Touristpm25 , left_index=True, right_index=True)

#test_df_Metzada_Touristpm25.head(10)

In [446]:
from sklearn import metrics
print("std of the objective function: 922.22")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Metzada_Touristpm25.Tourist_Visitors, train_df_Metzada_Touristpm25.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Metzada_Touristpm25.Tourist_Visitors, test_df_Metzada_Touristpm25.predicted_test_Visitors)))

std of the objective function: 922.22

------ TRAIN DATA ------
RMSE: 833.2626183167321

------ TEST DATA ------
RMSE: 839.4225343566063


In [447]:
from sklearn.metrics import r2_score
r2_score(train_df_Metzada_Touristpm25.Tourist_Visitors, train_df_Metzada_Touristpm25.predicted_train_Visitors)

0.16914542719356696

In [448]:
#get R2 for test set
r2_score(test_df_Metzada_Touristpm25.Tourist_Visitors, test_df_Metzada_Touristpm25.predicted_test_Visitors)

0.19859301222092907

**Metzada Tourist pm10**

In [449]:
MetzadaTouristpm10_df.std()

Tourist_Visitors        929.343095
Temp_C                    7.858730
pm10                     27.147154
Is_Weekend                0.452977
Is_Jewish_Holiday         0.305227
Is_Muslem_Holiday         0.180364
Is_Christian_Holiday      0.142689
Is_National_Park          0.000000
Is_Jewish_Site            0.000000
Camping                   0.000000
Duration_Of_Visit         0.000000
Winter                    0.437291
Spring                    0.440537
Summer                    0.435068
dtype: float64

In [450]:
from sklearn.model_selection import train_test_split
y_Metzada_Touristpm10 = MetzadaTouristpm10_df.Tourist_Visitors
X_Metzada_Touristpm10 = MetzadaTouristpm10_df[correlated_MetzadaTouristpm10_df]

X_train_Metzada_Touristpm10, X_test_Metzada_Touristpm10, y_train_Metzada_Touristpm10, y_test_Metzada_Touristpm10 = train_test_split(X_Metzada_Touristpm10, y_Metzada_Touristpm10, test_size=0.3, random_state=1234)
train_df_Metzada_Touristpm10 = pd.merge(left=X_train_Metzada_Touristpm10, right=y_train_Metzada_Touristpm10, left_index=True, right_index=True)
test_df_Metzada_Touristpm10 = pd.merge(left=X_test_Metzada_Touristpm10, right=y_test_Metzada_Touristpm10, left_index=True, right_index=True)
#test_df_Metzada_Touristpm10.head(10)

In [451]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Metzada_Touristpm10 = StandardScaler()
x_test_scaler_Metzada_Touristpm10 = StandardScaler()
y_train_scaler_Metzada_Touristpm10 = StandardScaler()
y_test_scaler_Metzada_Touristpm10 = StandardScaler()

X_train_scaled_Metzada_Touristpm10 = x_train_scaler_Metzada_Touristpm10.fit_transform(X_train_Metzada_Touristpm10)
X_test_scaled_Metzada_Touristpm10 = x_test_scaler_Metzada_Touristpm10.fit_transform(X_test_Metzada_Touristpm10)
y_train_scaled_Metzada_Touristpm10 = y_train_scaler_Metzada_Touristpm10.fit_transform(pd.DataFrame(y_train_Metzada_Touristpm10))
y_test_scaled_Metzada_Touristpm10 = y_test_scaler_Metzada_Touristpm10.fit_transform(pd.DataFrame(y_test_Metzada_Touristpm10))

In [452]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Metzada_Touristpm10,y_train_scaled_Metzada_Touristpm10)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Tourist Visitors = " ,intercept," + ",coeff[0]," * ",correlated_MetzadaTouristpm10_df[0],
      " + ",coeff[1]," * ",correlated_MetzadaTouristpm10_df[1]," + ",coeff[2]," * ",correlated_MetzadaTouristpm10_df[2]," + ",coeff[3]," * ",correlated_MetzadaTouristpm10_df[3]," + ",coeff[4]," * ",correlated_MetzadaTouristpm10_df[4]," + ",coeff[5]," * ",correlated_MetzadaTouristpm10_df[5]," + ",coeff[6]," * ",correlated_MetzadaTouristpm10_df[6]," + ",coeff[7]," * ",correlated_MetzadaTouristpm10_df[7]," + ",coeff[8]," * ",correlated_MetzadaTouristpm10_df[8]," + ",coeff[9]," * ",correlated_MetzadaTouristpm10_df[9]," + ",coeff[10]," * ",correlated_MetzadaTouristpm10_df[10]," + ",coeff[11]," * ",correlated_MetzadaTouristpm10_df[11]," + ",coeff[12]," * ",correlated_MetzadaTouristpm10_df[12])

Coefficients: 
 coeff = [-1.61462673e-01  8.13428688e-02  5.38671870e-02 -1.62743251e-01
 -2.36792387e-02  1.15596883e-01  6.93889390e-18 -5.55111512e-17
  0.00000000e+00  0.00000000e+00 -5.34938153e-01 -2.95274275e-02
 -2.92543988e-01] ,  Intercept= 9.99882672429608e-17 

The regression equation is: Tourist Visitors =  9.99882672429608e-17  +  -0.16146267272782797  *  Temp_C  +  0.0813428687744883  *  pm10  +  0.05386718701890445  *  Is_Weekend  +  -0.1627432511194499  *  Is_Jewish_Holiday  +  -0.02367923874406289  *  Is_Muslem_Holiday  +  0.11559688316433012  *  Is_Christian_Holiday  +  6.938893903907228e-18  *  Is_National_Park  +  -5.551115123125783e-17  *  Is_Jewish_Site  +  0.0  *  Camping  +  0.0  *  Duration_Of_Visit  +  -0.5349381533574757  *  Winter  +  -0.029527427464745563  *  Spring  +  -0.29254398831276274  *  Summer


In [453]:
fitted_scaled_visitors_Metzada_Touristpm10 = lm.predict(X_train_scaled_Metzada_Touristpm10)
fitted_visitors_Metzada_Touristpm10 = y_train_scaler_Metzada_Touristpm10.inverse_transform(fitted_scaled_visitors_Metzada_Touristpm10)
predicted_train_visitors_Touristpm10 = round(pd.Series(fitted_visitors_Metzada_Touristpm10[:,0], index= y_train_Metzada_Touristpm10.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Metzada_Touristpm10 = pd.merge(left=train_df_Metzada_Touristpm10, right=predicted_train_visitors_Touristpm10 , left_index=True, right_index=True)

#train_df_Metzada_Touristpm10.head(10)

In [454]:
fitted_scaled_visitors_Metzada_Touristpm10 = lm.predict(X_test_scaled_Metzada_Touristpm10)
fitted_visitors_Metzada_Touristpm10 = y_test_scaler_Metzada_Touristpm10.inverse_transform(fitted_scaled_visitors_Metzada_Touristpm10)
predicted_test_visitors_Touristpm10 = round(pd.Series(fitted_visitors_Metzada_Touristpm10[:,0], index= y_test_Metzada_Touristpm10.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Metzada_Touristpm10 = pd.merge(left=test_df_Metzada_Touristpm10, right=predicted_test_visitors_Touristpm10 , left_index=True, right_index=True)

#test_df_Metzada_Touristpm10.head(10)

In [455]:
from sklearn import metrics
print("std of the objective function: 929.34")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Metzada_Touristpm10.Tourist_Visitors, train_df_Metzada_Touristpm10.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Metzada_Touristpm10.Tourist_Visitors, test_df_Metzada_Touristpm10.predicted_test_Visitors)))

std of the objective function: 929.34

------ TRAIN DATA ------
RMSE: 822.8305197137614

------ TEST DATA ------
RMSE: 861.9000230097135


In [456]:
from sklearn.metrics import r2_score
r2_score(train_df_Metzada_Touristpm10.Tourist_Visitors, train_df_Metzada_Touristpm10.predicted_train_Visitors)

0.22224214322666658

In [457]:
#get R2 for test set
r2_score(test_df_Metzada_Touristpm10.Tourist_Visitors, test_df_Metzada_Touristpm10.predicted_test_Visitors)

0.1204991931137891

**Metzada Tourist no2**

In [458]:
MetzadaTouristno2_df.std()

Tourist_Visitors        935.571038
Temp_C                    7.866354
no2                       3.725900
Is_Weekend                0.450850
Is_Jewish_Holiday         0.295708
Is_Muslem_Holiday         0.183403
Is_Christian_Holiday      0.134369
Is_National_Park          0.000000
Is_Jewish_Site            0.000000
Camping                   0.000000
Duration_Of_Visit         0.000000
Winter                    0.436793
Spring                    0.435707
Summer                    0.434610
dtype: float64

In [459]:
from sklearn.model_selection import train_test_split
y_Metzada_Touristno2 = MetzadaTouristno2_df.Tourist_Visitors
X_Metzada_Touristno2 = MetzadaTouristno2_df[correlated_MetzadaTouristno2_df]

X_train_Metzada_Touristno2, X_test_Metzada_Touristno2, y_train_Metzada_Touristno2, y_test_Metzada_Touristno2 = train_test_split(X_Metzada_Touristno2, y_Metzada_Touristno2, test_size=0.3, random_state=1234)
train_df_Metzada_Touristno2 = pd.merge(left=X_train_Metzada_Touristno2, right=y_train_Metzada_Touristno2, left_index=True, right_index=True)
test_df_Metzada_Touristno2 = pd.merge(left=X_test_Metzada_Touristno2, right=y_test_Metzada_Touristno2, left_index=True, right_index=True)
#test_df_Metzada_Touristno2.head(10)

In [460]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_Metzada_Touristno2 = StandardScaler()
x_test_scaler_Metzada_Touristno2 = StandardScaler()
y_train_scaler_Metzada_Touristno2 = StandardScaler()
y_test_scaler_Metzada_Touristno2 = StandardScaler()

X_train_scaled_Metzada_Touristno2 = x_train_scaler_Metzada_Touristno2.fit_transform(X_train_Metzada_Touristno2)
X_test_scaled_Metzada_Touristno2 = x_test_scaler_Metzada_Touristno2.fit_transform(X_test_Metzada_Touristno2)
y_train_scaled_Metzada_Touristno2 = y_train_scaler_Metzada_Touristno2.fit_transform(pd.DataFrame(y_train_Metzada_Touristno2))
y_test_scaled_Metzada_Touristno2 = y_test_scaler_Metzada_Touristno2.fit_transform(pd.DataFrame(y_test_Metzada_Touristno2))

In [461]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_Metzada_Touristno2,y_train_scaled_Metzada_Touristno2)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Tourist Visitors = " ,intercept," + ",coeff[0]," * ",correlated_MetzadaTouristno2_df[0],
      " + ",coeff[1]," * ",correlated_MetzadaTouristno2_df[1]," + ",coeff[2]," * ",correlated_MetzadaTouristno2_df[2]," + ",coeff[3]," * ",correlated_MetzadaTouristno2_df[3]," + ",coeff[4]," * ",correlated_MetzadaTouristno2_df[4]," + ",coeff[5]," * ",correlated_MetzadaTouristno2_df[5]," + ",coeff[6]," * ",correlated_MetzadaTouristno2_df[6]," + ",coeff[7]," * ",correlated_MetzadaTouristno2_df[7]," + ",coeff[8]," * ",correlated_MetzadaTouristno2_df[8]," + ",coeff[9]," * ",correlated_MetzadaTouristno2_df[9]," + ",coeff[10]," * ",correlated_MetzadaTouristno2_df[10]," + ",coeff[11]," * ",correlated_MetzadaTouristno2_df[11]," + ",coeff[12]," * ",correlated_MetzadaTouristno2_df[12])

Coefficients: 
 coeff = [-1.08895553e-01  7.02559844e-02  5.48524903e-02 -1.29429566e-01
  2.09031303e-03  1.33111246e-01  4.85722573e-17 -1.38777878e-17
  0.00000000e+00  0.00000000e+00 -4.23521329e-01  1.62108655e-02
 -2.66869540e-01] ,  Intercept= -3.0355266835597042e-18 

The regression equation is: Tourist Visitors =  -3.0355266835597042e-18  +  -0.10889555256384967  *  Temp_C  +  0.07025598441804924  *  no2  +  0.05485249025275463  *  Is_Weekend  +  -0.12942956592898816  *  Is_Jewish_Holiday  +  0.0020903130330667333  *  Is_Muslem_Holiday  +  0.1331112462880438  *  Is_Christian_Holiday  +  4.85722573273506e-17  *  Is_National_Park  +  -1.3877787807814457e-17  *  Is_Jewish_Site  +  0.0  *  Camping  +  0.0  *  Duration_Of_Visit  +  -0.4235213292566059  *  Winter  +  0.016210865471610266  *  Spring  +  -0.266869540447213  *  Summer


In [462]:
fitted_scaled_visitors_Metzada_Touristno2 = lm.predict(X_train_scaled_Metzada_Touristno2)
fitted_visitors_Metzada_Touristno2 = y_train_scaler_Metzada_Touristno2.inverse_transform(fitted_scaled_visitors_Metzada_Touristno2)
predicted_train_visitors_Touristno2 = round(pd.Series(fitted_visitors_Metzada_Touristno2[:,0], index= y_train_Metzada_Touristno2.index, name='predicted_train_Visitors'), ndigits =2)
train_df_Metzada_Touristno2 = pd.merge(left=train_df_Metzada_Touristno2, right=predicted_train_visitors_Touristno2 , left_index=True, right_index=True)

#train_df_Metzada_Touristno2.head(10)

In [463]:
fitted_scaled_visitors_Metzada_Touristno2 = lm.predict(X_test_scaled_Metzada_Touristno2)
fitted_visitors_Metzada_Touristno2 = y_test_scaler_Metzada_Touristno2.inverse_transform(fitted_scaled_visitors_Metzada_Touristno2)
predicted_test_visitors_Touristno2 = round(pd.Series(fitted_visitors_Metzada_Touristno2[:,0], index= y_test_Metzada_Touristno2.index, name='predicted_test_Visitors'), ndigits =2)
test_df_Metzada_Touristno2 = pd.merge(left=test_df_Metzada_Touristno2, right=predicted_test_visitors_Touristno2 , left_index=True, right_index=True)

#test_df_Metzada_Touristno2.head(10)

In [464]:
from sklearn import metrics
print("std of the objective function: 935.57")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_Metzada_Touristno2.Tourist_Visitors, train_df_Metzada_Touristno2.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_Metzada_Touristno2.Tourist_Visitors, test_df_Metzada_Touristno2.predicted_test_Visitors)))

std of the objective function: 935.57

------ TRAIN DATA ------
RMSE: 854.8078122271324

------ TEST DATA ------
RMSE: 817.9161259468618


In [465]:
from sklearn.metrics import r2_score
r2_score(train_df_Metzada_Touristno2.Tourist_Visitors, train_df_Metzada_Touristno2.predicted_train_Visitors)

0.17913195229645607

In [466]:
#get R2 for test set
r2_score(test_df_Metzada_Touristno2.Tourist_Visitors, test_df_Metzada_Touristno2.predicted_test_Visitors)

0.19323254918726196

### **Full Table Summer**

**Full Table Summer Israeli pm25**

In [467]:
#FullTableSummerIsraelipm25_df

In [468]:
FullTableSummerIsraelipm25_df.std()

Israeli_Visitors        2144.004215
Temp_C                     3.670908
pm25                      17.831070
Is_Weekend                 0.452837
Is_Jewish_Holiday          0.123960
Is_Muslem_Holiday          0.278313
Is_Christian_Holiday       0.000000
Is_water_area              0.484223
Is_National_Park           0.492982
Is_Reserve                 0.464182
Is_Jewish_Site             0.429826
Is_Christian_Site          0.401652
Camping                    0.413213
Duration_Of_Visit          0.602647
Summer                     0.000000
dtype: float64

In [469]:
from sklearn.model_selection import train_test_split
y_FullTableSummer_Israelipm25 = FullTableSummerIsraelipm25_df.Israeli_Visitors
X_FullTableSummer_Israelipm25 = FullTableSummerIsraelipm25_df[correlated_FullTableSummerIsraelipm25_df]

X_train_FullTableSummer_Israelipm25, X_test_FullTableSummer_Israelipm25, y_train_FullTableSummer_Israelipm25, y_test_FullTableSummer_Israelipm25 = train_test_split(X_FullTableSummer_Israelipm25, y_FullTableSummer_Israelipm25, test_size=0.2, random_state=1234)
train_df_FullTableSummer_Israelipm25 = pd.merge(left=X_train_FullTableSummer_Israelipm25, right=y_train_FullTableSummer_Israelipm25, left_index=True, right_index=True)
test_df_FullTableSummer_Israelipm25 = pd.merge(left=X_test_FullTableSummer_Israelipm25, right=y_test_FullTableSummer_Israelipm25, left_index=True, right_index=True)
#test_df_FullTableSummer_Israelipm25.head(10)

In [470]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_FullTableSummer_Israelipm25 = StandardScaler()
x_test_scaler_FullTableSummer_Israelipm25 = StandardScaler()
y_train_scaler_FullTableSummer_Israelipm25 = StandardScaler()
y_test_scaler_FullTableSummer_Israelipm25 = StandardScaler()

X_train_scaled_FullTableSummer_Israelipm25 = x_train_scaler_FullTableSummer_Israelipm25.fit_transform(X_train_FullTableSummer_Israelipm25)
X_test_scaled_FullTableSummer_Israelipm25 = x_test_scaler_FullTableSummer_Israelipm25.fit_transform(X_test_FullTableSummer_Israelipm25)
y_train_scaled_FullTableSummer_Israelipm25 = y_train_scaler_FullTableSummer_Israelipm25.fit_transform(pd.DataFrame(y_train_FullTableSummer_Israelipm25))
y_test_scaled_FullTableSummer_Israelipm25 = y_test_scaler_FullTableSummer_Israelipm25.fit_transform(pd.DataFrame(y_test_FullTableSummer_Israelipm25))

In [471]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_FullTableSummer_Israelipm25,y_train_scaled_FullTableSummer_Israelipm25)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_FullTableSummerIsraelipm25_df[0],
      " + ",coeff[1]," * ",correlated_FullTableSummerIsraelipm25_df[1]," + ",coeff[2]," * ",correlated_FullTableSummerIsraelipm25_df[2]," + ",coeff[3]," * ",correlated_FullTableSummerIsraelipm25_df[3]," + ",coeff[4]," * ",correlated_FullTableSummerIsraelipm25_df[4]," + ",coeff[5]," * ",correlated_FullTableSummerIsraelipm25_df[5]," + ",coeff[6]," * ",correlated_FullTableSummerIsraelipm25_df[6]," + ",coeff[7]," * ",correlated_FullTableSummerIsraelipm25_df[7]," + ",coeff[8]," * ",correlated_FullTableSummerIsraelipm25_df[8]," + ",coeff[9]," * ",correlated_FullTableSummerIsraelipm25_df[9]," + ",coeff[10]," * ",correlated_FullTableSummerIsraelipm25_df[10]," + ",coeff[11]," * ",correlated_FullTableSummerIsraelipm25_df[11]," + ",coeff[12]," * ",correlated_FullTableSummerIsraelipm25_df[12]," + ",coeff[13]," * ",correlated_FullTableSummerIsraelipm25_df[13])

Coefficients: 
 coeff = [-8.62139149e-03  8.61093943e-03  3.95465585e-02  5.24063487e-03
  1.72052134e-02  1.38777878e-17  5.34265918e-02  4.74616664e-02
  2.33836260e-02  1.61481961e-02  2.44367832e-02 -1.52399044e-02
  1.37655478e-01  0.00000000e+00] ,  Intercept= -4.468648100952336e-17 

The regression equation is: Israeli Visitors =  -4.468648100952336e-17  +  -0.008621391493877868  *  Temp_C  +  0.008610939433493184  *  pm25  +  0.039546558508730394  *  Is_Weekend  +  0.005240634869416276  *  Is_Jewish_Holiday  +  0.0172052134076588  *  Is_Muslem_Holiday  +  1.3877787807814457e-17  *  Is_Christian_Holiday  +  0.05342659182240066  *  Is_water_area  +  0.04746166638192542  *  Is_National_Park  +  0.02338362595587014  *  Is_Reserve  +  0.016148196138174413  *  Is_Jewish_Site  +  0.0244367831596893  *  Is_Christian_Site  +  -0.015239904425966368  *  Camping  +  0.13765547779437617  *  Duration_Of_Visit  +  0.0  *  Summer


In [472]:
fitted_scaled_visitors_FullTableSummer_Israelipm25 = lm.predict(X_train_scaled_FullTableSummer_Israelipm25)
fitted_visitors_FullTableSummer_Israelipm25 = y_train_scaler_FullTableSummer_Israelipm25.inverse_transform(fitted_scaled_visitors_FullTableSummer_Israelipm25)
predicted_train_visitors_Israelipm25 = round(pd.Series(fitted_visitors_FullTableSummer_Israelipm25[:,0], index= y_train_FullTableSummer_Israelipm25.index, name='predicted_train_Visitors'), ndigits =2)
train_df_FullTableSummer_Israelipm25 = pd.merge(left=train_df_FullTableSummer_Israelipm25, right=predicted_train_visitors_Israelipm25 , left_index=True, right_index=True)

#train_df_FullTableSummer_Israelipm25.head(10)

In [473]:
fitted_scaled_visitors_FullTableSummer_Israelipm25 = lm.predict(X_test_scaled_FullTableSummer_Israelipm25)
fitted_visitors_FullTableSummer_Israelipm25 = y_test_scaler_FullTableSummer_Israelipm25.inverse_transform(fitted_scaled_visitors_FullTableSummer_Israelipm25)
predicted_test_visitors_Israelipm25 = round(pd.Series(fitted_visitors_FullTableSummer_Israelipm25[:,0], index= y_test_FullTableSummer_Israelipm25.index, name='predicted_test_Visitors'), ndigits =2)
test_df_FullTableSummer_Israelipm25 = pd.merge(left=test_df_FullTableSummer_Israelipm25, right=predicted_test_visitors_Israelipm25 , left_index=True, right_index=True)

#test_df_FullTableSummer_Israelipm25.head(10)

In [474]:
from sklearn import metrics
print("std of the objective function: 2144")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_FullTableSummer_Israelipm25.Israeli_Visitors, train_df_FullTableSummer_Israelipm25.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_FullTableSummer_Israelipm25.Israeli_Visitors, test_df_FullTableSummer_Israelipm25.predicted_test_Visitors)))

std of the objective function: 2144

------ TRAIN DATA ------
RMSE: 2115.7951806395727

------ TEST DATA ------
RMSE: 2094.4977462548027


In [475]:
from sklearn.metrics import r2_score
r2_score(train_df_FullTableSummer_Israelipm25.Israeli_Visitors, train_df_FullTableSummer_Israelipm25.predicted_train_Visitors)

0.030643017721868504

In [476]:
#get R2 for test set
r2_score(test_df_FullTableSummer_Israelipm25.Israeli_Visitors, test_df_FullTableSummer_Israelipm25.predicted_test_Visitors)

0.027267784338736334

**Full Table Summer Israeli no2**

In [477]:
#FullTableSummerIsraelino2_df

In [478]:
FullTableSummerIsraelino2_df.std()

Israeli_Visitors        2119.643628
Temp_C                     3.661016
no2                        8.529241
Is_Weekend                 0.452562
Is_Jewish_Holiday          0.122296
Is_Muslem_Holiday          0.279985
Is_Christian_Holiday       0.000000
Is_water_area              0.484632
Is_National_Park           0.493680
Is_Reserve                 0.467164
Is_Jewish_Site             0.430738
Is_Christian_Site          0.402048
Camping                    0.410754
Duration_Of_Visit          0.599501
Summer                     0.000000
dtype: float64

In [479]:
from sklearn.model_selection import train_test_split
y_FullTableSummer_Israelino2 = FullTableSummerIsraelino2_df.Israeli_Visitors
X_FullTableSummer_Israelino2 = FullTableSummerIsraelino2_df[correlated_FullTableSummerIsraelino2_df]

X_train_FullTableSummer_Israelino2, X_test_FullTableSummer_Israelino2, y_train_FullTableSummer_Israelino2, y_test_FullTableSummer_Israelino2 = train_test_split(X_FullTableSummer_Israelino2, y_FullTableSummer_Israelino2, test_size=0.3, random_state=1234)
train_df_FullTableSummer_Israelino2 = pd.merge(left=X_train_FullTableSummer_Israelino2, right=y_train_FullTableSummer_Israelino2, left_index=True, right_index=True)
test_df_FullTableSummer_Israelino2 = pd.merge(left=X_test_FullTableSummer_Israelino2, right=y_test_FullTableSummer_Israelino2, left_index=True, right_index=True)
#test_df_FullTableSummer_Israelino2.head(10)

In [480]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_FullTableSummer_Israelino2 = StandardScaler()
x_test_scaler_FullTableSummer_Israelino2 = StandardScaler()
y_train_scaler_FullTableSummer_Israelino2 = StandardScaler()
y_test_scaler_FullTableSummer_Israelino2 = StandardScaler()

X_train_scaled_FullTableSummer_Israelino2 = x_train_scaler_FullTableSummer_Israelino2.fit_transform(X_train_FullTableSummer_Israelino2)
X_test_scaled_FullTableSummer_Israelino2 = x_test_scaler_FullTableSummer_Israelino2.fit_transform(X_test_FullTableSummer_Israelino2)
y_train_scaled_FullTableSummer_Israelino2 = y_train_scaler_FullTableSummer_Israelino2.fit_transform(pd.DataFrame(y_train_FullTableSummer_Israelino2))
y_test_scaled_FullTableSummer_Israelino2 = y_test_scaler_FullTableSummer_Israelino2.fit_transform(pd.DataFrame(y_test_FullTableSummer_Israelino2))

In [481]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_FullTableSummer_Israelino2,y_train_scaled_FullTableSummer_Israelino2)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Israeli Visitors = " ,intercept," + ",coeff[0]," * ",correlated_FullTableSummerIsraelino2_df[0],
      " + ",coeff[1]," * ",correlated_FullTableSummerIsraelino2_df[1]," + ",coeff[2]," * ",correlated_FullTableSummerIsraelino2_df[2]," + ",coeff[3]," * ",correlated_FullTableSummerIsraelino2_df[3]," + ",coeff[4]," * ",correlated_FullTableSummerIsraelino2_df[4]," + ",coeff[5]," * ",correlated_FullTableSummerIsraelino2_df[5]," + ",coeff[6]," * ",correlated_FullTableSummerIsraelino2_df[6]," + ",coeff[7]," * ",correlated_FullTableSummerIsraelino2_df[7]," + ",coeff[8]," * ",correlated_FullTableSummerIsraelino2_df[8]," + ",coeff[9]," * ",correlated_FullTableSummerIsraelino2_df[9]," + ",coeff[10]," * ",correlated_FullTableSummerIsraelino2_df[10]," + ",coeff[11]," * ",correlated_FullTableSummerIsraelino2_df[11]," + ",coeff[12]," * ",correlated_FullTableSummerIsraelino2_df[12]," + ",coeff[13]," * ",correlated_FullTableSummerIsraelino2_df[13])

Coefficients: 
 coeff = [-6.62967953e-03  1.56170511e-02  3.56891016e-02  1.02587827e-02
  2.52415739e-02  1.38777878e-17  5.61142902e-02  5.25432461e-02
  3.55205152e-02  1.80425837e-02  2.77328069e-02 -1.36788793e-02
  1.42127521e-01  0.00000000e+00] ,  Intercept= -2.9294937903493314e-17 

The regression equation is: Israeli Visitors =  -2.9294937903493314e-17  +  -0.0066296795329551265  *  Temp_C  +  0.015617051053882114  *  no2  +  0.03568910162973592  *  Is_Weekend  +  0.010258782691703747  *  Is_Jewish_Holiday  +  0.025241573867630686  *  Is_Muslem_Holiday  +  1.3877787807814457e-17  *  Is_Christian_Holiday  +  0.0561142901849181  *  Is_water_area  +  0.0525432461344046  *  Is_National_Park  +  0.03552051524505388  *  Is_Reserve  +  0.018042583709224316  *  Is_Jewish_Site  +  0.027732806871678162  *  Is_Christian_Site  +  -0.013678879272415966  *  Camping  +  0.14212752089050992  *  Duration_Of_Visit  +  0.0  *  Summer


In [482]:
fitted_scaled_visitors_FullTableSummer_Israelino2 = lm.predict(X_train_scaled_FullTableSummer_Israelino2)
fitted_visitors_FullTableSummer_Israelino2 = y_train_scaler_FullTableSummer_Israelino2.inverse_transform(fitted_scaled_visitors_FullTableSummer_Israelino2)
predicted_train_visitors_Israelino2 = round(pd.Series(fitted_visitors_FullTableSummer_Israelino2[:,0], index= y_train_FullTableSummer_Israelino2.index, name='predicted_train_Visitors'), ndigits =2)
train_df_FullTableSummer_Israelino2 = pd.merge(left=train_df_FullTableSummer_Israelino2, right=predicted_train_visitors_Israelino2 , left_index=True, right_index=True)

#train_df_FullTableSummer_Israelino2.head(10)

In [483]:
fitted_scaled_visitors_FullTableSummer_Israelino2 = lm.predict(X_test_scaled_FullTableSummer_Israelino2)
fitted_visitors_FullTableSummer_Israelino2 = y_test_scaler_FullTableSummer_Israelino2.inverse_transform(fitted_scaled_visitors_FullTableSummer_Israelino2)
predicted_test_visitors_Israelino2 = round(pd.Series(fitted_visitors_FullTableSummer_Israelino2[:,0], index= y_test_FullTableSummer_Israelino2.index, name='predicted_test_Visitors'), ndigits =2)
test_df_FullTableSummer_Israelino2 = pd.merge(left=test_df_FullTableSummer_Israelino2, right=predicted_test_visitors_Israelino2 , left_index=True, right_index=True)

#test_df_FullTableSummer_Israelino2.head(10)

In [484]:
from sklearn import metrics
print("std of the objective function: 2119.64")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_FullTableSummer_Israelino2.Israeli_Visitors, train_df_FullTableSummer_Israelino2.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_FullTableSummer_Israelino2.Israeli_Visitors, test_df_FullTableSummer_Israelino2.predicted_test_Visitors)))

std of the objective function: 2119.64

------ TRAIN DATA ------
RMSE: 2014.3112260753376

------ TEST DATA ------
RMSE: 2247.3779580461687


In [485]:
from sklearn.metrics import r2_score
r2_score(train_df_FullTableSummer_Israelino2.Israeli_Visitors, train_df_FullTableSummer_Israelino2.predicted_train_Visitors)

0.03297618215117293

In [486]:
#get R2 for test set
r2_score(test_df_FullTableSummer_Israelino2.Israeli_Visitors, test_df_FullTableSummer_Israelino2.predicted_test_Visitors)

0.02589175354714357

**Full Table Summer Tourist pm25**

In [487]:
#FullTableSummerTouristpm25_df

In [488]:
FullTableSummerTouristpm25_df.std()

Tourist_Visitors        310.039977
Temp_C                    3.670908
pm25                     17.831070
Is_Weekend                0.452837
Is_Jewish_Holiday         0.123960
Is_Muslem_Holiday         0.278313
Is_Christian_Holiday      0.000000
Is_water_area             0.484223
Is_National_Park          0.492982
Is_Reserve                0.464182
Is_Jewish_Site            0.429826
Is_Christian_Site         0.401652
Camping                   0.413213
Duration_Of_Visit         0.602647
Summer                    0.000000
dtype: float64

In [489]:
from sklearn.model_selection import train_test_split
y_FullTableSummer_Touristpm25 = FullTableSummerTouristpm25_df.Tourist_Visitors
X_FullTableSummer_Touristpm25 = FullTableSummerTouristpm25_df[correlated_FullTableSummerTouristpm25_df]

X_train_FullTableSummer_Touristpm25, X_test_FullTableSummer_Touristpm25, y_train_FullTableSummer_Touristpm25, y_test_FullTableSummer_Touristpm25 = train_test_split(X_FullTableSummer_Touristpm25, y_FullTableSummer_Touristpm25, test_size=0.3, random_state=1234)
train_df_FullTableSummer_Touristpm25 = pd.merge(left=X_train_FullTableSummer_Touristpm25, right=y_train_FullTableSummer_Touristpm25, left_index=True, right_index=True)
test_df_FullTableSummer_Touristpm25 = pd.merge(left=X_test_FullTableSummer_Touristpm25, right=y_test_FullTableSummer_Touristpm25, left_index=True, right_index=True)
#test_df_FullTableSummer_Touristpm25.head(10)

In [490]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_FullTableSummer_Touristpm25 = StandardScaler()
x_test_scaler_FullTableSummer_Touristpm25 = StandardScaler()
y_train_scaler_FullTableSummer_Touristpm25 = StandardScaler()
y_test_scaler_FullTableSummer_Touristpm25 = StandardScaler()

X_train_scaled_FullTableSummer_Touristpm25 = x_train_scaler_FullTableSummer_Touristpm25.fit_transform(X_train_FullTableSummer_Touristpm25)
X_test_scaled_FullTableSummer_Touristpm25 = x_test_scaler_FullTableSummer_Touristpm25.fit_transform(X_test_FullTableSummer_Touristpm25)
y_train_scaled_FullTableSummer_Touristpm25 = y_train_scaler_FullTableSummer_Touristpm25.fit_transform(pd.DataFrame(y_train_FullTableSummer_Touristpm25))
y_test_scaled_FullTableSummer_Touristpm25 = y_test_scaler_FullTableSummer_Touristpm25.fit_transform(pd.DataFrame(y_test_FullTableSummer_Touristpm25))

In [491]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_FullTableSummer_Touristpm25,y_train_scaled_FullTableSummer_Touristpm25)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Tourist Visitors = " ,intercept," + ",coeff[0]," * ",correlated_FullTableSummerTouristpm25_df[0],
      " + ",coeff[1]," * ",correlated_FullTableSummerTouristpm25_df[1]," + ",coeff[2]," * ",correlated_FullTableSummerTouristpm25_df[2]," + ",coeff[3]," * ",correlated_FullTableSummerTouristpm25_df[3]," + ",coeff[4]," * ",correlated_FullTableSummerTouristpm25_df[4]," + ",coeff[5]," * ",correlated_FullTableSummerTouristpm25_df[5]," + ",coeff[6]," * ",correlated_FullTableSummerTouristpm25_df[6]," + ",coeff[7]," * ",correlated_FullTableSummerTouristpm25_df[7]," + ",coeff[8]," * ",correlated_FullTableSummerTouristpm25_df[8]," + ",coeff[9]," * ",correlated_FullTableSummerTouristpm25_df[9]," + ",coeff[10]," * ",correlated_FullTableSummerTouristpm25_df[10]," + ",coeff[11]," * ",correlated_FullTableSummerTouristpm25_df[11]," + ",coeff[12]," * ",correlated_FullTableSummerTouristpm25_df[12]," + ",coeff[13]," * ",correlated_FullTableSummerTouristpm25_df[13])

Coefficients: 
 coeff = [ 7.95898158e-02 -8.49038635e-02 -1.50630997e-02  7.28288942e-03
  1.19167060e-03  1.38777878e-16 -2.62219279e-01  1.45910093e-01
  3.19270603e-01  2.21066070e-01  3.26871477e-01  2.64052032e-02
  3.04605438e-01  0.00000000e+00] ,  Intercept= -1.7513508971994533e-16 

The regression equation is: Tourist Visitors =  -1.7513508971994533e-16  +  0.07958981584719516  *  Temp_C  +  -0.08490386354708745  *  pm25  +  -0.015063099677217695  *  Is_Weekend  +  0.007282889424667906  *  Is_Jewish_Holiday  +  0.001191670601806671  *  Is_Muslem_Holiday  +  1.3877787807814457e-16  *  Is_Christian_Holiday  +  -0.26221927853676563  *  Is_water_area  +  0.14591009341783634  *  Is_National_Park  +  0.31927060288857884  *  Is_Reserve  +  0.22106606951179084  *  Is_Jewish_Site  +  0.32687147749561285  *  Is_Christian_Site  +  0.026405203248392014  *  Camping  +  0.30460543776648885  *  Duration_Of_Visit  +  0.0  *  Summer


In [492]:
fitted_scaled_visitors_FullTableSummer_Touristpm25 = lm.predict(X_train_scaled_FullTableSummer_Touristpm25)
fitted_visitors_FullTableSummer_Touristpm25 = y_train_scaler_FullTableSummer_Touristpm25.inverse_transform(fitted_scaled_visitors_FullTableSummer_Touristpm25)
predicted_train_visitors_Touristpm25 = round(pd.Series(fitted_visitors_FullTableSummer_Touristpm25[:,0], index= y_train_FullTableSummer_Touristpm25.index, name='predicted_train_Visitors'), ndigits =2)
train_df_FullTableSummer_Touristpm25 = pd.merge(left=train_df_FullTableSummer_Touristpm25, right=predicted_train_visitors_Touristpm25 , left_index=True, right_index=True)

#train_df_FullTableSummer_Touristpm25.head(10)

In [493]:
fitted_scaled_visitors_FullTableSummer_Touristpm25 = lm.predict(X_test_scaled_FullTableSummer_Touristpm25)
fitted_visitors_FullTableSummer_Touristpm25 = y_test_scaler_FullTableSummer_Touristpm25.inverse_transform(fitted_scaled_visitors_FullTableSummer_Touristpm25)
predicted_test_visitors_Touristpm25 = round(pd.Series(fitted_visitors_FullTableSummer_Touristpm25[:,0], index= y_test_FullTableSummer_Touristpm25.index, name='predicted_test_Visitors'), ndigits =2)
test_df_FullTableSummer_Touristpm25 = pd.merge(left=test_df_FullTableSummer_Touristpm25, right=predicted_test_visitors_Touristpm25 , left_index=True, right_index=True)

#test_df_FullTableSummer_Touristpm25.head(10)

In [494]:
from sklearn import metrics
print("std of the objective function: 310.03")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_FullTableSummer_Touristpm25.Tourist_Visitors, train_df_FullTableSummer_Touristpm25.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_FullTableSummer_Touristpm25.Tourist_Visitors, test_df_FullTableSummer_Touristpm25.predicted_test_Visitors)))

std of the objective function: 310.03

------ TRAIN DATA ------
RMSE: 280.9353366061304

------ TEST DATA ------
RMSE: 279.2604888112902


In [495]:
from sklearn.metrics import r2_score
r2_score(train_df_FullTableSummer_Touristpm25.Tourist_Visitors, train_df_FullTableSummer_Touristpm25.predicted_train_Visitors)

0.17669894258273766

In [496]:
#get R2 for test set
r2_score(test_df_FullTableSummer_Touristpm25.Tourist_Visitors, test_df_FullTableSummer_Touristpm25.predicted_test_Visitors)

0.19359087751184378

**Full Table Summer Tourist no2**

In [497]:
#FullTableSummerTouristno2_df

In [498]:
FullTableSummerTouristno2_df.std()

Tourist_Visitors        305.820388
Temp_C                    3.661016
no2                       8.529241
Is_Weekend                0.452562
Is_Jewish_Holiday         0.122296
Is_Muslem_Holiday         0.279985
Is_Christian_Holiday      0.000000
Is_water_area             0.484632
Is_National_Park          0.493680
Is_Reserve                0.467164
Is_Jewish_Site            0.430738
Is_Christian_Site         0.402048
Camping                   0.410754
Duration_Of_Visit         0.599501
Summer                    0.000000
dtype: float64

In [499]:
from sklearn.model_selection import train_test_split
y_FullTableSummer_Touristno2 = FullTableSummerTouristno2_df.Tourist_Visitors
X_FullTableSummer_Touristno2 = FullTableSummerTouristno2_df[correlated_FullTableSummerTouristno2_df]

X_train_FullTableSummer_Touristno2, X_test_FullTableSummer_Touristno2, y_train_FullTableSummer_Touristno2, y_test_FullTableSummer_Touristno2 = train_test_split(X_FullTableSummer_Touristno2, y_FullTableSummer_Touristno2, test_size=0.3, random_state=1234)
train_df_FullTableSummer_Touristno2 = pd.merge(left=X_train_FullTableSummer_Touristno2, right=y_train_FullTableSummer_Touristno2, left_index=True, right_index=True)
test_df_FullTableSummer_Touristno2 = pd.merge(left=X_test_FullTableSummer_Touristno2, right=y_test_FullTableSummer_Touristno2, left_index=True, right_index=True)
#test_df_FullTableSummer_Touristno2.head(10)

In [500]:
from sklearn.preprocessing import StandardScaler

x_train_scaler_FullTableSummer_Touristno2 = StandardScaler()
x_test_scaler_FullTableSummer_Touristno2 = StandardScaler()
y_train_scaler_FullTableSummer_Touristno2 = StandardScaler()
y_test_scaler_FullTableSummer_Touristno2 = StandardScaler()

X_train_scaled_FullTableSummer_Touristno2 = x_train_scaler_FullTableSummer_Touristno2.fit_transform(X_train_FullTableSummer_Touristno2)
X_test_scaled_FullTableSummer_Touristno2 = x_test_scaler_FullTableSummer_Touristno2.fit_transform(X_test_FullTableSummer_Touristno2)
y_train_scaled_FullTableSummer_Touristno2 = y_train_scaler_FullTableSummer_Touristno2.fit_transform(pd.DataFrame(y_train_FullTableSummer_Touristno2))
y_test_scaled_FullTableSummer_Touristno2 = y_test_scaler_FullTableSummer_Touristno2.fit_transform(pd.DataFrame(y_test_FullTableSummer_Touristno2))

In [507]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_FullTableSummer_Touristno2,y_train_scaled_FullTableSummer_Touristno2)

coeff = lm.coef_[0]
intercept = lm.intercept_[0]
print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"Tourist Visitors = " ,intercept," + ",coeff[0]," * ",correlated_FullTableSummerTouristno2_df[0],
      " + ",coeff[1]," * ",correlated_FullTableSummerTouristno2_df[1]," + ",coeff[2]," * ",correlated_FullTableSummerTouristno2_df[2]," + ",coeff[3]," * ",correlated_FullTableSummerTouristno2_df[3]," + ",coeff[4]," * ",correlated_FullTableSummerTouristno2_df[4]," + ",coeff[5]," * ",correlated_FullTableSummerTouristno2_df[5]," + ",coeff[6]," * ",correlated_FullTableSummerTouristno2_df[6]," + ",coeff[7]," * ",correlated_FullTableSummerTouristno2_df[7]," + ",coeff[8]," * ",correlated_FullTableSummerTouristno2_df[8]," + ",coeff[9]," * ",correlated_FullTableSummerTouristno2_df[9]," + ",coeff[10]," * ",correlated_FullTableSummerTouristno2_df[10]," + ",coeff[11]," * ",correlated_FullTableSummerTouristno2_df[11]," + ",coeff[12]," * ",correlated_FullTableSummerTouristno2_df[12]," + ",coeff[13]," * ",correlated_FullTableSummerTouristno2_df[13])

Coefficients: 
 coeff = [ 4.66414277e-02 -8.49059213e-02 -2.55821173e-02  1.22921945e-02
  1.16582948e-02 -1.38777878e-17 -2.59732335e-01  1.28320407e-01
  3.16366873e-01  2.41444600e-01  3.31907483e-01  4.82040080e-02
  3.08075906e-01  0.00000000e+00] ,  Intercept= -1.3610511450194231e-16 

The regression equation is: Tourist Visitors =  -1.3610511450194231e-16  +  0.046641427729234304  *  Temp_C  +  -0.08490592129236137  *  no2  +  -0.0255821172852502  *  Is_Weekend  +  0.012292194472992672  *  Is_Jewish_Holiday  +  0.011658294809785329  *  Is_Muslem_Holiday  +  -1.3877787807814457e-17  *  Is_Christian_Holiday  +  -0.2597323354272429  *  Is_water_area  +  0.12832040661208782  *  Is_National_Park  +  0.3163668733085607  *  Is_Reserve  +  0.24144459990470338  *  Is_Jewish_Site  +  0.3319074826848118  *  Is_Christian_Site  +  0.04820400804115209  *  Camping  +  0.3080759058657815  *  Duration_Of_Visit  +  0.0  *  Summer


In [502]:
fitted_scaled_visitors_FullTableSummer_Touristno2 = lm.predict(X_train_scaled_FullTableSummer_Touristno2)
fitted_visitors_FullTableSummer_Touristno2 = y_train_scaler_FullTableSummer_Touristno2.inverse_transform(fitted_scaled_visitors_FullTableSummer_Touristno2)
predicted_train_visitors_Touristno2 = round(pd.Series(fitted_visitors_FullTableSummer_Touristno2[:,0], index= y_train_FullTableSummer_Touristno2.index, name='predicted_train_Visitors'), ndigits =2)
train_df_FullTableSummer_Touristno2 = pd.merge(left=train_df_FullTableSummer_Touristno2, right=predicted_train_visitors_Touristno2 , left_index=True, right_index=True)

#train_df_FullTableSummer_Touristno2.head(10)

In [503]:
fitted_scaled_visitors_FullTableSummer_Touristno2 = lm.predict(X_test_scaled_FullTableSummer_Touristno2)
fitted_visitors_FullTableSummer_Touristno2 = y_test_scaler_FullTableSummer_Touristno2.inverse_transform(fitted_scaled_visitors_FullTableSummer_Touristno2)
predicted_test_visitors_Touristno2 = round(pd.Series(fitted_visitors_FullTableSummer_Touristno2[:,0], index= y_test_FullTableSummer_Touristno2.index, name='predicted_test_Visitors'), ndigits =2)
test_df_FullTableSummer_Touristno2 = pd.merge(left=test_df_FullTableSummer_Touristno2, right=predicted_test_visitors_Touristno2 , left_index=True, right_index=True)

#test_df_FullTableSummer_Touristno2.head(10)

In [504]:
from sklearn import metrics
print("std of the objective function: 305.82")
print("")
print("------ TRAIN DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df_FullTableSummer_Touristno2.Tourist_Visitors, train_df_FullTableSummer_Touristno2.predicted_train_Visitors)))
print("")
print("------ TEST DATA ------")
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df_FullTableSummer_Touristno2.Tourist_Visitors, test_df_FullTableSummer_Touristno2.predicted_test_Visitors)))

std of the objective function: 305.82

------ TRAIN DATA ------
RMSE: 287.4742830287953

------ TEST DATA ------
RMSE: 250.4648003747029


In [505]:
from sklearn.metrics import r2_score
r2_score(train_df_FullTableSummer_Touristno2.Tourist_Visitors, train_df_FullTableSummer_Touristno2.predicted_train_Visitors)

0.17729107341651174

In [506]:
#get R2 for test set
r2_score(test_df_FullTableSummer_Touristno2.Tourist_Visitors, test_df_FullTableSummer_Touristno2.predicted_test_Visitors)

0.18790260188025543